In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("Spark version:", spark.version)
print("PySpark is working!")

Spark version: 4.0.1
PySpark is working!


In [4]:
import re

# Example text block containing titles with (genre)
titles_with_genres = """
Lincoln Letters (History/Politics, Non-Fiction)
Lincoln's First Inaugural Address (History/Politics, Speech/Non-Fiction)
Lincoln's Gettysburg Address, given November 19, 1863 (History/Politics, Speech/Non-Fiction)
Lincoln's Inaugurals, Addresses and Letters (Selections) (History/Politics, Non-Fiction Collection)
Lincoln's Second Inaugural Address (History/Politics, Speech/Non-Fiction)
Speeches and Letters of Abraham Lincoln, 1832-1865 (History/Politics, Non-Fiction Collection)
State of the Union Addresses (History/Politics, Non-Fiction)
The Emancipation Proclamation (History/Politics, Non-Fiction)
The Life and Public Service of General Zachary Taylor: An Address (Biography/History, Non-Fiction)
The Writings of Abraham Lincoln, Volume 1: 1832-1843 (History/Politics, Non-Fiction Collection)
The Writings of Abraham Lincoln, Volume 2: 1843-1858 (History/Politics, Non-Fiction Collection)
The Writings of Abraham Lincoln, Volume 3 (History/Politics, Non-Fiction Collection)
The Mysterious Affair at Styles (Mystery Fiction, Detective Novel)
The Secret Adversary (Mystery Fiction, Adventure Novel)
Relativity: The Special and General Theory (Science/Physics, Non-Fiction)
Sidelights on Relativity (Science/Physics, Non-Fiction)
Crome Yellow (Fiction, Satirical Novel)
Mortal Coils (Fiction, Short Stories)
An Essay on Criticism (Poetry, Literary Criticism)
Essay on Man (Poetry, Philosophical Verse)
The Poetical Works of Alexander Pope, Volume 1 (Poetry Collection)
The Poetical Works of Alexander Pope, Volume 2 (Poetry Collection)
The Rape of the Lock and Other Poems (Poetry, Satirical Verse)
The Works of Alexander Pope, Volume 1 (Poetry/Literary Collection)
Darwinism (Science/Biology, Non-Fiction Essays)
Is Mars Habitable? (Science/Astronomy, Non-Fiction)
Island Life (Science/Biology, Non-Fiction)
An Occurrence at Owl Creek Bridge (Fiction, Short Story/War)
Can Such Things Be? (Horror/Supernatural Fiction, Short Stories)
Cobwebs From an Empty Skull (Fiction, Satirical Fables)
Fantastic Fables (Fiction, Fables/Satire)
Present at a Hanging and Other Ghost Stories (Horror/Supernatural Fiction, Short Stories)
Shapes of Clay (Poetry Collection)
The Collected Works of Ambrose Bierce, Volume 1 (Fiction/Non-Fiction Collection, Satire)
The Collected Works of Ambrose Bierce, Volume 2 (Fiction/Non-Fiction Collection, Satire)
A Collection of Ballads (Poetry/Folklore Collection)
A Monk of Fife (Historical Fiction, Novel)
A Short History of Scotland (History, Non-Fiction)
Aaron Trow (Fiction, Short Story)
An Eye for an Eye (Fiction, Novel)
Autobiography of Anthony Trollope (Autobiography, Non-Fiction)
A Bride of the Plains (Historical Fiction, Romance Novel)
Castles in the Air (Fiction, Short Stories)
El Dorado (Historical Fiction, Adventure Novel)
A Collection of Beatrix Potter Stories (Children's Fiction, Animal Tales)
The Great Big Treasury of Beatrix Potter (Children's Fiction, Animal Tales Collection)
The Story of Miss Moppet (Children's Fiction, Picture Book)
The Tailor of Gloucester (Children's Fiction, Animal Tale)
The Tale of Ginger and Pickles (Children's Fiction, Animal Tale)
The Tale of Jemima Puddle-Duck (Children's Fiction, Animal Tale)
The Tale of Johnny Town-Mouse (Children's Fiction, Animal Tale)
The Tale of Mr. Tod (Children's Fiction, Animal Tale)
The Tale Of Peter Rabbit (Children's Fiction, Animal Tale)
The Tale of Samuel Whiskers (Children's Fiction, Animal Tale)
The Tale of the Flopsy Bunnies (Children's Fiction, Animal Tale)
The Tale of the Pie and the Patty Pan (Children's Fiction, Animal Tale)
The Tale of Tom Kitten (Children's Fiction, Animal Tale)
Alroy (Historical Fiction, Novel)
Coningsby (Fiction, Political Novel)
Endymion (Fiction, Political Novel)
Autobiography of Benjamin Franklin, Version 1 (Autobiography, Non-Fiction)
Autobiography of Benjamin Franklin, Version 2 (Autobiography, Non-Fiction)
Autobiography of Benjamin Franklin, Version 4 (Autobiography, Non-Fiction)
Mysticism and Logic and Other Essays (Philosophy/Essays, Non-Fiction)
Political Ideals (Philosophy/Politics, Non-Fiction)
Proposed Roads To Freedom (Philosophy/Politics, Non-Fiction)
The Analysis of Mind (Philosophy/Psychology, Non-Fiction)
The Practice and Theory of Bolshevism (History/Politics, Non-Fiction)
Dracula's Guest (Horror Fiction, Short Stories)
The Jewel of Seven Stars (Horror Fiction, Novel)
The Lady of the Shroud (Horror Fiction, Novel)
A First Family of Tasajara (Fiction, Western Novel)
A Millionaire of Rough-and-Ready (Fiction, Western Short Story)
A Phyllis of the Sierras (Fiction, Western Novel)
A Sappho of Green Springs (Fiction, Western Short Stories)
A Ward of the Golden Gate (Fiction, Western Novel)
A Monograph on the Sub-class Cirripedia (Volume 1 of 2) (Science/Biology, Non-Fiction)
A Naturalist's Voyage Round the World (Science/Travel, Non-Fiction)
A Child's History of England (History, Non-Fiction for Children)
A Christmas Carol (Fiction, Ghost Story/Novella)
A Tale of Two Cities (Historical Fiction, Novel)
Alexandria and her Schools (History/Philosophy, Non-Fiction)
All Saints' Day and Other Sermons (Religion/Sermons, Non-Fiction)
Andromeda and Other Poems (Poetry Collection)
At Last (Science/Travel, Non-Fiction)
Daily Thoughts (Essays/Philosophy, Non-Fiction)
Jane Eyre (Fiction, Gothic Romance Novel)
Shirley (Fiction, Social Novel)
A Book of Golden Deeds (History/Biography Collection, Non-Fiction)
A Modern Telemachus (Fiction, Adventure Novel)
A Reputed Changeling (Fiction, Historical Novel)
Abbeychurch (Fiction, Domestic Novel)
Bay (Poetry Collection)
England, My England (Fiction, Short Stories)
Fantasia of the Unconscious (Philosophy/Psychology, Non-Fiction)
Look! We Have Come Through! (Poetry Collection)
New Poems (Poetry Collection)
Sea and Sardinia (Travel/Non-Fiction)
Sons and Lovers (Fiction, Novel)
A Journal of the Plague Year (Historical Fiction, Novel)
A New Voyage Round the World by a Course Never Sailed Before (Fiction, Adventure Novel)
A True Relation of the Apparition of one Mrs. Veal (Fiction, Ghost Story)
A Vindication of the Press (Essays/Politics, Non-Fiction)
An Answer to a Question that Nobody thinks of, viz., But what if the Queen should Die? (Essays/Politics, Non-Fiction)
An Appeal to Honour and Justice, Though It Be of His Worst Enemies (Essays/Politics, Non-Fiction)
An Essay Upon Projects (Essays/Economics, Non-Fiction)
And What if the Pretender should Come? (Essays/Politics, Non-Fiction)
Atalantis Major (Fiction, Satirical Novel)
Augusta Triumphans (Essays/Social Commentary, Non-Fiction)
Edgar Allan Poe's Complete Poetical Works (Poetry Collection)
Eureka (Philosophy/Science, Non-Fiction Prose Poem)
The Cask of Amontillado (Horror Fiction, Short Story)
The Fall of the House of Usher (Horror Fiction, Short Story)
The Raven (Poetry)
The Works of Edgar Allan Poe Volume 1 (Fiction/Poetry Collection, Horror/Mystery)
The Works of Edgar Allan Poe Volume 3 (Fiction/Poetry Collection, Horror/Mystery)
A Princess of Mars (Science Fiction, Adventure Novel)
At the Earth's Core (Science Fiction, Adventure Novel)
Jungle Tales of Tarzan (Adventure Fiction, Short Stories)
Pellucidar (Science Fiction, Adventure Novel)
Tarzan and the Jewels of Opar (Adventure Fiction, Novel)
Tarzan of the Apes (Adventure Fiction, Novel)
Burke's Speech on Conciliation with America (History/Politics, Speech/Non-Fiction)
Selections from the Speeches and Writings of Edmund Burke (History/Politics, Non-Fiction Collection)
A Lost Leader (Poetry)
A Maker of History (Fiction, Spy Novel)
A Monk of Cruta (Fiction, Adventure Novel)
American Boy's Life of Theodore Roosevelt (Biography, Non-Fiction for Youth)
Boys of The Fort (Adventure Fiction, Youth Novel)
Dave Porter and His Double (Adventure Fiction, Youth Novel)
Dave Porter and the Runaways (Adventure Fiction, Youth Novel)
'He Giveth His Beloved Sleep' (Poetry)
Sonnets from the Portuguese (Poetry Collection)
The Poetical Works of Elizabeth Barrett Browning Volume I (Poetry Collection)
Wuthering Heights (Fiction, Gothic Romance Novel)
Anna Christie (Drama/Play)
The First Man (Drama/Play)
The Hairy Ape (Drama/Play)
The Straw (Drama/Play)
Certain Noble Plays of Japan (Drama/Plays, Translation)
Hugh Selwyn Mauberley (Poetry)
A Chosen Few (Fiction, Short Stories)
A Jolly Fellowship (Adventure Fiction, Youth Novel)
Amos Kilbright; His Adscititious Experiences (Fantasy Fiction, Short Stories)
Buccaneers and Pirates of Our Coasts (History/Adventure, Non-Fiction)
John Gayther's Garden and the Stories Told Therein (Fiction, Short Stories)
A Short History of England (History, Non-Fiction)
All Things Considered (Essays/Humor, Non-Fiction)
Appreciations and Criticisms of the Works of Charles Dickens (Literary Criticism, Non-Fiction)
George Bernard Shaw (Biography/Literary Criticism, Non-Fiction)
A Chapter of Adventures (Adventure Fiction, Youth Novel)
A Final Reckoning (Adventure Fiction, Novel)
A Girl of the Commune (Historical Fiction, Novel)
A Jacobite Exile (Historical Fiction, Novel)
A Treatise on Parents and Children (Essays/Education, Non-Fiction)
An Unsocial Socialist (Fiction, Satirical Novel)
Androcles and the Lion (Drama/Play)
Arms and the Man (Drama/Play)
Caesar and Cleopatra (Drama/Play)
Candida (Drama/Play)
Adam Bede (Fiction, Novel)
Brother Jacob (Fiction, Short Story)
Daniel Deronda (Fiction, Novel)
Charles Darwin (Biography, Non-Fiction)
Falling in Love (Science/Essays, Non-Fiction)
Hilda Wade (Mystery Fiction, Novel)
A Daughter of the Middle Border (Autobiography, Non-Fiction)
A Son of the Middle Border (Autobiography, Non-Fiction)
A Spoil of Office (Fiction, Political Novel)
A Prairie Courtship (Fiction, Romance Novel)
Blake's Burden (Fiction, Adventure Novel)
Household Papers and Stories (Essays/Fiction Collection)
Lady Byron Vindicated (Biography/Defense, Non-Fiction)
Oldtown Fireside Stories (Fiction, Short Stories/Folklore)
The Chronicles of Clovis (Fiction, Satirical Short Stories)
The Toys of Peace (Fiction, Satirical Short Stories)
The Unbearable Bassington (Fiction, Satirical Novel)
When William Came (Fiction, Satirical Novel)
A Plea for Captain John Brown (Essays/Politics, Non-Fiction)
Cape Cod (Travel/Nature, Non-Fiction)
Excursions (Essays/Nature, Non-Fiction)
On the Duty of Civil Disobedience (Philosophy/Politics, Non-Fiction Essay)
A Passionate Pilgrim (Fiction, Short Story)
A Little Tour in France, Version 1 (Travel, Non-Fiction)
A Little Tour in France, Version 2 (Travel, Non-Fiction)
A Small Boy and Others (Autobiography, Non-Fiction)
Allan and the Holy Flower (Adventure Fiction, Novel)
Allan's Wife (Adventure Fiction, Short Stories)
A Modern Utopia (Science Fiction, Utopian Novel)
A Short History of the World (History, Non-Fiction)
Ann Veronica (Fiction, Social Novel)
Essays on Education and Kindred Subjects (Essays/Education, Non-Fiction)
Essays: Scientific, Political, & Speculative, Volume I (Essays/Science/Politics, Non-Fiction)
The Right To Ignore The State (Philosophy/Politics, Non-Fiction)
Battle-Pieces and Aspects of the War (Poetry Collection)
I and My Chimney (Fiction, Short Story)
Israel Potter (Historical Fiction, Novel)
John Marr and Other Poems (Poetry Collection)
Mardi: and A Voyage Thither, Volume 1 (Fiction, Allegorical Novel)
Howard Pyle's Book of Pirates (Adventure Fiction, Illustrated Stories)
Men of Iron (Historical Fiction, Youth Novel)
Pepper & Salt (Children's Fiction, Fairy Tales)
Stolen Treasure (Adventure Fiction, Short Stories)
The Merry Adventures of Robin Hood (Adventure Fiction, Novel/Folklore)
The Rose of Paradise (Adventure Fiction, Novel)
Youth (Fiction, Novella)
A Daughter of the Snows (Adventure Fiction, Novel)
Brown Wolf and Other Jack London Stories (Fiction, Short Stories)
Burning Daylight (Adventure Fiction, Novel)
Alexander the Great (Biography/History, Non-Fiction)
Caleb in the Country (Children's Fiction, Moral Tale)
Cleopatra (Biography/History, Non-Fiction)
Forests of Maine (Children's Fiction, Nature Tale)
Genghis Khan (Biography/History, Non-Fiction)
Goblin Tales of Lancashire (Folklore/Fiction, Short Stories)
Afloat And Ashore (Adventure Fiction, Novel)
Autobiography of a Pocket-Hankerchief (Fiction, Short Story)
A Portrait of the Artist as a Young Man (Fiction, Bildungsroman Novel)
Chamber Music (Poetry Collection)
Dubliners (Fiction, Short Stories)
A Holiday in Bed and Other Sketches (Essays/Humor, Non-Fiction)
A Window in Thrums (Fiction, Novel)
Alice Sit-By-The-Fire (Drama/Play)
Auld Licht Idylls (Fiction, Short Stories)
Courage (Essays/Inspirational, Non-Fiction)
Dear Brutus (Drama/Play)
Margaret Ogilvy (Biography, Non-Fiction)
My Lady Nicotine (Essays/Humor, Non-Fiction)
A District Messenger Boy and a Necktie Party (Adventure Fiction, Youth Stories)
An Amateur Fireman (Adventure Fiction, Youth Novel)
Aunt Hannah and Seth (Fiction, Youth Novel)
Commodore Barney's Young Spies (Historical Fiction, Youth Novel)
Corporal 'Lige's Recruit (Historical Fiction, Youth Novel)
Defending the Island (Adventure Fiction, Youth Novel)
Dick in the Desert (Adventure Fiction, Youth Novel)
Among My Books, First Series (Literary Criticism, Non-Fiction)
Among My Books, Second Series (Literary Criticism, Non-Fiction)
My Garden Acquaintance (Essays/Nature, Non-Fiction)
Poems of James Russell Lowell (Poetry Collection)
Emma (Fiction, Romance Novel)
Lady Susan (Fiction, Epistolary Novel)
Love And Freindship And Other Early Works (Fiction, Satirical Stories)
Mansfield Park (Fiction, Romance Novel)
All Roads Lead to Calvary (Fiction, Novel)
Evergreens (Poetry Collection)
Fanny and the Servant Problem (Drama/Play)
Idle Thoughts of an Idle Fellow (Essays/Humor, Non-Fiction)
John Ingerfield and Other Stories (Fiction, Short Stories)
Mrs. Korner Sins Her Mercies (Fiction, Short Story)
Novel Notes (Fiction, Humorous Novel)
The Heavenly Footman (Religion/Allegory, Non-Fiction)
The Holy War (Religion/Allegory, Fiction)
The Jerusalem Sinner Saved (Religion/Sermon, Non-Fiction)
The Pharisee And The Publican (Religion/Sermon, Non-Fiction)
The Pilgrim's Progress (Religion/Allegory, Fiction)
Discourses on Satire and Epic Poetry (Literary Criticism, Non-Fiction)
Dryden's Works Vol. 16 (of 18) (Poetry/Drama Collection)
Dryden's Works Vol. 2 (of 18) (Poetry/Drama Collection)
A Family Man (Drama/Play)
Essays Concerning Letters (Essays/Literary, Non-Fiction)
Essays on Censorship and Art (Essays/Politics, Non-Fiction)
Five Tales (Fiction, Short Stories)
Fraternity (Fiction, Social Novel)
Keats: Poems Published in 1820 (Poetry Collection)
Lamia (Poetry)
Letters of John Keats to His Family and Friends (Biography/Letters, Non-Fiction)
Poems 1817 (Poetry Collection)
An Essay Concerning Humane Understanding, Volume 1 (Philosophy, Non-Fiction)
An Essay Concerning Humane Understanding, Volume 2 (Philosophy, Non-Fiction)
Second Treatise of Government (Philosophy/Politics, Non-Fiction)
The Economic Consequences of the Peace (Economics/History, Non-Fiction)
L'Allegro, Il Penseroso, Comus, and Lycidas (Poetry Collection)
Milton's Comus (Poetry/Drama)
Minor Poems by Milton (Poetry Collection)
Paradise Lost (Poetry, Epic)
Paradise Regained (Poetry, Epic)
Burke (Biography, Non-Fiction)
Critical Miscellanies, Volume 1, Essay 1, Robespierre (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 1, Essay 2, Carlyle (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 1, Essay 3, Byron (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 1, Essay 4, Macaulay (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 2, Essay 1, Vauvenargues (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 2, Essay 2, Turgot (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 2, Essay 3, Condorcet (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 2, Essay 4, Joseph de Maistre (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 1, On Popular Culture (Literary Criticism/Essays, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 10, Auguste Comte (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 4, The Life of George Eliot (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 5, On Pattison's Memoirs (Literary Criticism/Biography, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 7, A Sketch (Literary Criticism/Essays, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 8, France in the Eighteenth Century (Literary Criticism/History, Non-Fiction)
Critical Miscellanies, Volume 3, Essay 9, The Expansion of England (Literary Criticism/History, Non-Fiction)
Ariadne Florentina (Art Criticism, Non-Fiction)
Hortus Inclusus (Letters/Nature, Non-Fiction)
Lectures on Architecture and Painting (Art Criticism, Non-Fiction)
A Tale of a Tub (Fiction, Satire)
Gulliver's Travels (Fiction, Satirical Adventure Novel)
The Battle of the Books (Fiction, Satire)
The Journal to Stella (Letters/Biography, Non-Fiction)
A Personal Record (Autobiography, Non-Fiction)
A Set of Six (Fiction, Short Stories)
Almayer's Folly (Fiction, Adventure Novel)
Amy Foster (Fiction, Short Story)
An Outcast of the Islands (Fiction, Adventure Novel)
Captain Sword and Captain Pen (Poetry, Satirical)
Stories from the Italian Poets: With Lives of the Writers, Volume 1 (Poetry/Biography, Non-Fiction)
A Tangled Tale (Fiction, Mathematical Puzzles)
Eight or Nine Wise Words about Letter-Writing (Essays/Humor, Non-Fiction)
Rhyme? And Reason? (Poetry/Humor Collection)
The Game of Logic (Philosophy/Mathematics, Non-Fiction)
The Hunting of the Snark (Poetry, Nonsense Verse)
Through the Looking-Glass (Children's Fiction, Fantasy Novel)
Childe Harold's Pilgrimage (Poetry, Epic)
Don Juan (Poetry, Epic Satire)
Fugitive Pieces (Poetry Collection)
Becket and other plays (Drama/Plays)
Enoch Arden, &c (Poetry Collection)
Idylls of the King (Poetry, Epic)
Lady Clare (Poetry)
Queen Mary and Harold (Drama/Plays)
The Princess (Poetry)
The Suppressed Poems of Alfred Lord Tennyson (Poetry Collection)
Aunt Jo's Scrap-Bag (Fiction, Short Stories for Youth)
Behind A Mask, Or A Woman's Power (Fiction, Gothic Novella)
Anne Of Avonlea (Fiction, Youth Novel)
Anne Of Green Gables (Fiction, Youth Novel)
Anne Of The Island (Fiction, Youth Novel)
Anne's House of Dreams (Fiction, Youth Novel)
American Fairy Tales (Children's Fiction, Fairy Tales)
Aunt Jane's Nieces Abroad (Adventure Fiction, Youth Novel)
Aunt Jane's Nieces and Uncle John (Adventure Fiction, Youth Novel)
Aunt Jane's Nieces at Millville (Adventure Fiction, Youth Novel)
Aunt Jane's Nieces at Work (Adventure Fiction, Youth Novel)
1601 (Fiction, Satirical Short Story)
A Connecticut Yankee in King Arthur's Court, Complete (Fiction, Satirical Fantasy Novel)
A Dog's Tale (Fiction, Animal Story)
A Tramp Abroad (Travel/Humor, Non-Fiction)
Adventures of Huckleberry Finn, Complete (Adventure Fiction, Novel)
Frankenstein (Science Fiction, Gothic Novel)
Mathilda (Fiction, Gothic Novella)
Notes to the Complete Poetical Works of Percy Bysshe Shelley (Literary Criticism, Non-Fiction)
Proserpine and Midas (Drama/Mythological Plays)
The Last Man (Science Fiction, Apocalyptic Novel)
Mariposilla (Fiction, Novel)
Experimental Researches in Electricity, Volume 1 (Science/Physics, Non-Fiction)
A Book of Autographs (Essays/Literary, Non-Fiction)
A Rill from the Town Pump (From "Twice Told Tales") (Fiction, Short Story)
A Select Party (From "Mosses From An Old Manse") (Fiction, Short Story)
A Virtuoso's Collection (From "Mosses From An Old Manse") (Fiction, Short Story)
A Wonder Book and Tanglewood Tales (Children's Fiction, Mythology Retellings)
A Wonder Book for Girls & Boys (Children's Fiction, Mythology Retellings)
An Old Woman's Tale (Fiction, Short Story)
Buds and Bird Voices (From "Mosses From An Old Manse") (Essays/Nature, Non-Fiction)
Chippings With A Chisel (From "Twice Told Tales") (Fiction, Short Story)
Doctor Grimshawe's Secret (Fiction, Novel Fragment)
Dr. Bullivant (Fiction, Short Story)
Earth's Holocaust (From "Mosses From An Old Manse") (Fiction, Allegorical Short Story)
Edward Fane's Rosebud (From "Twice Told Tales") (Fiction, Short Story)
Fancy's Show-Box (From "Twice Told Tales") (Fiction, Short Story)
Fanshawe (Fiction, Novel)
Heart of the West (Fiction, Western Short Stories)
Options (Fiction, Short Stories)
A Critic in Pall Mall (Literary Criticism, Non-Fiction)
A House of Pomegranates (Children's Fiction, Fairy Tales)
A Woman of No Importance (Drama/Play)
An Ideal Husband (Drama/Play)
Essays and Lectures (Essays/Literary, Non-Fiction)
Intentions (Essays/Literary, Non-Fiction)
Lord Arthur Savile's Crime and Other Stories (Fiction, Short Stories)
A Vindication of Natural Diet (Essays/Health, Non-Fiction)
Adonais (Poetry, Elegy)
The Complete Poetical Works of Percy Bysshe Shelley (Poetry Collection)
A Damsel in Distress (Fiction, Humorous Novel)
A Prefect's Uncle (Fiction, School Story)
A Wodehouse Miscellany (Fiction/Essays Collection)
Jill the Reckless (Fiction, Humorous Novel)
Noto, An Unexplored Corner of Japan (Travel, Non-Fiction)
The Soul of the Far East (Philosophy/Cultural, Non-Fiction)
Beyond Lies the Wub (Science Fiction, Short Story)
Beyond the Door (Science Fiction, Short Story)
Mr. Spaceship (Science Fiction, Short Story)
Piper in the Woods (Science Fiction, Short Story)
Second Variety (Science Fiction, Short Story)
The Crystal Crypt (Science Fiction, Short Story)
The Eyes Have It (Science Fiction, Short Story)
The Gun (Science Fiction, Short Story)
The Variable Man (Science Fiction, Novella)
Away in the Wilderness (Adventure Fiction, Youth Novel)
Battles with the Sea (Adventure Fiction, Non-Fiction Elements)
Black Ivory (Adventure Fiction, Novel)
Blown to Bits (Adventure Fiction, Novel)
Blue Lights (Adventure Fiction, Novel)
Captain Blood (Adventure Fiction, Pirate Novel)
Essays, First Series (Essays/Philosophy, Non-Fiction)
Essays, Second Series (Essays/Philosophy, Non-Fiction)
Essays (Essays/Philosophy, Non-Fiction)
Nature (Essays/Philosophy, Non-Fiction)
Poems (Poetry Collection)
Representative Men (Biography/Essays, Non-Fiction)
Scarborough and the Critic (Drama/Play)
St. Patrick's Day (Drama/Play)
The Duenna (Drama/Play)
The Rivals (Drama/Play)
The School For Scandal (Drama/Play)
Christmas Eve (Poetry)
Dramatic Romances (Poetry Collection)
Men and Women (Poetry Collection)
Selections from the Poems and Plays of Robert Browning (Poetry/Drama Collection)
The Pied Piper of Hamelin (Poetry, Narrative Verse)
Poems And Songs Of Robert Burns (Poetry Collection)
Tam O'Shanter (Poetry)
The Letters of Robert Burns (Biography/Letters, Non-Fiction)
A Boy's Will (Poetry Collection)
Mountain Interval (Poetry Collection)
North of Boston (Poetry Collection)
Micrographia (Science/Biology, Non-Fiction)
A Child's Garden of Verses, Verse 130 (Poetry, Children's)
A Child's Garden of Verses, Verse 154 (Poetry, Children's)
A Child's Garden of Verses, Verse 158 (Poetry, Children's)
A Child's Garden of Verses, Version 2 (Poetry Collection, Children's)
A Christmas Sermon (Religion/Sermon, Non-Fiction)
A Footnote to History (History, Non-Fiction)
Across The Plains (Travel, Non-Fiction)
An Inland Voyage (Travel, Non-Fiction)
Ballads (Poetry Collection)
Catriona (Adventure Fiction, Novel)
Poems, 1799 (Poetry Collection)
A Diversity of Creatures (Fiction, Short Stories)
A Song of the English (Poetry)
Actions and Reactions (Fiction, Short Stories)
Barrack-Room Ballads (Poetry Collection)
France At War (History/War, Non-Fiction)
From Sea to Sea (Travel, Non-Fiction)
Biographia Epistolaris, Volume 1 (Biography/Letters, Non-Fiction)
Biographia Literaria (Literary Criticism, Non-Fiction)
Coleridge's Ancient Mariner and Select Poems (Poetry Collection)
Coleridge's Literary Remains, Volume 1 (Literary Criticism, Non-Fiction)
Babbitt (Fiction, Satirical Novel)
Free Air (Fiction, Road Novel)
A Desert Drama (Adventure Fiction, Novel)
A Study In Scarlet (Mystery Fiction, Detective Novel)
A Visit to Three Fronts (History/War, Non-Fiction)
Beyond the City (Fiction, Novel)
Danger! and Other Stories (Fiction, Short Stories)
His Last Bow (Mystery Fiction, Short Stories)
Memoirs of Sherlock Holmes (Mystery Fiction, Short Stories)
Micah Clarke (Historical Fiction, Novel)
Finger Prints (Science/Forensics, Non-Fiction)
Inquiries into Human Faculty and Its Development (Science/Psychology, Non-Fiction)
The Art of Travel (Travel/Guide, Non-Fiction)
Opticks (Science/Physics, Non-Fiction)
The Chronology of Ancient Kingdoms Amended (History, Non-Fiction)
Himalayan Journals (Travel/Science, Non-Fiction)
First Footsteps in East Africa (Travel/Adventure, Non-Fiction)
Bride of Lammermoor (Historical Fiction, Novel)
Chronicles of the Canongate (Historical Fiction, Short Stories)
Fifty Bab Ballads (Poetry/Humor Collection)
Songs of a Savoyard (Poetry/Humor Collection)
The Bab Ballads (Poetry/Humor Collection)
Liberalism and the Social Problem (Politics/Essays, Non-Fiction)
London to Ladysmith via Pretoria (History/War, Non-Fiction)
The Story of the Malakand Field Force (History/Military, Non-Fiction)
Adventurers of the Far North (History/Exploration, Non-Fiction)
Arcadian Adventures with the Idle Rich (Fiction, Satirical Short Stories)
Behind the Beyond (Fiction, Short Stories)
Frenzied Fiction (Fiction, Humorous Short Stories)
Further Foolishness (Fiction, Humorous Short Stories)
Literary Lapses (Fiction, Humorous Short Stories)
My Discovery of England (Essays/Humor, Non-Fiction)
Nonsense Novels (Fiction, Parody Novels)
Eeldrop and Appleplex (Fiction, Short Story)
Ezra Pound: His Metric and Poetry (Literary Criticism, Non-Fiction)
Prufrock and Other Observations (Poetry Collection)
History of Friedrich II of Prussia, Volume 10 (History/Biography, Non-Fiction)
History of Friedrich II of Prussia, Volume 11 (History/Biography, Non-Fiction)
History of Friedrich II of Prussia, Volume 12 (History/Biography, Non-Fiction)
History of Friedrich II of Prussia, Volume 13 (History/Biography, Non-Fiction)
A Changed Man and Other Tales (Fiction, Short Stories)
A Group of Noble Dames (Fiction, Short Stories)
A Laodicean (Fiction, Novel)
A Pair of Blue Eyes (Fiction, Novel)
A Critical Examination Of The Position Of Mr. Darwin's Work, "On The Origin Of Species" (Science/Criticism, Non-Fiction)
American Addresses, with a Lecture on the Study of Biology (Science/Lectures, Non-Fiction)
Aphorisms and Reflections from the Works of T. H. Huxley (Philosophy/Science, Non-Fiction)
Collected Essays, Volume V (Essays/Science, Non-Fiction)
Criticisms on "The Origin of Species" (Science/Criticism, Non-Fiction)
Critiques and Addresses (Essays/Science, Non-Fiction)
An Essay on the Principle of Population (Economics/Sociology, Non-Fiction)
Nature and Progress of Rent (Economics, Non-Fiction)
The Grounds of an Opinion on the Policy of Restricting the Importation of Foreign Corn (Economics/Politics, Non-Fiction)
Blacky the Crow (Children's Fiction, Animal Tale)
Bowser The Hound (Children's Fiction, Animal Tale)
Happy Jack (Children's Fiction, Animal Tale)
Mother West Wind 'Why' Stories (Children's Fiction, Animal Tales)
Mother West Wind "Where" Stories (Children's Fiction, Animal Tales)
Mother West Wind's Animal Friends (Children's Fiction, Animal Tales)
Mother West Wind's Children (Children's Fiction, Animal Tales)
Mrs. Peter Rabbit (Children's Fiction, Animal Tale)
Old Granny Fox (Children's Fiction, Animal Tale)
Old Mother West Wind (Children's Fiction, Animal Tales)
The Adventures of Buster Bear (Children's Fiction, Animal Tale)
The Adventures of Chatterer the Red Squirrel (Children's Fiction, Animal Tale)
The Adventures of Danny Meadow Mouse (Children's Fiction, Animal Tale)
The Adventures of Grandfather Frog (Children's Fiction, Animal Tale)
The Adventures of Jerry Muskrat (Children's Fiction, Animal Tale)
Letters of Ulysses S. Grant (Biography/Letters, Non-Fiction)
Personal Memoirs of U. S. Grant (Autobiography, Non-Fiction)
State of the Union Addresses of Ulysses S. Grant (History/Politics, Non-Fiction)
Jacob's Room (Fiction, Modernist Novel)
Monday or Tuesday (Fiction, Short Stories)
Night and Day (Fiction, Novel)
The Voyage Out (Fiction, Novel)
Drum Taps (Poetry Collection)
Collected Poems 1901-1918 in Two Volumes Volume 1 (Poetry Collection)
Collected Poems 1901-1918 in Two Volumes Volume 2 (Poetry Collection)
Down-Adown-Derry (Poetry, Children's Collection)
Henry Brocken (Fiction, Fantasy Novel)
Memoirs of a Midget (Fiction, Fantasy Novel)
Peacock Pie, A Book of Rhymes (Poetry, Children's Collection)
Songs of Childhood (Poetry, Children's Collection)
The Listeners and Other Poems (Poetry Collection)
The Return (Fiction, Supernatural Novel)
The Three Mulla-mulgars (Children's Fiction, Fantasy Novel)
Abbotsford and Newstead Abbey (Travel/Biography, Non-Fiction)
Bracebridge Hall, or The Humorists (Fiction, Sketches)
Chronicle of the Conquest of Granada (History, Non-Fiction)
A Rogue's Life (Fiction, Novel)
Antonina (Historical Fiction, Novel)
Poems of William Blake (Poetry Collection)
Songs of Innocence and Songs of Experience (Poetry Collection)
Discoveries (Drama/Play)
Four Years (Autobiography, Non-Fiction)
Ideas of Good and Evil (Essays/Literary, Non-Fiction)
In The Seven Woods (Poetry Collection)
Per Amica Silentia Lunae (Essays/Philosophy, Non-Fiction)
Responsibilities (Poetry Collection)
Reveries over Childhood and Youth (Autobiography, Non-Fiction)
Seven Poems and a Fragment (Poetry Collection)
Synge And The Ireland Of His Time (Literary Criticism, Non-Fiction)
The Celtic Twilight (Folklore/Essays, Non-Fiction)
The Green Helmet and Other Poems (Poetry Collection)
The Land Of Heart's Desire (Drama/Play)
The Secret Rose (Fiction, Short Stories/Folklore)
A Belated Guest (Essays/Literary, Non-Fiction)
A Boy's Town (Autobiography, Non-Fiction)
A Fearful Responsibility and Other Stories (Fiction, Short Stories)
A Foregone Conclusion (Fiction, Novel)
A Hazard of New Fortunes (Fiction, Social Novel)
On Books and the Housing of Them (Essays/Literary, Non-Fiction)
A Crystal Age (Science Fiction, Utopian Novel)
A Little Boy Lost (Children's Fiction, Adventure)
A Shepherd's Life (Autobiography/Nature, Non-Fiction)
A Traveller in Little Things (Essays/Nature, Non-Fiction)
English Literature (Literary Criticism/History, Non-Fiction)
A Brief Account of the Rise and Progress of the People Called Quakers (Religion/History, Non-Fiction)
A Sermon Preached at the Quaker's Meeting House (Religion/Sermon, Non-Fiction)
Caesar's Wife (Drama/Play)
Liza of Lambeth (Fiction, Novel)
Of Human Bondage (Fiction, Bildungsroman Novel)
Orientations (Fiction, Short Stories)
Lyrical Ballads, With Other Poems, 1800, Volume 2 (Poetry Collection)
Poems in Two Volumes, Volume 1 (Poetry Collection)
Poems in Two Volumes, Volume 2 (Poetry Collection)
The Poetical Works of William Wordsworth, Volume 3 (Poetry Collection)
A Golden Venture, The Lady of the Barge and Others, Part 11 (Fiction, Short Story)
A Master of Craft (Fiction, Humorous Novel)
A Spirit of Avarice, Odd Craft, Part 11 (Fiction, Short Story)
Admiral Peters, Odd Craft, Part 14 (Fiction, Short Story)
An Adulteration Act, The Lady of the Barge and Others, Part 10 (Fiction, Short Story)
At Sunwich Port (Fiction, Novel)
Back to Back, Night Watches, Part 1 (Fiction, Short Story)
Bedridden and The Winter Offensive, Deep Waters, Part 8 (Fiction, Short Story)
Bill's Lapse, Odd Craft, Part 4 (Fiction, Short Story)
Bill's Paper Chase, Lady of the Barge and Others, Part 3 (Fiction, Short Story)
Bob's Redemption, Captains All, Book 5 (Fiction, Short Story)
Captain Rogers, The Lady of the Barge and Others, Part 7 (Fiction, Short Story)
Captains All (Fiction, Short Stories Collection)
Cupboard Love, The Lady of the Barge and Others, Part 5 (Fiction, Short Story)
Deep Waters (Fiction, Short Stories Collection)
Deserted, Sailor's Knots, Part 1 (Fiction, Short Story)
Dialstone Lane (Fiction, Novel)
Dirty Work, Deep Waters, Part 11 (Fiction, Short Story)
Dixon's Return, Odd Craft, Part 10 (Fiction, Short Story)
Dual Control, Ship's Company, Part 8 (Fiction, Short Story)
Establishing Relations, Odd Craft, Part 7 (Fiction, Short Story)
A Far Country, Complete (Fiction, Novel)
A Traveller in War-Time (Travel/War, Non-Fiction)
An Essay On The American Contribution And The Democratic Idea (Essays/Politics, Non-Fiction)
Betty Zane (Historical Fiction, Romance Novel)
Frankenstein; Or, The Modern Prometheus (Science Fiction, Gothic Novel) [Note: This is a duplicate of an earlier Frankenstein entry, same genre]
Romeo and Juliet (Drama/Play, Tragedy)
Pride and Prejudice (Fiction, Romance Novel)
The Complete Works of William Shakespeare (Drama/Plays Collection, Poetry)
Middlemarch (Fiction, Novel)
Little Women; Or, Meg, Jo, Beth, and Amy (Fiction, Coming-of-Age Novel)
The Blue Castle: a novel (Fiction, Romance Novel)
How to Observe: Morals and Manners (Sociology/Etiquette, Non-Fiction)
The Enchanted April (Fiction, Novel)
Cranford (Fiction, Social Novel)
The Expedition of Humphry Clinker (Fiction, Epistolary Novel)
The Adventures of Ferdinand Count Fathom — Complete (Fiction, Picaresque Novel)
History of Tom Jones, a Foundling (Fiction, Picaresque Novel)
The Picture of Dorian Gray (Fiction, Gothic Novel)
Leviathan (Philosophy/Politics, Non-Fiction)
Society in America, Volume 1 (of 2) (Sociology/Travel, Non-Fiction)
The Strange Case of Dr. Jekyll and Mr. Hyde (Horror Fiction, Novella)
Metamorphosis (Fiction, Absurdist Novella)
Right Ho, Jeeves (Fiction, Humorous Novel)
The Brothers Karamazov (Fiction, Philosophical Novel)
Project Gutenberg Compilation of Short Stories by Chekhov (Fiction, Short Stories Collection)
The Scarlet Letter (Fiction, Historical Novel)
Jane Eyre: An Autobiography (Fiction, Gothic Romance Novel) [Note: Duplicate of earlier Jane Eyre]
Thus Spake Zarathustra: A Book for All and None (Philosophy, Non-Fiction)
The Iliad (Poetry, Epic)
The Count of Monte Cristo (Adventure Fiction, Novel)
Grimms' Fairy Tales (Children's Fiction, Fairy Tales Collection)
War and Peace (Historical Fiction, Novel)
Simple Sabotage Field Manual (Non-Fiction, Military/Instructional)
A Doll's House : a play (Drama/Play)
Adventures of Huckleberry Finn (Adventure Fiction, Novel) [Note: Duplicate of earlier entry]
A Modest Proposal: For preventing the children of poor people in Ireland, from being a burden on their parents or country, and for making them beneficial to the publick (Satire/Essays, Non-Fiction)
Rip Van Winkle (Fiction, Short Story/Folklore)
The Prince (Philosophy/Politics, Non-Fiction)
The Importance of Being Earnest: A Trivial Comedy for Serious People (Drama/Play, Comedy)
The Adventures of Tom Sawyer, Complete (Adventure Fiction, Youth Novel)
The divine comedy (Poetry, Epic)
On Liberty (Philosophy/Politics, Non-Fiction)
Moby Word Lists (Reference/Linguistics, Non-Fiction) [Note: This seems like a utility list, not a literary work; genre is reference]
We (Science Fiction, Dystopian Novel)
Les Bijoux Indiscrets, or, The Indiscreet Toys (Fiction, Erotic Satire Novel)
R.U.R. (Rossum's Universal Robots): A Fantastic Melodrama in Three Acts and an Epilogue (Drama/Play, Science Fiction)
St. John's College, Cambridge (History/Education, Non-Fiction)
The Republic (Philosophy, Non-Fiction)
Great Expectations (Fiction, Bildungsroman Novel)
Beyond Good and Evil (Philosophy, Non-Fiction)
The Kama Sutra of Vatsyayana: Translated From the Sanscrit in Seven Parts With Preface, Introduction and Concluding Remarks (Philosophy/Erotica, Non-Fiction)
A passage to India (Fiction, Novel)
The Yellow Wallpaper (Fiction, Short Story/Horror)
The Wars of Religion in France 1559-1576: The Huguenots, Catherine de Medici and Philip II (History, Non-Fiction)
Society as I Have Found It (Autobiography/Social Commentary, Non-Fiction)
Moby Multiple Language Lists of Common Words (Reference/Linguistics, Non-Fiction) [Similar to Moby Word Lists]
The Odyssey: Rendered into English prose for the use of those who cannot read the original (Poetry/Epic, Translation)
The lesser Key of Solomon, Goetia, the book of evil spirits : $b contains two hundred diagrams and seals for invocation and convocation of spirits, necromancy, witchcraft and black art (Occult/Religion, Non-Fiction)
More Letters of Charles Darwin — Volume 1: A Record of His Work in a Series of Hitherto Unpublished Letters (Biography/Letters, Non-Fiction)
Chambers's Twentieth Century Dictionary (part 1 of 4: A-D) (Reference/Dictionary, Non-Fiction)
The Romance of Lust: A classic Victorian erotic novel (Fiction, Erotic Novel)
Auguste Comte and Positivism (Philosophy, Non-Fiction)
The Comic Poems of Thomas Hood: A New and Complete Edition (Poetry/Humor Collection)
The Modern Traveller (Poetry/Satire)
Narrative of the Life of Frederick Douglass, an American Slave (Autobiography, Non-Fiction)
"""

# Build dictionary
genre_map = {}
for line in titles_with_genres.splitlines():
    line = line.strip()
    if line.endswith(")"):  # only process lines with parentheses
        try:
            title, genre = line.rsplit("(", 1)   # split only at the last "("
            genre = genre.strip(") ")            # remove closing parenthesis
            genre_map[title.strip()] = genre.strip()
        except ValueError:
            pass  # skip malformed lines

# Print in nice dict format
print("# Define the mapping of title → genre")
print("genre_map = {")
for title, genre in genre_map.items():
    print(f'    "{title}": "{genre}",')
print("}")

# Define the mapping of title → genre
genre_map = {
    "Lincoln Letters": "History/Politics, Non-Fiction",
    "Lincoln's First Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Gettysburg Address, given November 19, 1863": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Inaugurals, Addresses and Letters (Selections)": "History/Politics, Non-Fiction Collection",
    "Lincoln's Second Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Speeches and Letters of Abraham Lincoln, 1832-1865": "History/Politics, Non-Fiction Collection",
    "State of the Union Addresses": "History/Politics, Non-Fiction",
    "The Emancipation Proclamation": "History/Politics, Non-Fiction",
    "The Life and Public Service of General Zachary Taylor: An Address": "Biography/History, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 1: 1832-1843": "History/Politics, Non-Fiction Collection",
    "The Writings of Abraham Lincoln, Volume 2: 1843-1858": "History

In [5]:
import re

# Example text block containing titles with (genre)
titles_with_genres = """
Peter Pan : $b [Peter and Wendy] (Children's Fiction, Fantasy Novel)
Justice (Philosophy/Non-Fiction)
Meditations (Philosophy/Autobiography, Non-Fiction)
Don't Marry; or, Advice on How, When and Who to Marry (Self-Help/Advice, Non-Fiction)
Society in America, Volume 2 (of 2) (Sociology/Travel, Non-Fiction) [Note: Companion to Volume 1 from previous list]
Manual of Classical Erotology (De figuris Veneris) (Erotica/Philosophy, Non-Fiction)
The Works of Edgar Allan Poe — Volume 2 (Fiction/Poetry Collection, Horror/Mystery) [Note: Part of Poe's works series from previous list]
Cookery and Dining in Imperial Rome (Cookbook/History, Non-Fiction)
The Mahabharata of Krishna-Dwaipayana Vyasa, Volume 1: Books 1, 2 and 3 (Poetry/Epic, Mythology/Religion)
The Thousand and One Nights, Vol. I.: Commonly Called the Arabian Nights' Entertainments (Folklore/Fiction, Short Stories Collection)
On the Origin of Species By Means of Natural Selection: Or, the Preservation of Favoured Races in the Struggle for Life (Science/Biology, Non-Fiction)
The Wonderful Wizard of Oz (Children's Fiction, Fantasy Novel)
A Christmas Carol in Prose; Being a Ghost Story of Christmas (Fiction, Ghost Story/Novella) [Note: Variant of A Christmas Carol from previous list]
The Convent School; Or, Early Experiences of a Young Flagellant (Fiction, Erotic Novel)
The Legend of Sleepy Hollow (Horror Fiction, Short Story)
Confidential Chats with Girls (Self-Help/Advice, Non-Fiction)
The King James Version of the Bible (Religion/Scripture, Non-Fiction)
Pascal's Pensées (Philosophy/Religion, Non-Fiction Essays)
The Twilight of the Idols; or, How to Philosophize with the Hammer. The Antichrist: Complete Works, Volume Sixteen (Philosophy, Non-Fiction)
Shakespeare's family (Biography/History, Non-Fiction)
Heart of Darkness (Fiction, Novella/Adventure)
The Philippine Islands, 1493-1898, Volume 33, 1519-1522: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century (History/Exploration, Non-Fiction)
Le Morte d'Arthur: Volume 1 (Fiction, Arthurian Legend/Romance)
An Essay on Laughter: Its Forms, Its Causes, Its Development and Its Value (Philosophy/Psychology, Non-Fiction)
Carmilla (Horror Fiction, Vampire Novella)
Treasure Island (Adventure Fiction, Novel)
The sexual life of woman in its physiological, pathological and hygienic aspects (Science/Medicine, Non-Fiction)
The Ethics of Aristotle (Philosophy, Non-Fiction)
The History of the Peloponnesian War (History, Non-Fiction)
The King in Yellow (Horror Fiction, Short Stories)
The Waterloo Roll Call: With Biographical Notes and Anecdotes (History/Military, Non-Fiction)
A Journey to the Centre of the Earth (Science Fiction, Adventure Novel)
Candide (Fiction, Satirical Novel)
Sense and Sensibility (Fiction, Romance Novel)
The Innocents Abroad (Travel/Humor, Non-Fiction)
Faust [part 1]. Translated Into English in the Original Metres (Drama/Poetry, Tragedy)
The Time Machine (Science Fiction, Novel)
The Life and Adventures of Robinson Crusoe (Adventure Fiction, Novel)
The Tragical History of Doctor Faustus: From the Quarto of 1604 (Drama/Play, Tragedy)
Lady Chatterley's lover (Fiction, Romance Novel/Erotic)
Winnie-the-Pooh (Children's Fiction, Animal Tales)
Don Quixote (Fiction, Satirical Novel)
Democracy in America — Volume 1 (Politics/Sociology, Non-Fiction)
Antiquities of the Jews (History/Religion, Non-Fiction)
Harriet Martineau (Biography, Non-Fiction)
The Art of War (Military/Philosophy, Non-Fiction)
The Philippines a Century Hence (History/Politics, Non-Fiction Essay)
A Complete Guide to Heraldry (History/Reference, Non-Fiction)
The Jungle Book (Children's Fiction, Animal Tales/Short Stories)
Twenty Thousand Leagues under the Sea (Science Fiction, Adventure Novel)
Legends of the Bastille (History, Non-Fiction)
The Philosophy of Auguste Comte (Philosophy, Non-Fiction)
A Complete Dictionary of Synonyms and Antonyms: or, Synonyms and Words of Opposite Meaning (Reference/Dictionary, Non-Fiction)
The Decameron of Giovanni Boccaccio (Fiction, Short Stories Collection)
The Genealogy of Morals: The Complete Works, Volume Thirteen, edited by Dr. Oscar Levy. (Philosophy, Non-Fiction)
The History of the Decline and Fall of the Roman Empire: Table of Contents with links in the HTML file to the two Project Gutenberg editions (12 volumes) (History, Non-Fiction)
Aesop's Fables; a new translation (Children's Fiction, Fables)
An Inquiry into the Nature and Causes of the Wealth of Nations (Economics, Non-Fiction)
The Hound of the Baskervilles (Mystery Fiction, Detective Novel)
The Rámáyan of Válmíki, translated into English verse (Poetry/Epic, Mythology)
On War (Military/Strategy, Non-Fiction)
An Essay Concerning Humane Understanding, Volume 1: MDCXC, Based on the 2nd Edition, Books 1 and 2 (Philosophy, Non-Fiction) [Note: Variant of previous list entry]
Rizal's own story of his life (Autobiography, Non-Fiction)
Poems by Emily Dickinson, Three Series, Complete (Poetry Collection)
The Elementary Forms of the Religious Life (Sociology/Religion, Non-Fiction)
Complete Original Short Stories of Guy De Maupassant (Fiction, Short Stories Collection)
Goethe's Theory of Colours (Science/Art, Non-Fiction)
Tess of the d'Urbervilles: A Pure Woman (Fiction, Novel)
Three men in a boat (to say nothing of the dog) (Fiction, Humorous Novel)
The Anabasis of Alexander : $b or, The history of the wars and conquests of Alexander the Great (History/Biography, Non-Fiction)
Spenser's The Faerie Queene, Book I (Poetry, Epic)
Struwwelpeter: Merry Stories and Funny Pictures (Children's Fiction, Illustrated Rhymes)
The Works of Edgar Allan Poe — Volume 1 (Fiction/Poetry Collection, Horror/Mystery) [Note: Duplicate from previous list]
The Story of Beowulf, Translated from Anglo-Saxon into Modern English Prose (Poetry/Epic, Translation)
Notes and Queries, Number 82, May 24, 1851: A Medium of Inter-communication for Literary Men, Artists, Antiquaries, Genealogists, etc. (Literary Criticism/History, Non-Fiction Journal)
The History of Herodotus — Volume 1 (History, Non-Fiction)
The Birth of Tragedy; or, Hellenism and Pessimism (Philosophy/Literary Criticism, Non-Fiction)
Ethics (Philosophy, Non-Fiction)
The Marriage of Heaven and Hell (Poetry/Philosophy)
The History of the Standard Oil Company (History/Business, Non-Fiction)
The Sorrows of Young Werther (Fiction, Epistolary Novel)
The Sins of the Cities of the Plain; or, The Recollections of a Mary-Ann: with Short Essays on Sodomy and Tribadism (Fiction, Erotic Memoir)
Around the World in Eighty Days (Adventure Fiction, Novel)
Doctrina Christiana: The first book printed in the Philippines, Manila, 1593. (Religion/History, Non-Fiction)
The Ten Books on Architecture (Architecture/Science, Non-Fiction)
A Dictionary of Cebuano Visayan (Reference/Dictionary, Non-Fiction)
My Life and Loves, Vol. 1 (of 4) (Autobiography/Erotic, Non-Fiction)
Lives of Fair and Gallant Ladies. Vol 1 (Biography/Erotic History, Non-Fiction)
A serious proposal to the Ladies, for the advancement of their true and greatest interest (In Two Parts) (Essays/Feminism, Non-Fiction)
The Sign of the Four (Mystery Fiction, Detective Novel)
The Sun Also Rises (Fiction, Novel)
Reminiscences of a Stock Operator (Autobiography/Business, Non-Fiction)
The Beautiful and Damned (Fiction, Novel)
The Rape of the Lock, and Other Poems (Poetry, Satirical Verse) [Note: Duplicate from previous list]
The Lady of the Lake (Poetry, Narrative Verse)
The Enchiridion (Philosophy, Non-Fiction)
The Travels of Marco Polo — Volume 1 (Travel/History, Non-Fiction)
Little Brother (Science Fiction, Novel)
The Erotic Motive in Literature (Literary Criticism/Psychology, Non-Fiction)
The Devil's Dictionary (Humor/Satire, Non-Fiction Reference)
Divine Comedy, Longfellow's Translation, Hell (Poetry, Epic) [Note: Part of Divine Comedy from previous list]
Titan: A Romance. v. 1 (of 2) (Fiction, Romance Novel)
Fifteen sonnets of Petrarch (Poetry Collection)
Revelations of Divine Love (Religion/Mysticism, Non-Fiction)
Uncle Tom's Cabin (Fiction, Anti-Slavery Novel)
The City of God, Volume I (Religion/Philosophy, Non-Fiction)
The Mummy! A Tale of the Twenty-Second Century (Science Fiction, Novel)
Vanity Fair (Fiction, Satirical Novel)
The art of money getting : $b or, golden rules for making money (Self-Help/Business, Non-Fiction)
The Phantom of the Opera (Fiction, Gothic Novel)
Just so stories (Children's Fiction, Short Stories)
Philippine Folk Tales (Folklore/Fiction, Short Stories Collection)
Mrs. Dalloway (Fiction, Modernist Novel)
A Dictionary of Slang, Cant, and Vulgar Words: Used at the Present Day in the Streets of London; the Universities of Oxford and Cambridge; the Houses of Parliament; the Dens of St. Giles; and the Palaces of St. James. (Reference/Dictionary, Non-Fiction)
Princes and Poisoners: Studies of the Court of Louis XIV (History/Biography, Non-Fiction)
A History of Greek Economic Thought (History/Economics, Non-Fiction)
Plays (Drama/Plays Collection)
Phaedrus (Philosophy, Non-Fiction Dialogue)
The war drama of the Eagles : $b Napoleon's standard-bearers on the battlefield in victory and defeat from Austerlitz to Waterloo, a record of hard fighting, heroism, and adventure (History/Military, Non-Fiction)
Omens and Superstitions of Southern India (Folklore/Anthropology, Non-Fiction)
The Turn of the Screw (Horror Fiction, Novella)
Discourse on the Method of Rightly Conducting One's Reason and of Seeking Truth in the Sciences (Philosophy/Science, Non-Fiction)
Golden Days for Boys and Girls, Vol. XII, Jan. 3, 1891 (Children's Fiction, Magazine/Short Stories)
Heath's Modern Language Series: The Spanish American Reader (Language/Education, Non-Fiction)
Amusements in Mathematics (Mathematics/Puzzles, Non-Fiction)
Degeneration (Sociology/Cultural Criticism, Non-Fiction)
A short history of the printing press and of the improvements in printing machinery from the time of Gutenberg up to the present day (History/Technology, Non-Fiction)
Spoon River Anthology (Poetry Collection)
The Lives of the Twelve Caesars, Complete (History/Biography, Non-Fiction)
Forty-Eighth Annual Report of the Bureau of American Ethnology to the Secretary of the Smithsonian Institution, 1930-1931, Government Printing Office, Washington, 1933. (Anthropology/History, Non-Fiction Report)
The Philippine Islands, 1493-1898 — Volume 07 of 55: 1588-1591; Explorations by Early Navigators, Descriptions of the Islands and Their Peoples, Their History and Records of the Catholic Missions, as Related in Contemporaneous Books and Manuscripts, Showing the Political, Economic, Commercial and Religious Conditions of Those Islands from Their Earliest Relations with European Nations to the Close of the Nineteenth Century (History/Exploration, Non-Fiction)
Reliques of Ancient English Poetry, Volume 2 (of 3): Consisting of Old Heroic Ballads, Songs and Other Pieces of Our Earlier Poets Together With Some Few of Later Date (Poetry/Folklore Collection)
The Complete Herbal: To which is now added, upwards of one hundred additional herbs, with a display of their medicinal and occult qualities physically applied to the cure of all disorders incident to mankind: to which are now first annexed, the English physician enlarged, and key to Physic. (Medicine/Herbalism, Non-Fiction)
2 B R 0 2 B (Science Fiction, Short Story)
Memoirs of Extraordinary Popular Delusions and the Madness of Crowds (History/Psychology, Non-Fiction)
Gulliver's Travels into Several Remote Regions of the World (Fiction, Satirical Adventure Novel) [Note: Variant of Gulliver's Travels from previous list]
The Pursuit of God (Religion/Spirituality, Non-Fiction)
All About Coffee (History/Food, Non-Fiction)
Plutarch's Morals (Philosophy/Ethics, Non-Fiction)
Helps to Latin Translation at Sight (Education/Language, Non-Fiction)
A Pickle for the Knowing Ones (Humor/Satire, Non-Fiction)
The reader's guide to the Encyclopaedia Britannica : $b A handbook containing sixty-six courses of systematic study or occasional reading (Reference/Education, Non-Fiction)
Gorgias (Philosophy, Non-Fiction Dialogue)
The Diary of Samuel Pepys — Complete (Autobiography/Diary, Non-Fiction)
Brief Lives, Vol. 1 (Biography Collection, Non-Fiction)
The Red Record: Tabulated Statistics and Alleged Causes of Lynching in the United States (History/Social Justice, Non-Fiction)
Highways and Byways in Cambridge and Ely (Travel/History, Non-Fiction)
The Awakening, and Selected Short Stories (Fiction, Short Stories/Novel)
Poems, with The Ballad of Reading Gaol (Poetry Collection)
International Law. A Treatise. Volume 1 (of 2): Peace. Second Edition (Law/Politics, Non-Fiction)
Poems and Songs of Robert Burns (Poetry Collection) [Note: Duplicate from previous list]
Scenes in the Life of Harriet Tubman (Biography/History, Non-Fiction)
Death in Venice (Fiction, Novella)
Southern Horrors: Lynch Law in All Its Phases (History/Social Justice, Non-Fiction)
The Book of the Thousand Nights and a Night — Volume 02 (of 10) (Folklore/Fiction, Short Stories)
The Song Celestial; Or, Bhagavad-Gîtâ (from the Mahâbhârata): Being a discourse between Arjuna, Prince of India, and the Supreme Being under the form of Krishna (Poetry/Religion, Translation)
Anabasis (History/Military, Non-Fiction)
Narrative of the Captivity and Restoration of Mrs. Mary Rowlandson (Autobiography/History, Non-Fiction)
Narrative and Critical History of America, Vol. 2 (of 8): Spanish Explorations and Settlements in America from the Fifteenth to the Seventeenth Century (History, Non-Fiction)
The Slayer of Souls (Fiction, Occult Novel)
Reliques of Ancient English Poetry, Volume 1 (of 3): Consisting of Old Heroic Ballads, Songs and Other Pieces of Our Earlier Poets Together With Some Few of Later Date (Poetry/Folklore Collection)
The Glories of Ireland (History/Cultural, Non-Fiction)
Encyclopaedia Britannica, 11th Edition, "Cincinnatus" to "Cleruchy": Volume 6, Slice 4 (Reference/Encyclopedia, Non-Fiction)
The Nursery Rhymes of England (Children's Poetry, Collection)
Swann's Way (Fiction, Novel)
The Aeneid (Poetry, Epic)
The History of Rome, Books 01 to 08 (History, Non-Fiction)
Staffordshire pottery and its history (History/Art, Non-Fiction)
A History of Witchcraft in England from 1558 to 1718 (History/Occult, Non-Fiction)
Bradford's History of 'Plimoth Plantation': From the Original Manuscript. With a Report of the Proceedings Incident to the Return of the Manuscript to Massachusetts (History, Non-Fiction)
The Flowers of Evil (Poetry Collection)
Napoleon's Letters to Josephine, 1796-1812: For the First Time Collected and Translated, with Notes Social, Historical, and Chronological, from Contemporary Sources (Biography/Letters, Non-Fiction)
The Methods of Ethics (Philosophy/Ethics, Non-Fiction)
A Popular History of Astronomy During the Nineteenth Century: Fourth Edition (Science/History, Non-Fiction)
Chess Fundamentals (Games/Strategy, Non-Fiction)
Black Beauty (Children's Fiction, Animal Tale)
Democracy in America — Volume 2 (Politics/Sociology, Non-Fiction)
The Divine Comedy of Dante Alighieri: The Inferno (Poetry, Epic)
The House of the Wolfings: A Tale of the House of the Wolfings and All the Kindreds of the Mark Written in Prose and in Verse (Fiction, Fantasy Novel)
In Praise of Folly: Illustrated with Many Curious Cuts (Satire/Philosophy, Non-Fiction)
The Ivory Gate, a new edition (Fiction, Novel)
Eugene Oneguine [Onegin]: A Romance of Russian Life in Verse (Poetry, Novel in Verse)
George Cruikshank (Biography/Art, Non-Fiction)
Apology (Philosophy, Non-Fiction Dialogue)
Encyclopaedia Britannica, 11th Edition, "Gichtel, Johann" to "Glory": Volume 12, Slice 1 (Reference/Encyclopedia, Non-Fiction)
Bartleby, the Scrivener: A Story of Wall-Street (Fiction, Short Story)
The Satires of Juvenal, Persius, Sulpicia, and Lucilius: Literally translated into English prose, with notes, chronological tables, arguments, &c. (Poetry/Satire, Translation)
The girl from Scotland Yard (Mystery Fiction, Novel)
Pen-portraits of literary women : $b by themselves and others, Volume 2 (of 2) (Biography/Literary Criticism, Non-Fiction)
On Heroes, Hero-Worship, and the Heroic in History (Essays/Philosophy, Non-Fiction)
De Officiis (Philosophy/Ethics, Non-Fiction)
The Life of Ludwig van Beethoven, Volume I (Biography, Non-Fiction)
Anthem (Science Fiction, Dystopian Novella)
The Ancient Stone Implements, Weapons and Ornaments, of Great Britain: Second Edition, Revised (Archaeology/History, Non-Fiction)
Demonology and Devil-lore (Folklore/Occult, Non-Fiction)
Some Jewish Witnesses For Christ (Religion/Biography, Non-Fiction)
Up from Slavery: An Autobiography (Autobiography, Non-Fiction)
The Man Who Was Thursday: A Nightmare (Fiction, Philosophical Novel)
Poems on various subjects, religious and moral (Poetry Collection)
History of the Royal Regiment of Artillery, Vol. 1: Compiled from the Original Records (History/Military, Non-Fiction)
The Canterbury Tales, and Other Poems (Poetry Collection)
Twelve Years a Slave: Narrative of Solomon Northup, a Citizen of New-York, Kidnapped in Washington City in 1841, and Rescued in 1853, from a Cotton Plantation near the Red River in Louisiana (Autobiography/History, Non-Fiction)
Curious Facts in the History of Insects; Including Spiders and Scorpions.: A Complete Collection of the Legends, Superstitions, Beliefs, and Ominous Signs Connected with Insects; Together with Their Uses in Medicine, Art, and as Food; and a Summary of Their Remarkable Injuries and Appearances. (Science/Folklore, Non-Fiction)
The Monkey's Paw: The Lady of the Barge and Others, Part 2. (Horror Fiction, Short Story)
The Online World (Technology/Reference, Non-Fiction)
Principles of Political Economy: Abridged with Critical, Bibliographical, and Explanatory Notes, and a Sketch of the History of Political Economy (Economics, Non-Fiction)
The Vampyre; a Tale (Horror Fiction, Short Story)
Twenty Thousand Leagues Under the Seas: An Underwater Tour of the World (Science Fiction, Adventure Novel) [Note: Variant of previous entry]
Portuguese literature (Literary Criticism/History, Non-Fiction)
My Studio Neighbors (Essays/Nature, Non-Fiction)
A Short History of Freethought Ancient and Modern, Volume 1 of 2: Third edition, Revised and Expanded, in two volumes (History/Philosophy, Non-Fiction)
The Satyricon — Complete (Fiction, Satirical Novel)
The Social Cancer: A Complete English Version of Noli Me Tangere (Fiction, Social Novel)
The Trial (Fiction, Absurdist Novel)
New Latin Grammar (Education/Language, Non-Fiction)
The Extraordinary Adventures of Arsène Lupin, Gentleman-Burglar (Mystery Fiction, Short Stories)
The Outline of History: Being a Plain History of Life and Mankind (History, Non-Fiction)
An Enquiry Concerning Human Understanding (Philosophy, Non-Fiction)
The Works of Edgar Allan Poe, The Raven Edition: Table Of Contents And Index Of The Five Volumes (Fiction/Poetry Collection, Horror/Mystery)
Medea of Euripides (Drama/Play, Tragedy)
Bacon's Essays, and Wisdom of the Ancients (Essays/Philosophy, Non-Fiction)
Ivanhoe: A Romance (Historical Fiction, Novel)
The Germany and the Agricola of Tacitus (History, Non-Fiction)
The Medicinal Plants of the Philippines (Science/Botany, Non-Fiction)
The Wit and Humor of America, Volume X (of X) (Humor/Fiction Collection)
Mosses from an old manse (Fiction, Short Stories) [Note: Variant of previous list entry]
The Anatomy of Melancholy (Philosophy/Medicine, Non-Fiction)
Anne of Avonlea (Fiction, Youth Novel) [Note: Duplicate from previous list]
The Marching Morons (Science Fiction, Short Story)
The Mediterranean : $b Seaports and sea routes including Madeira, the Canary Islands, the coast of Morocco, Algeria, and Tunisia; handbook for travellers (Travel/Guide, Non-Fiction)
Ancient Pagan and Modern Christian Symbolism: With an Essay on Baal Worship, on the Assyrian Sacred "Grove," and Other Allied Symbols (Religion/History, Non-Fiction)
Historic Paris (History/Travel, Non-Fiction)
Mesopotamian Archaeology: An introduction to the archaeology of Mesopotamia and Assyria (Archaeology/History, Non-Fiction)
A Concise Anglo-Saxon Dictionary: For the Use of Students (Reference/Dictionary, Non-Fiction)
The Survey of London (History/Architecture, Non-Fiction)
North and South (Fiction, Social Novel)
Susan B. Anthony: Rebel, Crusader, Humanitarian (Biography, Non-Fiction)
Roget's Thesaurus (Reference/Dictionary, Non-Fiction)
The World as Will and Idea (Vol. 1 of 3) (Philosophy, Non-Fiction)
Showell's Dictionary of Birmingham: A History and Guide, Arranged Alphabetically (History/Guide, Non-Fiction)
The Philippine Islands, 1493-1898—Volume 34 of 55, 1519-1522; 1280-1605: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century (History/Exploration, Non-Fiction)
De Re Metallica, Translated from the First Latin Edition of 1556 (Science/Mining, Non-Fiction)
Devotions Upon Emergent Occasions; Together with Death's Duel (Religion/Meditation, Non-Fiction)
How France Built Her Cathedrals: A Study in the Twelfth and Thirteenth Centuries (Architecture/History, Non-Fiction)
Shakespearean Tragedy: Lectures on Hamlet, Othello, King Lear, Macbeth (Literary Criticism, Non-Fiction)
The Essays of Arthur Schopenhauer: the Wisdom of Life (Philosophy/Essays, Non-Fiction)
A Comprehensive History of Norwich (History, Non-Fiction)
The Younger Edda; Also called Snorre's Edda, or The Prose Edda (Mythology/Folklore, Non-Fiction)
The Progress of Invention in the Nineteenth Century. (History/Technology, Non-Fiction)
The Decoration of Houses (Architecture/Design, Non-Fiction)
Modern Spanish Lyrics (Poetry Collection)
Webster's Unabridged Dictionary (Reference/Dictionary, Non-Fiction)
Facts and Speculations on the Origin and History of Playing Cards (History/Games, Non-Fiction)
The Geography of Strabo, Volume 3 (of 3): Literally Translated, with Notes (Geography/History, Non-Fiction)
The Adventures of Pinocchio (Children's Fiction, Fantasy Novel)
Man and Superman: A Comedy and a Philosophy (Drama/Play, Comedy)
Stories from Tagore (Fiction, Short Stories)
Herland (Science Fiction, Utopian Novel)
The Practice and Science of Drawing (Art/Instruction, Non-Fiction)
Etiquette (Self-Help/Social, Non-Fiction)
Fifteen Thousand Useful Phrases (Reference/Language, Non-Fiction)
Friday, the Thirteenth: A Novel (Fiction, Novel)
The Golden Bough: A Study in Comparative Religion (Vol. 1 of 2) (Anthropology/Religion, Non-Fiction)
The Natural History of Pliny, Volume 1 (of 6) (Science/Natural History, Non-Fiction)
The Garden Party, and Other Stories (Fiction, Short Stories)
The Ballad of the White Horse (Poetry, Narrative Verse)
The Life of Florence Nightingale, vol. 2 of 2 (Biography, Non-Fiction)
Brief Lives, Vol. 2 (Biography Collection, Non-Fiction)
The Pilgrim's Progress by John Bunyan Every Child Can Read (Religion/Allegory, Fiction/Children's Adaptation)
Modern English biography, volume 2 (of 4), I-Q (Biography/Reference, Non-Fiction)
Utopia (Fiction, Utopian Satire)
Shell-shock and other neuropsychiatric problems : $b Presented in five hundred and eighty-nine case histories from the war literature, 1914-1918 (Medicine/Psychology, Non-Fiction)
Geographical etymology: a dictionary of place-names giving their derivations (Reference/Geography, Non-Fiction)
A Treatise on Painting (Art/Instruction, Non-Fiction)
Autobiography of a Yogi (Autobiography/Spirituality, Non-Fiction)
I, Mars (Science Fiction, Short Story)
Moby-Dick; or, The Whale (Adventure Fiction, Novel)
A farewell to arms (Fiction, War Novel)
Bible Myths and their Parallels in other Religions: Being a Comparison of the Old and New Testament Myths and Miracles with those of the Heathen Nations of Antiquity Considering also their Origin and Meaning (Religion/Mythology, Non-Fiction)
The Expositor's Bible: The Book of the Twelve Prophets, Vol. 1: Commonly Called the Minor (Religion/Commentary, Non-Fiction)
Haunted London (History/Occult, Non-Fiction)
One Thousand Ways to Make Money (Self-Help/Business, Non-Fiction)
A Defence of Poesie and Poems (Literary Criticism/Poetry, Non-Fiction)
Kant's Critique of Judgement (Philosophy, Non-Fiction)
The Castle of Otranto (Fiction, Gothic Novel)
The History of Cuba, vol. 4 (History, Non-Fiction)
The Roll of Honour, Volume 1 : $b A biographical record of all members of His Majesty's naval and military forces who have fallen in the war (History/Military, Non-Fiction)
A vocabulary of criminal slang : $b with some examples of common usages (Reference/Language, Non-Fiction)
The Influence of Sea Power Upon History, 1660-1783 (History/Military, Non-Fiction)
Three hundred Aesop’s fables: Translated by George Fyler Townsend (Children's Fiction, Fables)
Opticks : $b or, A treatise of the reflections, refractions, inflections and colours of light (Science/Physics, Non-Fiction) [Note: Duplicate from previous list]
The Handbook to English Heraldry (History/Reference, Non-Fiction)
A general history of the pyrates : $b from their first rise and settlement in the island of Providence, to the present time (History/Adventure, Non-Fiction)
Principia Ethica (Philosophy/Ethics, Non-Fiction)
Letters of Abelard and Heloise: To which is prefix'd a particular account of their lives, amours, and misfortunes (Biography/Letters, Non-Fiction)
The Balkan Wars: 1912-1913: Second Edition (History/Military, Non-Fiction)
A history of the Peninsular War, Vol. 3, Sep. 1809-Dec. 1810 : $b Ocaña, Cadiz, Bussaco, Torres Vedras (History/Military, Non-Fiction)
The Tenniel Illustrations for Carroll's Alice in Wonderland (Art/Illustration, Non-Fiction)
The Life of Lazarillo de Tormes: His Fortunes & Adversities; with a Notice of the Mendoza Family, a Short Life of the Author, Don Diego Hurtado De Mendoza, a Notice of the Work, and Some Remarks on the Character of Lazarillo de Tormes (Fiction, Picaresque Novel)
The History of Orange County New York (History, Non-Fiction)
Gargantua and Pantagruel (Fiction, Satirical Novel)
The Valleys of Tirol: Their traditions and customs and how to visit them (Travel/Folklore, Non-Fiction)
The Nature of Animal Light (Science/Biology, Non-Fiction)
Lineage, Life and Labors of José Rizal, Philippine Patriot (Biography/History, Non-Fiction)
Mr. Honey's Large Business Dictionary (English-German) (Reference/Dictionary, Non-Fiction)
Penrod (Fiction, Youth Novel)
The South of France—East Half (Travel/Guide, Non-Fiction)
The Tribes and Castes of the Central Provinces of India, Volume 3 (Anthropology/History, Non-Fiction)
Pragmatism: A New Name for Some Old Ways of Thinking (Philosophy, Non-Fiction)
A Boy's Voyage Round the World (Travel/Adventure, Non-Fiction)
The Golden Bough: A Study of Magic and Religion (Anthropology/Religion, Non-Fiction)
History of the Spanish Conquest of Yucatan and of the Itzas: Papers of the Peabody Museum of American Archaeology and Ethnology, Hard University. Vol. VII. (History/Exploration, Non-Fiction)
A Journal of the First Voyage of Vasco da Gama 1497-1499 (History/Exploration, Non-Fiction)
A Treatise of Human Nature (Philosophy, Non-Fiction)
The American Language: A Preliminary Inquiry into the Development of English in the United States (Linguistics, Non-Fiction)
The Alchemist (Drama/Play, Comedy)
The Problem of China (History/Politics, Non-Fiction)
Triplanetary (Science Fiction, Novel)
A Short History of French Literature (Literary Criticism/History, Non-Fiction)
McGuffey's Fourth Eclectic Reader (Education/Reader, Non-Fiction)
The Story of Little Black Sambo, and The Story of Little Black Mingo (Children's Fiction, Tales)
The Haciendas of Mexico: An Artist's Record (History/Art, Non-Fiction)
Plato and the Other Companions of Sokrates, 3rd ed. Volume 3 (Philosophy, Non-Fiction)
Journal of a Residence on a Georgian Plantation: 1838-1839 (Autobiography/History, Non-Fiction)
The Argonautica (Poetry, Epic)
Voltaire's Philosophical Dictionary (Philosophy/Essays, Non-Fiction)
Surgical Anatomy (Medicine/Science, Non-Fiction)
An anthology of German literature (Literary Collection/Anthology)
A Concise Dictionary of Middle English from A.D. 1150 to 1580 (Reference/Dictionary, Non-Fiction)
The Art of Public Speaking (Self-Help/Communication, Non-Fiction)
Folk-Tales of Bengal (Folklore/Fiction, Short Stories)
Mother Goose's Nursery Rhymes: A Collection of Alphabets, Rhymes, Tales, and Jingles (Children's Poetry, Collection)
Women in the fine arts, from the Seventh Century B.C. to the Twentieth Century A.D. (History/Biography, Non-Fiction)
The Woman in White (Mystery Fiction, Novel)
The Sign of the Seven Sins (Fiction, Novel)
The World as Will and Idea (Vol. 3 of 3) (Philosophy, Non-Fiction)
The symbolism of Freemasonry : $b Illustrating and explaining its science and philosophy, its legends, myths and symbols (Occult/History, Non-Fiction)
The Philippines: Past and Present (Volume 2 of 2) (History, Non-Fiction)
Psychological Warfare (Military/Psychology, Non-Fiction)
Plutarch: Lives of the noble Grecians and Romans (Biography/History, Non-Fiction)
The Orbis Pictus (Education/Children's, Non-Fiction)
The origin and development of the moral ideas (Philosophy/Anthropology, Non-Fiction)
Familiar Quotations: A Collection of Passages, Phrases, and Proverbs Traced to; Their Sources in Ancient and Modern Literature (Reference/Literary, Non-Fiction)
The Sailor's Word-Book: An Alphabetical Digest of Nautical Terms, including Some More Especially Military and Scientific, but Useful to Seamen; as well as Archaisms of Early Voyagers, etc. (Reference/Nautical, Non-Fiction)
Heidi (Children's Fiction, Novel)
The Last of the Mohicans; A narrative of 1757 (Historical Fiction, Adventure Novel)
The Principles of Psychology, Volume 1 (of 2) (Psychology, Non-Fiction)
The Declaration of Independence of the United States of America (History/Politics, Non-Fiction)
The Well of Saint Clare (Fiction, Short Stories)
The Complete Poetic and Dramatic Works of Robert Browning: Cambridge Edition (Poetry/Drama Collection)
Tragic Sense Of Life (Philosophy/Religion, Non-Fiction)
A Short Biographical Dictionary of English Literature (Biography/Reference, Non-Fiction)
The best short stories of 1920, and the yearbook of the American short story (Fiction, Short Stories Collection)
A Short History of Freethought Ancient and Modern, Volume 2 of 2: Third edition, Revised and Expanded, in two volumes (History/Philosophy, Non-Fiction)
Myths and Legends of China (Folklore/Mythology, Non-Fiction)
The Rise of the Mediaeval Church: And Its Influence on the Civilization of Western Europe from the First to the Thirteenth Century (History/Religion, Non-Fiction)
The battles of the British Army : $b being a popular account of all the principal engagements during the last hundred years (History/Military, Non-Fiction)
Domestic Manners of the Americans (Travel/Sociology, Non-Fiction)
Kidnapped (Adventure Fiction, Novel)
Boy Scouts Handbook: The First Edition, 1911 (Education/Guide, Non-Fiction)
Complete Prose Works: Specimen Days and Collect, November Boughs and Goodbye My Fancy (Essays/Autobiography, Non-Fiction)
The Antiquarian Magazine & Bibliographer; Vol. 4, July-Dec 1884 (Literary Criticism/History, Non-Fiction Journal)
Democracy and Education: An Introduction to the Philosophy of Education (Education/Philosophy, Non-Fiction)
On the Duties of the Clergy (Religion/Theology, Non-Fiction)
The Poetical Works of William Wordsworth — Volume 3 (of 8) (Poetry Collection) [Note: Variant of previous list entry]
Diccionario Ingles-Español-Tagalog: Con partes de la oracion y pronunciacion figurada (Reference/Dictionary, Non-Fiction)
The Works of Lord Byron. Vol. 4 (Poetry Collection)
Nil Darpan; or, The Indigo Planting Mirror, A Drama.: Translated from the Bengali by a Native. (Drama/Play, Social Commentary)
The Interpretation of Dreams (Psychology, Non-Fiction)
The Legends of King Arthur and His Knights (Fiction, Arthurian Legend)
The Journal of Negro History, Volume 8, 1923 (History, Non-Fiction Journal)
The House of Mirth (Fiction, Social Novel)
The Gay Cockade (Fiction, Short Stories)
Politics: A Treatise on Government (Philosophy/Politics, Non-Fiction)
Letters on the equality of the sexes, and the condition of woman (Essays/Feminism, Non-Fiction)
Letters to His Son, Complete: On the Fine Art of Becoming a Man of the World and a Gentleman (Essays/Advice, Non-Fiction)
The Odyssey (Poetry, Epic) [Note: Variant of previous list entry]
The Danish History, Books I-IX (History/Mythology, Non-Fiction)
The Wonders of the Invisible World: Being an Account of the Tryals of Several Witches Lately Executed in New-England, to which is added A Farther Account of the Tryals of the New-England Witches (History/Occult, Non-Fiction)
Dio's Rome, Volume 1: An Historical Narrative Originally Composed in Greek during the Reigns of Septimius Severus, Geta and Caracalla, Macrinus, Elagabalus and Alexander Severus: and Now Presented in English Form (History, Non-Fiction)
The history of magic : $b including a clear and precise exposition of its procedure, its rites and its mysteries (Occult/History, Non-Fiction)
A Literary History of the English People, from the Origins to the Renaissance (Literary Criticism/History, Non-Fiction)
St. Benedict's Rule for Monasteries (Religion/Guide, Non-Fiction)
John F. Kennedy's Inaugural Address (History/Politics, Speech/Non-Fiction)
Pollyanna (Children's Fiction, Novel)
The Hindu-Arabic Numerals (History/Mathematics, Non-Fiction)
The Oriental Rug: A Monograph on Eastern Rugs and Carpets, Saddle-Bags, Mats & Pillows, with a Consideration of Kinds and Classes, Types, Borders, Figures, Dyes, Symbols, etc. Together with Some Practical Advice to Collectors. (Art/History, Non-Fiction)
How to sing [Meine Gesangskunst] (Music/Instruction, Non-Fiction)
Strange Stories from a Chinese Studio (Volumes 1 and 2) (Folklore/Fiction, Short Stories)
The Meditations of the Emperor Marcus Aurelius Antoninus: A new rendering based on the Foulis translation of 1742 (Philosophy, Non-Fiction)
The Sketch-Book of Geoffrey Crayon (Fiction, Essays/Short Stories)
On the Nature of Things (Philosophy/Poetry)
The Every Day Book of History and Chronology: Embracing the Anniversaries of Memorable Persons and Events in Every Period and State of the World, from the Creation to the Present Time (History/Reference, Non-Fiction)
On the Sublime (Literary Criticism, Non-Fiction)
Old and New London, Volume I: A Narrative of Its History, Its People, and Its Places (History/Travel, Non-Fiction)
The Churches and Modern Thought: An inquiry into the grounds of unbelief and an appeal for candour (Religion/Philosophy, Non-Fiction)
The Nibelungenlied: Translated into Rhymed English Verse in the Metre of the Original (Poetry, Epic)
The Instruction of Ptah-Hotep and the Instruction of Ke'Gemni: The Oldest Books in the World (Philosophy/Wisdom Literature, Non-Fiction)
Marmion: A Tale Of Flodden Field (Poetry, Narrative Verse)
Travels in the Central Parts of Indo-China (Siam), Cambodia, and Laos (Vol. 1 of 2): During the Years 1858, 1859, and 1860 (Travel/History, Non-Fiction)
Every Day Life in the Massachusetts Bay Colony (History, Non-Fiction)
Memoirs of General William T. Sherman — Complete (Autobiography/Military, Non-Fiction)
History of the Reformation in the Sixteenth Century, Volume 1 (History/Religion, Non-Fiction)
The Bible, King James Version, Complete Contents (Religion/Scripture, Non-Fiction)
The Frontier in American History (History, Non-Fiction)
The new science of space speech (Science/Non-Fiction, Speculative)
The Cook's Decameron: A Study in Taste, Containing over Two Hundred Recipes for Italian Dishes (Cookbook, Non-Fiction)
A Brief Account of the Destruction of the Indies: Or, a faithful NARRATIVE OF THE Horrid and Unexampled Massacres, Butcheries, and all manner of Cruelties, that Hell and Malice could invent, committed by the Popish Spanish Party on the inhabitants of West-India, TOGETHER With the Devastations of several Kingdoms in America by Fire and Sword, for the space of Forty and Two Years, from the time of its first Discovery by them. (History, Non-Fiction)
The Seven Lamps of Architecture (Architecture/Art Criticism, Non-Fiction)
Familiar Letters: The Writings of Henry David Thoreau, Volume 06 (of 20) (Letters/Essays, Non-Fiction)
Types of News Writing (Journalism/Education, Non-Fiction)
The Duchess of Malfi (Drama/Play, Tragedy)
Castes and Tribes of Southern India. Vol. 7 of 7 (Anthropology/History, Non-Fiction)
The French Revolution: A History (History, Non-Fiction)
Life on a Mediaeval Barony: A Picture of a Typical Feudal Community in the Thirteenth Century (History, Non-Fiction)
A Wanderer in Paris (Travel, Non-Fiction)
The Squire's Daughter: Being the First Book in the Chronicles of the Clintons (Fiction, Novel)
The Blue Fairy Book (Children's Fiction, Fairy Tales Collection)
Poems Every Child Should Know: The What-Every-Child-Should-Know-Library (Poetry Collection, Children's)
The Catacombs of Rome, and Their Testimony Relative to Primitive Christianity (History/Religion, Non-Fiction)
Cyrano de Bergerac (Drama/Play, Comedy)
Red Nails (Fantasy Fiction, Short Story)
Famous Modern Ghost Stories (Horror Fiction, Short Stories Collection)
Stage-coach and Tavern Days (History/Social, Non-Fiction)
A Classification and Subject Index for Cataloguing and Arranging the Books and Pamphlets of a Library (Library Science/Reference, Non-Fiction)
The Life of Charles Dickens, Vol. I-III, Complete (Biography, Non-Fiction)
Memoirs of Napoleon Bonaparte — Complete (Biography/History, Non-Fiction)
Fifty Years In The Northwest: With An Introduction And Appendix Containing Reminiscences, Incidents And Notes (History/Autobiography, Non-Fiction)
Grimm's Fairy Tales (Children's Fiction, Fairy Tales Collection) [Note: Variant of previous list entry]
Laughter: An Essay on the Meaning of the Comic (Philosophy/Humor, Non-Fiction)
The Letters of Wolfgang Amadeus Mozart — Volume 01 (Biography/Letters, Non-Fiction)
The Fables of La Fontaine: Translated into English Verse by Walter Thornbury and Illustrated by Gustave Doré (Poetry/Fables, Translation)
The Cumulative Book Review Digest, Volume 1, 1905: Complete in a single alphabet (Literary Criticism/Reference, Non-Fiction)
The History of Sulu (History, Non-Fiction)
Archæology and the Bible (Archaeology/Religion, Non-Fiction)
Miss Eden's Letters (Biography/Letters, Non-Fiction)
Giordano Bruno (Biography/Philosophy, Non-Fiction)
Poirot Investigates (Mystery Fiction, Short Stories)
The Anglo-Saxon Chronicle (History, Non-Fiction)
The Boy Mechanic, Volume 1: 700 Things for Boys to Do (Education/Crafts, Non-Fiction)
Cowboy Songs, and Other Frontier Ballads (Poetry/Folklore Collection)
The New Testament of our Lord and Savior Jesus Christ.: The common English version, corrected by the final committee of the American Bible Union. (Religion/Scripture, Non-Fiction)
Boswell's Life of Johnson: Abridged and edited, with an introduction by Charles Grosvenor Osgood (Biography, Non-Fiction)
The Rising Tide of Color Against White World-Supremacy (Politics/Sociology, Non-Fiction)
Wine, Women, and Song: Mediaeval Latin Students' songs; Now first translated into English verse (Poetry Collection, Translation)
Nobody (Fiction, Novel)
The carnal god (Fiction, Novel)
The Book of Household Management (Home Economics/Guide, Non-Fiction)
The Bronze Age and the Celtic World (Archaeology/History, Non-Fiction)
The Central Eskimo: Sixth Annual Report of the Bureau of Ethnology to the Secretary of the Smithsonian Institution, 1884-1885, Government Printing Office, Washington, 1888, pages 399-670 (Anthropology, Non-Fiction)
An Account of Egypt (History/Travel, Non-Fiction)
Argot and slang : $b a new French and English dictionary of the cant words, quaint expressions, slang terms and flash phrases used in the high and low life of old and new Paris (Reference/Dictionary, Non-Fiction)
A Short View of the Immorality, and Profaneness of the English Stage: Together with the Sense of Antiquity on this Argument (Literary Criticism, Non-Fiction)
Pals: Young Australians in Sport and Adventure (Adventure Fiction, Youth Novel)
British Goblins: Welsh Folk-lore, Fairy Mythology, Legends and Traditions (Folklore, Non-Fiction)
Russian Fairy Tales: A Choice Collection of Muscovite Folk-lore (Folklore/Fiction, Short Stories)
The Congo, and Other Poems (Poetry Collection)
Poems of American History (Poetry Collection)
Annals and Antiquities of Rajasthan, v. 1 of 3: or the Central and Western Rajput States of India (History, Non-Fiction)
Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume XIII, Oklahoma Narratives (History/Oral History, Non-Fiction)
The Swastika, the Earliest Known Symbol, and Its Migration: With Observations on the Migration of Certain Industries in Prehistoric Times (History/Symbolism, Non-Fiction)
Scurvy, Past and Present (Medicine/History, Non-Fiction)
Keats: Poems Published in 1820 (Poetry Collection) [Note: Duplicate from previous list]
The Way of the World (Drama/Play, Comedy)
The Mill on the Floss (Fiction, Novel)
Feudal England: Historical Studies on the Eleventh and Twelfth Centuries (History, Non-Fiction)
The Scarlet Pimpernel (Adventure Fiction, Novel)
Volpone; Or, The Fox (Drama/Play, Comedy)
The Moon and Sixpence (Fiction, Novel)
Woodworking Tools 1600-1900 (History/Technology, Non-Fiction)
Le Morte d'Arthur: Volume 2 (Fiction, Arthurian Legend/Romance)
The letters of Queen Victoria, vol. 1, 1837-1843 : $b A selection from Her Majesty's correspondence between the years 1837 and 1861. (Biography/Letters, Non-Fiction)
Roster of the Rainbow division (Forty-Second) Major General Wm. A. Mann commanding (History/Military, Non-Fiction)
The Wendigo (Horror Fiction, Short Story)
Church History (Volumes 1-3) (Religion/History, Non-Fiction)
Little Masterpieces of American Wit and Humor, Volume I (Humor/Fiction Collection)
Man a Machine (Philosophy/Science, Non-Fiction)
Domesday Book and Beyond: Three Essays in the Early History of England (History, Non-Fiction)
Myths and Legends of Our Own Land — Complete (Folklore/Fiction, Short Stories)
A Child's Garden of Verses (Poetry, Children's Collection) [Note: Variant of previous list entry]
Franklin's Way to Wealth; or, "Poor Richard Improved" (Self-Help/Economics, Non-Fiction)
Rubáiyát of Omar Khayyam, Rendered into English Verse (Poetry Collection)
London and the Kingdom - Volume 2: A History Derived Mainly from the Archives at Guildhall in the Custody of the Corporation of the City of London. (History, Non-Fiction)
The Book of the Thousand Nights and a Night — Volume 10 (of 10) (Folklore/Fiction, Short Stories)
The Red and the Black: A Chronicle of 1830 (Fiction, Novel)
The Conflict of Religions in the Early Roman Empire (History/Religion, Non-Fiction)
Prayers of the Early Church (Religion/Prayer, Non-Fiction)
The Angel in the House (Poetry, Narrative Verse)
The History of the First West India Regiment (History/Military, Non-Fiction)
British Flags: Their Early History, and Their Development at Sea: With an Account of the Origin of the Flag as a National Device (History, Non-Fiction)
The Friars in the Philippines (History/Religion, Non-Fiction)
Anarchism (Politics/Philosophy, Non-Fiction)
Letters of Pliny (Biography/Letters, Non-Fiction)
The History of Signboards, from the Earliest times to the Present Day (History/Cultural, Non-Fiction)
The Ego and His Own (Philosophy/Anarchism, Non-Fiction)
Fairy and Folk Tales of the Irish Peasantry (Folklore/Fiction, Short Stories)
Ars Amatoria; or, The Art Of Love: Literally Translated into English Prose, with Copious Notes (Poetry/Erotica, Translation)
The Amenities of Book-Collecting and Kindred Affections (Essays/Literary, Non-Fiction)
Memoirs of Louis XIV and His Court and of the Regency — Complete (Autobiography/History, Non-Fiction)
The Letters of Amerigo Vespucci, and Other Documents Illustrative of His Career (History/Letters, Non-Fiction)
The Art of Perfumery, and Methods of Obtaining the Odors of Plants: With Instructions for the Manufacture of Perfumes for the Handkerchief, Scented Powders, Odorous Vinegars, Dentifrices, Pomatums, Cosmetics, Perfumed Soap, Etc., to which is Added an Appendix on Preparing Artificial Fruit-Essences, Etc. (Science/Instruction, Non-Fiction)
Frederick Chopin, as a Man and Musician — Complete (Biography, Non-Fiction)
Crimes of Preachers in the United States and Canada (History/Social, Non-Fiction)
Principles of Political Economy, Vol. 2 (Economics, Non-Fiction)
The Russian Army and the Japanese War, Vol. 2 (of 2): Being Historical and Critical Comments on the Military Policy and Power of Russia and on the Campaign in the Far East (History/Military, Non-Fiction)
An Essay on Man; Moral Essays and Satires (Poetry, Philosophical Verse)
The Jargon File, Version 4.2.2, 20 Aug 2000 (Reference/Technology, Non-Fiction)
Myths of the Norsemen: From the Eddas and Sagas (Mythology/Folklore, Non-Fiction)
West African Folk-Tales (Folklore/Fiction, Short Stories)
Historic Jamaica (History, Non-Fiction)
The Sufistic Quatrains of Omar Khayyam (Poetry Collection)
History, Manners, and Customs of the Indian Nations Who Once Inhabited Pennsylvania and the Neighbouring States. (Anthropology/History, Non-Fiction)
The Devil is an Ass (Drama/Play, Comedy)
Self Help; with Illustrations of Conduct and Perseverance (Self-Help, Non-Fiction)
The Book of Tea (Philosophy/Cultural, Non-Fiction)
Life of Harriet Beecher Stowe: Compiled From Her Letters and Journals by Her Son Charles Edward Stowe (Biography, Non-Fiction)
The Letters of Cassiodorus: Being a Condensed Translation of the Variae Epistolae of Magnus Aurelius Cassiodorus Senator (History/Letters, Non-Fiction)
Humour, Wit, & Satire of the Seventeenth Century (Humor/Literary Collection)
Life's Handicap: Being Stories of Mine Own People (Fiction, Short Stories)
Young's Night Thoughts: With Life, Critical Dissertation and Explanatory Notes (Poetry, Philosophical Verse)
The house at Pooh Corner (Children's Fiction, Animal Tales)
The Gentleman: A Romance of the Sea (Fiction, Adventure Novel)
The Bābur-nāma in English (Memoirs of Bābur) (Autobiography/History, Non-Fiction)
Cyropaedia: The Education of Cyrus (History/Philosophy, Non-Fiction)
The English Lakes (Travel/Nature, Non-Fiction)
The Spectator, Volumes 1, 2 and 3: With Translations and Index for the Series (Essays/Literary, Non-Fiction)
A Practical Guide to Self-Hypnosis (Self-Help/Psychology, Non-Fiction)
English Fairy Tales (Children's Fiction, Fairy Tales Collection)
An Englishman in Paris: Notes and Recollections (Autobiography/Travel, Non-Fiction)
The Chronicle of the Discovery and Conquest of Guinea. Vol. I (History/Exploration, Non-Fiction)
The New Life (La Vita Nuova) (Poetry/Autobiography)
Hidden Symbolism of Alchemy and the Occult Arts (Occult/Philosophy, Non-Fiction)
Parish Priests and Their People in the Middle Ages in England (History/Religion, Non-Fiction)
A dictionary of American authors (Reference/Biography, Non-Fiction)
Entretiens / Interviews / Entrevistas (Interviews/Literary, Non-Fiction)
Jane's All the World's Aircraft. 1913 (Aviation/History, Non-Fiction)
The Katipunan; or, The Rise and Fall of the Filipino Commune (History/Politics, Non-Fiction)
Johann Sebastian Bach: His Life, Art, and Work (Biography, Non-Fiction)
The Geography of Strabo, Volume 2 (of 3): Literally Translated, with Notes (Geography/History, Non-Fiction)
Flatland: A Romance of Many Dimensions (Science Fiction, Satirical Novel)
Tradition, Principally with Reference to Mythology and the Law of Nations (History/Law, Non-Fiction)
The Itinerary of Benjamin of Tudela (Travel/History, Non-Fiction)
The Philippine Islands, 1493-1898; Volume 46, 1721-1739: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century (History/Exploration, Non-Fiction)
Now we are six (Children's Poetry, Collection)
Printers' Marks: A Chapter in the History of Typography (History/Art, Non-Fiction)
The Chronicle of the Discovery and Conquest of Guinea. Vol. II (History/Exploration, Non-Fiction)
A Visit to the Philippine Islands (Travel/History, Non-Fiction)
The Jesuit Relations and Allied Documents, Vol. 1: Acadia, 1610-1613 (History/Religion, Non-Fiction)
Tales From Jókai (Fiction, Short Stories)
The Arabian Nights: Their Best-known Tales (Folklore/Fiction, Short Stories Collection)
Chaucer's Works, Volume 1 — Romaunt of the Rose; Minor Poems (Poetry Collection)
The Great God Pan (Horror Fiction, Novella)
Petrarch's Letters to Classical Authors (Biography/Letters, Non-Fiction)
Argonauts of the Western Pacific: An Account of Native Enterprise and Adventure in the Archipelagoes of Melanesian New Guinea (Anthropology, Non-Fiction)
A History of Literary Criticism in the Renaissance: With special reference to the influence of Italy in the formation and development of modern classicism (Literary Criticism/History, Non-Fiction)
Fifty Famous Stories Retold (Children's Fiction, Short Stories)
England, Picturesque and Descriptive: A Reminiscence of Foreign Travel (Travel/History, Non-Fiction)
The Bondage and Travels of Johann Schiltberger, a Native of Bavaria, in Europe, Asia, and Africa, 1396-1427 (Travel/Autobiography, Non-Fiction)
Institutes of the Christian Religion (Vol. 1 of 2) (Religion/Theology, Non-Fiction)
Struggles and Triumphs: or, Forty Years' Recollections of P. T. Barnum (Autobiography, Non-Fiction)
Ireland under the Tudors, with a Succinct Account of the Earlier History. Vol. 3 (of 3) (History, Non-Fiction)
The Witch-cult in Western Europe: A Study in Anthropology (Anthropology/Occult, Non-Fiction)
Manifesto of the Communist Party (Politics/Philosophy, Non-Fiction)
Horton Genealogy: or Chronicles of the Descendants of Barnabas Horton, of Southold, L. I., 1640. (Genealogy/History, Non-Fiction)
An Illustrated Dictionary of Words used in Art and Archaeology: Explaining terms frequently used in works on architecture, arms, bronzes, Christian art, colour, costume, decoration, devices, emblems, heraldry, lace, personal ornaments, pottery, painting, sculpture, &c., with their derivations. (Reference/Art, Non-Fiction)
An Essay on the Development of Christian Doctrine (Religion/Theology, Non-Fiction)
Japanese Girls and Women: Revised and Enlarged Edition (Sociology/Cultural, Non-Fiction)
Lucian's True History (Fiction, Satirical Fantasy)
The Wonderful "One-Hoss-Shay", and Other Poems (Poetry Collection)
My Bondage and My Freedom (Autobiography, Non-Fiction)
The Ancient Irish Epic Tale Táin Bó Cúalnge (Poetry/Epic, Folklore)
Japanese Fairy Tales (Folklore/Fiction, Short Stories)
The Writings of Thomas Paine, Complete: With Index to Volumes I - IV (Politics/Essays, Non-Fiction)
The Tenant of Wildfell Hall (Fiction, Novel)
Women's Bathing and Swimming Costume in the United States (History/Fashion, Non-Fiction)
The Kādambarī of Bāṇa (Fiction, Romance Novel)
The Poems of John Donne, Volume 1 (of 2): Edited from the Old Editions and Numerous Manuscripts (Poetry Collection)
Heimskringla; Or, The Chronicle of the Kings of Norway (History/Mythology, Non-Fiction)
Castes and Tribes of Southern India. Vol. 1 of 7 (Anthropology/History, Non-Fiction)
Letters of Samuel Taylor Coleridge, Vol. 2 (of 2) (Biography/Letters, Non-Fiction)
The Post Office (Drama/Play)
The Planters of Colonial Virginia (History, Non-Fiction)
History of Lace (History/Fashion, Non-Fiction)
Twas the Night before Christmas: A Visit from St. Nicholas (Poetry, Children's)
The Poor Plutocrats (Fiction, Novel)
Cole's Funny Picture Book No. 1 (Children's Humor, Illustrated)
Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume XVI, Texas Narratives, Part 1 (History/Oral History, Non-Fiction)
Argentina, Legend and History (History, Non-Fiction)
Every Man in His Humor (Drama/Play, Comedy)
Slang and its analogues past and present, volume 3 (of 7) : $b A dictionary, historical and comparative, of the heterodox speech of all classes of society for more than three hundred years. With synonyms in English, French, German, Italian, etc. (Reference/Dictionary, Non-Fiction)
Dio's Rome, Volume 5: An Historical Narrative Originally Composed in Greek During The Reigns of Septimius Severus, Geta and Caracalla, Macrinus, Elagabalus and Alexander Severus: and Now Presented in English Form (History, Non-Fiction)
An Enquiry Concerning the Principles of Morals (Philosophy, Non-Fiction)
The Mantle, and Other Stories (Fiction, Short Stories)
Harriet, the Moses of Her People (Biography/History, Non-Fiction)
The Advancement of Learning (Philosophy/Education, Non-Fiction)
Henry of Ofterdingen: A Romance. (Fiction, Romance Novel)
The Home Book of Verse — Volume 4 (Poetry Collection)
The Curiosities of Heraldry (History/Reference, Non-Fiction)
The Queen of Hearts (Fiction, Novel)
Isis unveiled, Volume 1 (of 2), Science : $b A master-key to mysteries of ancient and modern science and theology (Occult/Religion, Non-Fiction)
The Expositor's Bible: The Book of the Twelve Prophets, Vol. 2: Commonly Called the Minor (Religion/Commentary, Non-Fiction)
The Inhabitants of the Philippines (History/Anthropology, Non-Fiction)
She (Adventure Fiction, Novel)
Philippine progress prior to 1898 : $b a source book of Philippine history to supply a fairer view of Filipino participation and supplement the defective Spanish accounts (History, Non-Fiction)
A Cyclopaedia of Canadian Biography: Being Chiefly Men of the Time: A Collection of Persons Distinguished in Professional and Political Life, Leaders in the Commerce and Industry of Canada, and Successful Pioneers (Biography/Reference, Non-Fiction)
The Phoenix and the Carpet (Children's Fiction, Fantasy Novel)
Doña Perfecta (Fiction, Novel)
History of Linn County Iowa: From Its Earliest Settlement to the Present Time (History, Non-Fiction)
Poems of James Russell Lowell: With biographical sketch by Nathan Haskell Dole (Poetry Collection)
The Fairy Mythology: Illustrative of the Romance and Superstition of Various Countries (Folklore, Non-Fiction)
The Education of Henry Adams (Autobiography, Non-Fiction)
Pushing to the Front (Self-Help, Non-Fiction)
The Civilisation of the Renaissance in Italy (History/Cultural, Non-Fiction)
Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume III, Florida Narratives (History/Oral History, Non-Fiction)
Twice-told tales (Fiction, Short Stories)
The Underground Railroad: A Record of Facts, Authentic Narratives, Letters, &c., Narrating the Hardships, Hair-Breadth Escapes and Death Struggles of the Slaves in Their Efforts for Freedom, As Related by Themselves and Others, or Witnessed by the Author. (History, Non-Fiction)
The Life of John Marshall, Volume 2: Politician, diplomatist, statesman, 1789-1801 (Biography/History, Non-Fiction)
Henry IV, Makers of History (Biography/History, Non-Fiction)
Astounding Stories of Super-Science February 1930 (Science Fiction, Magazine/Short Stories)
Seneca myths and folk tales (Folklore, Non-Fiction)
An Introduction to Entomology: Vol. 1: or Elements of the Natural History of the Insects (Science/Entomology, Non-Fiction)
Ireland in Fiction: A Guide to Irish Novels, Tales, Romances, and Folk-lore (Literary Criticism/Reference, Non-Fiction)
Plotinos: Complete Works, v. 4: In Chronological Order, Grouped in Four Periods (Philosophy, Non-Fiction)
The History of Prostitution: Its Extent, Causes, and Effects throughout the World (History/Sociology, Non-Fiction)
The Philippine Islands: A Political, Geographical, Ethnographical, Social and Commercial History of the Philippine Archipelago, Embracing the Whole Period of Spanish Rule (History, Non-Fiction)
A Selection from the Works of Frederick Locker (Poetry Collection)
Galicia, the Switzerland of Spain (Travel/History, Non-Fiction)
Thoughts of Marcus Aurelius Antoninus (Philosophy, Non-Fiction)
The Part Borne by the Dutch in the Discovery of Australia 1606-1765 (History/Exploration, Non-Fiction)
Discourses on the First Decade of Titus Livius (History/Politics, Non-Fiction)
Practical taxidermy : $b a manual of instruction to the amateur in collecting, preserving, and setting up natural history specimens of all kinds. To which is added a chapter upon the pictorial arrangement of museums. With additional instructions in modelling and artistic taxidermy. (Science/Instruction, Non-Fiction)
The Great Illusion: A Study of the Relation of Military Power to National Advantage (Politics/Economics, Non-Fiction)
Plant Lore, Legends, and Lyrics: Embracing the Myths, Traditions, Superstitions, and Folk-Lore of the Plant Kingdom (Folklore/Botany, Non-Fiction)
Mythology among the Hebrews and Its Historical Development (Mythology/History, Non-Fiction)
The Younger Set (Fiction, Novel)
Mount Everest, the Reconnaissance, 1921 (Travel/Exploration, Non-Fiction)
Letters to the young from the Old World : $b Notes of travel (Travel/Letters, Non-Fiction)
The Blue Goose (Fiction, Novel)
The call of Cthulhu (Horror Fiction, Short Story)
Household Tales by Brothers Grimm (Children's Fiction, Fairy Tales Collection)
The Lusiad; Or, The Discovery of India, an Epic Poem (Poetry, Epic)
Secret Service Under Pitt (History/Espionage, Non-Fiction)
Man and Nature; Or, Physical Geography as Modified by Human Action (Geography/Environment, Non-Fiction)
The Hour of the Dragon (Fantasy Fiction, Novel)
A Cyclopædia of Canadian Biography: Brief biographies of persons distinguished in the professional, military and political life, and the commerce and industry of Canada, in the twentieth century (Biography/Reference, Non-Fiction)
The Letters of Cicero, Volume 1: The Whole Extant Correspodence in Chronological Order (Biography/Letters, Non-Fiction)
A Collection of Essays and Fugitiv Writings: On Moral, Historical, Political, and Literary Subjects (Essays/Politics, Non-Fiction)
Mr. Punch's Golf Stories (Humor/Fiction, Short Stories)
The whole truth and nothing but (Autobiography, Non-Fiction)
The Nuttall Encyclopædia: Being a Concise and Comprehensive Dictionary of General Knowledge (Reference/Encyclopedia, Non-Fiction)
"""

# Build dictionary
genre_map = {}
for line in titles_with_genres.splitlines():
    line = line.strip()
    if line.endswith(")"):  # only process lines with parentheses
        try:
            title, genre = line.rsplit("(", 1)   # split only at the last "("
            genre = genre.strip(") ")            # remove closing parenthesis
            genre_map[title.strip()] = genre.strip()
        except ValueError:
            pass  # skip malformed lines

# Print in nice dict format
print("# Define the mapping of title → genre")
print("genre_map = {")
for title, genre in genre_map.items():
    print(f'    "{title}": "{genre}",')
print("}")

# Define the mapping of title → genre
genre_map = {
    "Peter Pan : $b [Peter and Wendy]": "Children's Fiction, Fantasy Novel",
    "Justice": "Philosophy/Non-Fiction",
    "Meditations": "Philosophy/Autobiography, Non-Fiction",
    "Don't Marry; or, Advice on How, When and Who to Marry": "Self-Help/Advice, Non-Fiction",
    "Manual of Classical Erotology (De figuris Veneris)": "Erotica/Philosophy, Non-Fiction",
    "Cookery and Dining in Imperial Rome": "Cookbook/History, Non-Fiction",
    "The Mahabharata of Krishna-Dwaipayana Vyasa, Volume 1: Books 1, 2 and 3": "Poetry/Epic, Mythology/Religion",
    "The Thousand and One Nights, Vol. I.: Commonly Called the Arabian Nights' Entertainments": "Folklore/Fiction, Short Stories Collection",
    "On the Origin of Species By Means of Natural Selection: Or, the Preservation of Favoured Races in the Struggle for Life": "Science/Biology, Non-Fiction",
    "The Wonderful Wizard of Oz": "Children's Fiction, Fantasy Novel",
    "The Convent Scho

In [6]:
import re

# Example text block containing titles with (genre)
titles_with_genres = """
The History of Modern Painting, Volume 1 (of 4): Revised edition continued by the author to the end of the XIX century (Art/History, Non-Fiction)
Lays of Ancient Rome (Poetry, Historical Verse)
Jerusalem Explored, Volume 1—Text: Being a Description of the Ancient and Modern City, with Numerous Illustrations Consisting of Views, Ground Plans and Sections (History/Travel, Non-Fiction)
The passing of the great race; or, The racial basis of European history (History/Sociology, Non-Fiction) [Note: Controversial eugenics text]
The Gutenberg Webster's Unabridged Dictionary: Section X, Y, and Z (Reference/Dictionary, Non-Fiction)
The Box-Car Children (Children's Fiction, Adventure Novel)
Sketches and Studies in Italy and Greece, Complete: Series I, II, and III (Travel/Essays, Non-Fiction)
The Hacker Crackdown: Law and Disorder on the Electronic Frontier (Technology/History, Non-Fiction)
A collection of Latin maxims and phrases literally translated: Intended for the use of students for all legal examinations (Reference/Law, Non-Fiction)
Richard III: His Life & Character, Reviewed in the Light of Recent Research (Biography/History, Non-Fiction)
A guide book of art, architecture, and historic interests in Pennsylvania (Art/History/Guide, Non-Fiction)
History of Mexico, Volume 1, 1516-1521 (History, Non-Fiction)
Ernest Maltravers — Complete (Fiction, Novel)
The Idea of a University Defined and Illustrated: In Nine Discourses Delivered to the Catholics of Dublin (Education/Philosophy, Non-Fiction)
The Surprising Adventures of Baron Munchausen (Fiction, Satirical Adventure Novel)
History of Civilization in England, Vol. 2 of 3 (History, Non-Fiction)
The Organism as a Whole, from a Physicochemical Viewpoint (Science/Biology, Non-Fiction)
Narrative and Critical History of America, Vol. 3 (of 8): English Explorations and Settlements in North America 1497-1689 (History/Exploration, Non-Fiction)
A Japanese Boy (Autobiography/Cultural, Non-Fiction)
Tremendous Trifles (Essays/Humor, Non-Fiction)
Plotinos: Complete Works, v. 3: In Chronological Order, Grouped in Four Periods (Philosophy, Non-Fiction)
Cautionary Tales for Children (Children's Poetry, Satirical Verse)
Hegel's Lectures on the History of Philosophy: Volume 1 (of 3) (Philosophy/History, Non-Fiction)
Essays — First Series (Essays/Philosophy, Non-Fiction) [Note: Variant of previous list entry]
Lilith: A Romance (Fiction, Fantasy Novel)
Hector Berlioz: A Romantic Tragedy (Biography, Non-Fiction)
History, Gazetteer, and Directory of Shropshire (History/Reference, Non-Fiction)
Lives of the Most Eminent Painters Sculptors and Architects, Vol. 01 (of 10): Cimabue to Agnolo Gaddi (Biography/Art History, Non-Fiction)
Hossfeld's Spanish Dialogues, and Idiomatic Phrases Indispensible for a Rapid Acquisition of the Spanish Language (Education/Language, Non-Fiction)
The Story of Mankind (History, Non-Fiction for Youth)
The Works of the Right Honourable Edmund Burke, Vol. 03 (of 12) (Politics/Essays, Non-Fiction)
Lady Audley's Secret (Mystery Fiction, Novel)
The Monk: A Romance (Fiction, Gothic Novel)
The History of Cuba, vol. 2 (History, Non-Fiction) [Note: Variant of previous list entry]
The Oera Linda Book, from a Manuscript of the Thirteenth Century (Folklore/History, Non-Fiction) [Note: Considered a hoax by some]
Alchemy: Ancient and Modern: Being a Brief Account of the Alchemistic Doctrines, and Their Relations, to Mysticism on the One Hand, and to Recent Discoveries in Physical Science on the Other Hand; Together with Some Particulars Regarding the Lives and Teachings of the Most Noted Alchemists (Science/History/Occult, Non-Fiction)
The story hunter : $b or, Tales of the weird and wild (Fiction, Short Stories/Horror)
The Princess and the Goblin (Children's Fiction, Fantasy Novel)
The Molly Maguires and the detectives (History/Crime, Non-Fiction)
Dickens and His Illustrators: Cruikshank, Seymour, Buss, "Phiz," Cattermole, Leech, Doyle, Stanfield, Maclise, Tenniel, Frank Stone, Landseer, Palmer, Topham, Marcus Stone, and Luke Fildes; 2nd. Ed. (Biography/Art History, Non-Fiction)
The origin of the family, private property, and the state (Sociology/Philosophy, Non-Fiction)
Ancient Monuments of the Mississippi Valley: Smithsonian Contributions to Knowledge, Vol. I. (Archaeology/History, Non-Fiction)
Flappers and Philosophers (Fiction, Short Stories)
Human Nature and Conduct: An introduction to social psychology (Psychology/Sociology, Non-Fiction)
The Negro in the United States; a selected bibliography. Compiled by Dorothy B. Porter (Reference/History, Non-Fiction)
A Literary History of the Arabs (Literary Criticism/History, Non-Fiction)
The Hyborian Age (Fantasy Fiction, Essay/Short Story)
India: What can it teach us?: A Course of Lectures Delivered before the University Of Cambridge (History/Cultural, Non-Fiction)
An Old Babylonian Version of the Gilgamesh Epic (Mythology/Poetry, Translation)
Financial Crime and Corruption (Economics/Crime, Non-Fiction)
The Lani People (Science Fiction, Novel)
Faust: a Tragedy [part 1], Translated from the German of Goethe (Drama/Poetry, Tragedy) [Note: Variant of previous list entry]
Bloodletting Instruments in the National Museum of History and Technology (History/Medicine, Non-Fiction)
The Histories of Polybius, Vol. 2 (of 2) (History, Non-Fiction)
The Swiss Family Robinson (Adventure Fiction, Novel) [Note: Variant of previous list entry]
The Hanging Stranger (Science Fiction, Short Story)
The Four Million (Fiction, Short Stories)
On the Old Road Vol. 1 (of 2): A Collection of Miscellaneous Essays and Articles on Art and Literature (Essays/Literary Criticism, Non-Fiction)
Tamburlaine the Great — Part 1 (Drama/Play, Tragedy)
A History of Lancashire (History, Non-Fiction)
Index of the Project Gutenberg Works of O. Henry (Reference/Literary, Non-Fiction)
The History of Herodotus — Volume 2 (History, Non-Fiction) [Note: Variant of previous list entry]
A History of Epidemics in Britain, Volume 1 (of 2): From A.D. 664 to the Extinction of Plague (History/Medicine, Non-Fiction)
Aids to Reflection; and, The Confessions of an Inquiring Spirit (Philosophy/Religion, Non-Fiction)
Montreal, 1535-1914. Vol. 2. Under British Rule, 1760-1914 (History, Non-Fiction)
Areopagitica: A Speech for the Liberty of Unlicensed Printing to the Parliament of England (Essays/Politics, Non-Fiction)
The Virginian: A Horseman of the Plains (Fiction, Western Novel)
The first voyage round the world, by Magellan : $b translated from the accounts of Pigafetta, and other contemporary writers (History/Exploration, Non-Fiction)
How music grew, from prehistoric times to the present day (Music/History, Non-Fiction)
The Marvelous Land of Oz (Children's Fiction, Fantasy Novel)
How I found Livingstone : $b Travels, adventures, and discoveres in Central Africa, including an account of four months' residence with Dr. Livingstone (Travel/Exploration, Non-Fiction)
A History of the Inquisition of Spain; vol. 1 (History/Religion, Non-Fiction)
Phantastes: A Faerie Romance for Men and Women (Fiction, Fantasy Novel)
The Economist (Economics/Non-Fiction)
The Fables of Phædrus: Literally translated into English prose with notes (Poetry/Fables, Translation)
The Poems of Jonathan Swift, D.D., Volume 2 (Poetry Collection)
Letters of Asa Gray; Vol. 2 (Biography/Letters, Non-Fiction)
A Contribution to the Critique of Political Economy (Economics/Philosophy, Non-Fiction)
Virginia (Fiction, Novel)
On The Principles of Political Economy, and Taxation (Economics, Non-Fiction)
Spanish Explorers in the Southern United States, 1528-1543.: The Narrative of Alvar Nunez Cabeca de Vaca. The Narrative of the Expedition of Hernando De Soto by the Gentleman of Elvas (History/Exploration, Non-Fiction)
The Churches of Paris, from Clovis to Charles X (History/Architecture, Non-Fiction)
A History of Matrimonial Institutions, Vol. 1 of 3 (History/Sociology, Non-Fiction)
A History of Inventions, Discoveries, and Origins, Volume 1 (of 2) (History/Science, Non-Fiction)
Tarrano the Conqueror (Science Fiction, Novel)
I. Beówulf: an Anglo-Saxon poem. II. The fight at Finnsburh: a fragment. (Poetry/Epic)
A Budget of Paradoxes, Volume I (Mathematics/History, Non-Fiction)
The letters of Hart Crane, 1916-1932 (Biography/Letters, Non-Fiction)
Chaucer and His England (Biography/History, Non-Fiction)
Northern Nut Growers Association Report of the Proceedings at the 44th Annual Meeting: Rochester, N.Y. August 31 and September 1, 1953 (Science/Agriculture, Non-Fiction Report)
The Memoirs of François René Vicomte de Chateaubriand sometime Ambassador to England, Volume 1 (of 6): Mémoires d'outre-tombe, volume 1 (Autobiography/History, Non-Fiction)
The Christ: A Critical Review and Analysis of the Evidences of His Existence (Religion/History, Non-Fiction)
The Imaginary Invalid (Drama/Play, Comedy)
Shakespeare's environment (Literary Criticism/History, Non-Fiction)
Tacitus: The Histories, Volumes I and II (History, Non-Fiction)
Asser's Life of King Alfred (Biography/History, Non-Fiction)
King Spruce, A Novel (Fiction, Novel)
Zoological Mythology; or, The Legends of Animals, Volume 2 (of 2) (Folklore/Mythology, Non-Fiction)
North Dakota: A Guide to the Northern Prairie State (Travel/Guide, Non-Fiction)
London Labour and the London Poor, Vol. 1 (Sociology/History, Non-Fiction)
The seventh shot : $b A detective story (Mystery Fiction, Novel)
The Agamemnon of Aeschylus: Translated into English Rhyming Verse with Explanatory Notes (Drama/Play, Tragedy)
The Discoverie of Witchcraft (Occult/History, Non-Fiction)
Round the Corner in Gay Street (Fiction, Novel)
Plymouth memories of an octogenarian (Autobiography/History, Non-Fiction)
Latin for Beginners (Education/Language, Non-Fiction)
Nature Books Popular and Scientific from The Franklin Bookshop, 1910: Catalogue 24, 1910-11 Season (Reference/Bibliography, Non-Fiction)
Handbook to the Mennonite Hymnary (Religion/Music, Non-Fiction)
"De Bello Gallico" and Other Commentaries (History/Military, Non-Fiction)
The Philippine Islands, 1493-1898, Volume 35, 1640-1649: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century (History/Exploration, Non-Fiction)
Christina of Denmark, Duchess of Milan and Lorraine, 1522-1590 (Biography/History, Non-Fiction)
The Metamorphoses of Ovid, Books VIII-XV (Poetry/Mythology, Translation)
The language of flowers : $b The floral offering ; a token of affection and esteem ; comprising the language and poetry of flowers (Reference/Cultural, Non-Fiction)
De Orbe Novo, Volume 1 (of 2): The Eight Decades of Peter Martyr D'Anghera (History/Exploration, Non-Fiction)
The Bacchae of Euripides (Drama/Play, Tragedy)
The American Joe Miller: A Collection of Yankee Wit and Humor (Humor/Folklore, Non-Fiction Collection)
Nineteen Centuries of Drink in England: A History (History/Cultural, Non-Fiction)
Home Geography for Primary Grades (Education/Geography, Non-Fiction)
History of Woman Suffrage, Volume I (History/Feminism, Non-Fiction)
The Hungry Stones, and Other Stories (Fiction, Short Stories)
Against the Grain (Fiction, Decadent Novel)
Raymond; or, Life and Death: With examples of the evidence for survival of memory and affection after death. (Spirituality/Non-Fiction)
Ancient Nahuatl Poetry, Containing the Nahuatl Text of XXVII Ancient Mexican Poems: Brinton's Library of Aboriginal American Literature Number VII. (Poetry Collection, Translation)
The Stones of Venice, Volume 2 (of 3), (Art/Architecture Criticism, Non-Fiction)
Steam-ships : $b The story of their development to the present day (History/Technology, Non-Fiction)
Alice's Adventures in Wonderland: Illustrated by Arthur Rackham. With a Proem by Austin Dobson (Children's Fiction, Fantasy Novel) [Note: Variant of Alice in Wonderland]
The Yeoman Adventurer (Adventure Fiction, Novel)
Magic and Religion (Anthropology/Religion, Non-Fiction)
France and England in North America, Part III: La Salle, Discovery of The Great West (History/Exploration, Non-Fiction)
Lost Face (Fiction, Short Stories)
France in the Nineteenth Century (History, Non-Fiction)
The Riddle of the Sands (Adventure Fiction, Spy Novel)
The Arabian Nights Entertainments (Folklore/Fiction, Short Stories Collection)
Vanishing England (History/Architecture, Non-Fiction)
Cicero: Letters to Atticus, Vol. 2 of 3 (Biography/Letters, Non-Fiction)
My Musical Life (Autobiography/Music, Non-Fiction)
Erewhon; Or, Over the Range (Fiction, Satirical Utopian Novel)
Nietzsche and Art (Philosophy/Art Criticism, Non-Fiction)
The magnificent possession (Fiction, Novel)
Journeys and Experiences in Argentina, Paraguay, and Chile: Including a Side Trip to the Source of the Paraguay River in the State of Matto Grosso, Brazil, and a Journey Across the Andes to the Rio Tambo in Peru (Travel/Adventure, Non-Fiction)
Smithsonian Institution - United States National Museum - Bulletin 240: Contributions From the Museum of History and Technology; Papers 34-44 on Science and Technology (Science/History, Non-Fiction)
Beyond the hills (Fiction, Novel)
Inventions in the Century (History/Technology, Non-Fiction)
The Story of Magellan and The Discovery of the Philippines (History/Exploration, Non-Fiction)
The Sorrows of Satan: or, The Strange Experience of One Geoffrey Tempest, Millionaire: A Romance (Fiction, Fantasy Novel)
Library of the World's Best Literature, Ancient and Modern — Volume 16 (Literary Anthology)
The decline of the West : $b Volume 1, Form and actuality (Philosophy/History, Non-Fiction)
Annals of the Bodleian Library, Oxford, A.D. 1598-A.D. 1867: With a Preliminary Notice of the earlier Library founded in the Fourteenth Century (History/Literary, Non-Fiction)
The German Terror in Belgium: An Historical Record (History/War, Non-Fiction)
Curiosities of Literature, Vol. 2 (Literary Criticism/Essays, Non-Fiction)
Repertory of The Comedie Humaine, Complete, A — Z (Reference/Literary, Non-Fiction)
The Old Inns of Old England, Volume 1 (of 2): A Picturesque Account of the Ancient and Storied Hostelries of Our Own Country (History/Travel, Non-Fiction)
The Doctor's Dilemma: Preface on Doctors (Drama/Play, Comedy)
Memoirs of the Reign of King George the Third, Volume 1 (of 4) (History/Biography, Non-Fiction)
The Story of London (History, Non-Fiction)
The Witness of the Stars (Religion/Astrology, Non-Fiction)
The Tao Teh King, or the Tao and its Characteristics (Philosophy/Religion, Non-Fiction)
The Formation of Christendom, Volume II (Religion/History, Non-Fiction)
The Races of Man: An Outline of Anthropology and Ethnography (Anthropology, Non-Fiction)
The massacre of St. Bartholomew : $b Preceded by a history of the religious wars in the reign of Charles IX (History/Religion, Non-Fiction)
The Regimental Records of the British Army: A historical résumé chronologically arranged of titles, campaigns, honours, uniforms, facings, badges, nicknames, etc. (History/Military, Non-Fiction)
The Book of the Damned (Science/Occult, Non-Fiction)
The Life and Amours of the Beautiful, Gay and Dashing Kate Percival: The Belle of the Delaware (Fiction, Erotic Memoir)
Vikram and the Vampire: Classic Hindu Tales of Adventure, Magic, and Romance (Folklore/Fiction, Short Stories)
Blue eyes and diamonds (Fiction, Novel)
The Middle English Poem, Erthe Upon Erthe (Poetry, Medieval Verse)
Random Reminiscences of Men and Events (Autobiography/Business, Non-Fiction)
Pan Tadeusz; or, The last foray in Lithuania: a story of life among Polish gentlefolk in the years 1811 and 1812 (Poetry, Epic)
The Romance of Plant Life: Interesting Descriptions of the Strange and Curious in the Plant World (Science/Botany, Non-Fiction)
Lincoln's yarns and stories : $b A complete collection of the funny and witty anecdotes that made Lincoln famous as America's greatest story teller (Humor/History, Non-Fiction Collection)
The International Jew, the world's foremost problem [volume I] : $b being a reprint of a series of articles appearing in the Dearborn Independent from May 22 to October 2, 1920 (Politics/Antisemitic, Non-Fiction) [Note: Controversial and biased text]
Guernsey Folk Lore: a collection of popular superstitions, legendary tales, peculiar customs, proverbs, weather sayings, etc., of the people of that island (Folklore, Non-Fiction)
English Surnames: Their Sources and Significations (History/Linguistics, Non-Fiction)
The Tale of Benjamin Bunny (Children's Fiction, Animal Tale)
A Manual of American Literature (Literary Criticism/History, Non-Fiction)
Buddhist birth stories; or, Jataka tales, Volume 1 (Folklore/Religion, Non-Fiction)
The Works of Horace (Poetry Collection, Translation)
The Goddess of Atvatabar: Being the history of the discovery of the interior world and conquest of Atvatabar (Science Fiction, Adventure Novel)
A Military Dictionary and Gazetteer: Comprising ancient and modern military technical terms, historical accounts of all North American Indians, as well as ancient warlike tribes; also notices of battles from the earliest period to the present time, with a concise explanation of terms used in heraldry and the offices thereof. The work also gives valuable geographical information. Compiled from the best authorities of all nations. With an appendix containing the Articles of war, etc. (Reference/Military, Non-Fiction)
Æschylos Tragedies and Fragments (Drama/Plays, Tragedy Collection)
The Romance of Tristan and Iseult (Fiction, Arthurian Romance)
Complete Hypnotism, Mesmerism, Mind-Reading and Spiritualism: How to Hypnotize: Being an Exhaustive and Practical System of Method, Application, and Use (Occult/Science, Non-Fiction)
A Popular History of France from the Earliest Times, Volume 4 (History, Non-Fiction)
The Luck of Roaring Camp and Other Tales: With Condensed Novels, Spanish and American Legends, and Earlier Papers (Fiction, Short Stories)
Trial by water (Fiction, Novel)
Line and Form (1900) (Art/Instruction, Non-Fiction)
The Industrial Arts in Spain (Art/History, Non-Fiction)
Howards End (Fiction, Social Novel)
Bahaism and Its Claims: A Study of the Religion Promulgated by Baha Ullah and Abdul Baha (Religion/History, Non-Fiction)
The censorship of the Church of Rome and its influence upon the production and distribution of literature, volume 2 (of 2) (History/Religion, Non-Fiction)
"Everyman," with other interludes, including eight miracle plays (Drama/Plays Collection, Medieval)
Hazlitt on English Literature: An Introduction to the Appreciation of Literature (Literary Criticism, Non-Fiction)
Ancient society : $b or, Researches in the lines of human progress from savagery, through barbarism to civilization (Anthropology/Sociology, Non-Fiction)
Daisy Miller: A Study (Fiction, Novella)
Confessions of an English Opium-Eater (Autobiography, Non-Fiction)
The Rainbow (Fiction, Novel)
The Expositor's Bible: The First Book of Kings (Religion/Commentary, Non-Fiction)
Adventures and Reminiscences of a Volunteer; Or, A Drummer Boy from Maine (Autobiography/War, Non-Fiction)
Great Astronomers (Biography/Science, Non-Fiction)
The History of "Punch" (History/Literary, Non-Fiction)
The Mysteries of Udolpho (Fiction, Gothic Novel)
The Suppression of the African Slave Trade to the United States of America: 1638-1870 (History, Non-Fiction)
Hawaiian folk tales : $b a collection of native legends (Folklore/Fiction, Short Stories)
The Chinese Fairy Book (Folklore/Fiction, Short Stories Collection)
Aeschylus' Prometheus Bound and the Seven Against Thebes (Drama/Play, Tragedy)
Memoirs of the Dukes of Urbino, Volume 1 (of 3): Illustrating the Arms, Arts, and Literature of Italy, from 1440 To 1630. (History/Biography, Non-Fiction)
An Advanced English Grammar with Exercises (Education/Language, Non-Fiction)
The Story of Siena and San Gimignano (History/Travel, Non-Fiction)
Biographical Anecdotes of William Hogarth, With a Catalogue of His Works (Biography/Art History, Non-Fiction)
Some Principles of Maritime Strategy (Military/Strategy, Non-Fiction)
A History of French Literature: Short Histories of the Literatures of the World: II. (Literary Criticism/History, Non-Fiction)
The Clansman: An Historical Romance of the Ku Klux Klan (Historical Fiction, Novel) [Note: Controversial racist narrative]
The Ingoldsby Legends; or, Mirth and Marvels (Poetry/Humor Collection)
Dramatic Technique (Literary Criticism/Drama, Non-Fiction)
Kwaidan: Stories and Studies of Strange Things (Folklore/Horror, Short Stories)
Italian Highways and Byways from a Motor Car (Travel, Non-Fiction)
The nigger of the "Narcissus" : $b A tale of the forecastle (Fiction, Novella)
The Strange Adventures of Andrew Battell, of Leigh, in Angola and the Adjoining Regions (Travel/Adventure, Non-Fiction)
How to Live on 24 Hours a Day (Self-Help, Non-Fiction)
The Magic and Science of Jewels and Stones (History/Occult, Non-Fiction)
Up the Orinoco and down the Magdalena (Travel/Adventure, Non-Fiction)
The sound and the fury (Fiction, Modernist Novel)
The eleventh hour in the life of Julia Ward Howe (Biography, Non-Fiction)
The Inimitable Jeeves (Fiction, Humorous Novel)
The Cavalier Songs and Ballads of England from 1642 to 1684 (Poetry/Folklore Collection)
The Fables of Aesop: Selected, Told Anew, and Their History Traced (Children's Fiction, Fables)
Haviland's Chum (Fiction, Novel)
The World's Greatest Books — Volume 02 — Fiction (Literary Anthology/Fiction)
In the hollow of His hand (Fiction, Novel)
The Man of the Forest (Fiction, Western Novel)
Play-Making: A Manual of Craftsmanship (Literary Criticism/Drama, Non-Fiction)
The Ballads and Songs of Yorkshire: Transcribed from Private Manuscripts, Rare Broadsides, and Scarce Publications; with Notes and a Glossary (Poetry/Folklore Collection)
Mrs. Warren's Profession (Drama/Play)
The Practice of the Presence of God (Religion/Spirituality, Non-Fiction)
Old Christmas: from the Sketch Book of Washington Irving (Fiction, Sketches/Holiday)
Waverley; or, 'Tis sixty years since (Historical Fiction, Novel)
Notes on Nursing: What It Is, and What It Is Not (Medicine/Health, Non-Fiction)
The Mediaeval Mind (Volume 2 of 2): A History of the Development of Thought and Emotion in the Middle Ages (History/Philosophy, Non-Fiction)
Armenia, Travels and Studies (Volume 1 of 2): The Russian Provinces (Travel/History, Non-Fiction)
The Forsyte Saga - Complete (Fiction, Novel Series)
Renshaw Fanning's Quest: A Tale of the High Veldt (Adventure Fiction, Novel)
A Century of Parody and Imitation (Literary Criticism/Humor, Non-Fiction)
Life in the Iron-Mills; Or, The Korl Woman (Fiction, Short Story)
The Classic Myths in English Literature and in Art (2nd ed.) (1911): Based Originally on Bulfinch's "Age of Fable" (1855) (Mythology/Literary Criticism, Non-Fiction)
The English and Scottish popular ballads, volume 1 (of 5) (Poetry/Folklore Collection)
Lancashire Folk-lore: Illustrative of the Superstitious Beliefs and Practices, Local Customs and Usages of the People of the County Palatine (Folklore, Non-Fiction)
The Sayings of Confucius: A New Translation of the Greater Part of the Confucian Analects (Philosophy/Religion, Non-Fiction)
Ten Thousand Wonderful Things: Comprising whatever is marvellous and rare, curious, eccentric and extraordinary in all ages and nations (Reference/Curiosities, Non-Fiction)
Giovanni Boccaccio, a Biographical Study (Biography/Literary Criticism, Non-Fiction)
Ethnological results of the Point Barrow expedition: Ninth Annual Report of the Bureau of Ethnology to the Secretary of the Smithsonian Institution, 1887-1888, Government Printing Office, Washington, 1892, pages 3-442 (Anthropology, Non-Fiction)
With the Night Mail: A Story of 2000 A.D.: (Together with extracts from the comtemporary magazine in which it appeared) (Science Fiction, Short Story)
The Book Review Digest, v. 16, 1920 : $b Sixteenth annual accumulation. Reviews of 1920 books (Literary Criticism/Reference, Non-Fiction)
Aesthetical Essays of Friedrich Schiller (Philosophy/Art Criticism, Non-Fiction)
Good Friday, and other poems (Poetry Collection)
What Every Girl Should Know (Self-Help/Health, Non-Fiction)
The La Chance Mine Mystery (Mystery Fiction, Novel)
A Treatise on Wood Engraving, Historical and Practical (Art/History, Non-Fiction)
The Velveteen Rabbit (Children's Fiction, Fantasy Tale)
Curious Creatures in Zoology (Science/Natural History, Non-Fiction)
History of the Colony and Ancient Dominion of Virginia (History, Non-Fiction)
Father Goriot (Fiction, Novel)
Babylonian and Assyrian Laws, Contracts and Letters (History/Law, Non-Fiction)
The Brighton Road: The Classic Highway to the South (Travel/History, Non-Fiction)
The History of Antiquity, Vol. 1 (of 6) (History, Non-Fiction)
Character Sketches of Romance, Fiction, and the Drama, Vol. 4: A Revised American Edition of the Reader's Handbook (Reference/Literary Criticism, Non-Fiction)
The inventions, researches and writings of Nikola Tesla: With special reference to his work in polyphase currents and high potential lighting (Science/Biography, Non-Fiction)
A Complete History of Music: for Schools, Clubs, and Private Reading (Music/History, Non-Fiction)
Soap-Making Manual: A Practical Handbook on the Raw Materials, Their Manipulation, Analysis and Control in the Modern Soap Plant. (Science/Instruction, Non-Fiction)
Venice and Its Story (History/Travel, Non-Fiction)
Lady Jane Grey and Her Times (Biography/History, Non-Fiction)
Self-instructor in the art of hair work, dressing hair, making curls, switches, braids, and hair jewelry of every description. (Instruction/Beauty, Non-Fiction)
Life of Mozart, Vol. 3 (of 3) (Biography, Non-Fiction) [Note: Variant of previous list entry]
History of the Jews in Russia and Poland, Volume 3 [of 3]: From the Accession of Nicholas II until the Present Day (History, Non-Fiction)
Recollections of a Tour Made in Scotland A.D. 1803 (Travel/Autobiography, Non-Fiction)
Correspondence and Report from His Majesty's Consul at Boma Respecting the Administration of the Independent State of the Congo [and Further Correspondence] (History/Politics, Non-Fiction)
Town Life in the Fifteenth Century, Volume 2 (of 2) (History/Social, Non-Fiction)
The Project Gutenberg Encyclopedia, Volume 1 of 28 (Reference/Encyclopedia, Non-Fiction)
Prisoners of War in Britain 1756 to 1815: A record of their lives, their romance and their sufferings (History/Military, Non-Fiction)
My past and thoughts, vol 1 (of 6) : $b The memoirs of Alexander Herzen (Autobiography/History, Non-Fiction)
Creative Evolution (Philosophy/Science, Non-Fiction)
A Gothic Grammar, with selections for reading and a glossary (Education/Language, Non-Fiction)
The Voyages of Pedro Fernandez de Quiros, 1595 to 1606. Volume 1 (History/Exploration, Non-Fiction)
The First Blast of the Trumpet against the monstrous regiment of Women (Politics/Religion, Non-Fiction)
Comrades: A Story of Social Adventure in California (Fiction, Social Novel)
Beethoven's Letters 1790-1826, Volume 1 (Biography/Letters, Non-Fiction)
On Love (Philosophy/Essays, Non-Fiction)
The Pencil of Nature (Photography/Art, Non-Fiction)
The eternal moment, and other stories (Fiction, Short Stories)
The Mysteries of All Nations: Rise and Progress of Superstition, Laws Against and Trials of Witches, Ancient and Modern Delusions Together with Strange Customs, Fables, and Tales (Folklore/History, Non-Fiction)
Records of the Spanish Inquisition, Translated from the Original Manuscripts (History/Religion, Non-Fiction)
Parzival: A Knightly Epic (vol. 1 of 2) (Poetry, Epic Romance)
L. Annaeus Seneca on Benefits (Philosophy/Ethics, Non-Fiction)
Historia Amoris: A History of Love, Ancient and Modern (History/Cultural, Non-Fiction)
The Natural History of Pliny, Volume 6 (of 6) (Science/Natural History, Non-Fiction) [Note: Variant of previous list entry]
Human Animals (Folklore/Occult, Non-Fiction)
The best British short stories of 1922 (Fiction, Short Stories Collection)
United States Presidents' Inaugural Speeches: From Washington to George W. Bush (History/Politics, Non-Fiction Collection)
The Phase Rule and Its Applications (Science/Chemistry, Non-Fiction)
One Woman's Life (Fiction, Novel)
In Morocco (Travel, Non-Fiction)
Autobiography of Andrew Carnegie (Autobiography, Non-Fiction)
The best short stories of 1922, and the yearbook of the American short story (Fiction, Short Stories Collection)
Commentary on Genesis, Vol. 1: Luther on the Creation (Religion/Commentary, Non-Fiction)
Uncle Remus, his songs and his sayings (Folklore/Fiction, Short Stories)
Jewellery (Art/History, Non-Fiction)
Don Juan Tenorio (Drama/Play)
Fools and mules : $b [A Shorty McKay story] (Fiction, Short Story)
Little Eyebright and her pund o' care (Fiction, Novel)
The Memoirs of François René Vicomte de Chateaubriand sometime Ambassador to England. volume 5 (of 6): Mémoires d'outre-tombe volume 5 (Autobiography/History, Non-Fiction)
The World's Sixteen Crucified Saviors; Or, Christianity Before Christ (Religion/Mythology, Non-Fiction)
The Posthumous Works of Thomas De Quincey, Vol. 1 (Essays/Literary, Non-Fiction)
The Analects of Confucius (from the Chinese Classics) (Philosophy/Religion, Non-Fiction)
Custom and Myth: New Edition (Mythology/Anthropology, Non-Fiction)
Character Sketches of Romance, Fiction, and the Drama, Vol. 2: A Revised American Edition of the Reader's Handbook (Reference/Literary Criticism, Non-Fiction)
Machiavelli, Volume I (Politics/Philosophy, Non-Fiction)
Yachting, Vol. 2 (Sports/History, Non-Fiction)
My Ántonia (Fiction, Novel)
Karl Grier : $b The strange story of a man with a sixth sense (Fiction, Novel)
The Story of the Great Fire in St. John, N.B., June 20th, 1877 (History, Non-Fiction)
The United States Constitution (History/Politics, Non-Fiction)
My Reminiscences (Autobiography, Non-Fiction)
A Catalogue of Books in English Later than 1700, Vol. 1: Forming a portion of the library of Robert Hoe (Reference/Bibliography, Non-Fiction)
Selected Poems (Poetry Collection)
Childhood's Favorites and Fairy Stories: The Young Folks Treasury, Volume 1 (Children's Fiction, Anthology)
The Little Mother Goose (Children's Poetry, Collection)
The Ethics of Medical Homicide and Mutilation (Medicine/Ethics, Non-Fiction)
The Complete Plays of Gilbert and Sullivan (Drama/Plays Collection)
Four Arthurian Romances (Fiction, Arthurian Legend)
The Clicking of Cuthbert (Fiction, Humorous Short Stories)
Tobacco Leaves: Being a Book of Facts for Smokers (History/Health, Non-Fiction)
Fort Amity (Historical Fiction, Novel)
A General View of Positivism: Or, Summary exposition of the System of Thought and Life (Philosophy, Non-Fiction)
Memoirs (Autobiography, Non-Fiction)
Language: Its Nature, Development and Origin (Linguistics, Non-Fiction)
History of Florence and of the Affairs of Italy: From the Earliest Times to the Death of Lorenzo the Magnificent (History, Non-Fiction)
The Great American Fraud: The Patent Medicine Evil (History/Medicine, Non-Fiction)
A history of the Peninsular War, Vol. 5, Oct. 1811-Aug. 31, 1812 : $b Valencia, Ciudad Rodrigo, Badajoz, Salamanca, Madrid (History/Military, Non-Fiction)
Architecture : $b nineteenth and twentieth centuries (Architecture/History, Non-Fiction)
The movie boys in the jungle : $b or, Lively times among the wild beasts (Adventure Fiction, Youth Novel)
The Task of Social Hygiene (Sociology/Health, Non-Fiction)
The Diary of John Evelyn (Volume 2 of 2) (Autobiography/Diary, Non-Fiction)
The history of fashion in France : $b or, the dress of women from the Gallo-Roman period to the present time (History/Fashion, Non-Fiction)
Library of the World's Best Mystery and Detective Stories (Mystery Fiction, Anthology)
The best short stories of 1917, and the yearbook of the American short story (Fiction, Short Stories Collection)
The Journal of a Disappointed Man (Autobiography, Non-Fiction)
A collection of short-stories (Fiction, Short Stories Collection)
Unknown to History: A Story of the Captivity of Mary of Scotland (Historical Fiction, Novel)
The history of steam navigation (History/Technology, Non-Fiction)
Telling Fortunes By Tea Leaves: How to Read Your Fate in a Teacup (Occult/Guide, Non-Fiction)
A Child's History of the World (History, Non-Fiction for Children)
The Red Seal (Mystery Fiction, Novel)
The hero of the Filipinos : $b The story of José Rizal, poet, patriot and martyr (Biography/History, Non-Fiction)
Fictitious & Symbolic Creatures in Art: With Special Reference to Their Use in British Heraldry (Art/History, Non-Fiction)
The Amores; or, Amours: Literally Translated into English Prose, with Copious Notes (Poetry/Erotica, Translation)
The Satyricon of Petronius Arbiter (Fiction, Satirical Novel) [Note: Variant of previous list entry]
Life and Letters of Thomas Cromwell, Vol. 1 of 2: Life, Letters to 1535 (Biography/Letters, Non-Fiction)
The Kipling Reader: Selections from the Books of Rudyard Kipling (Literary Anthology)
Selections from the Table Talk of Martin Luther (Religion/Philosophy, Non-Fiction)
Highways and Byways in Surrey (Travel/History, Non-Fiction)
The History of the Thirteen Colonies of North America 1497-1763 (History, Non-Fiction)
The Georgics (Poetry, Didactic Verse)
Colour in the flower garden (Gardening/Guide, Non-Fiction)
The Principles of Masonic Law: A Treatise on the Constitutional Laws, Usages and Landmarks of Freemasonry (Occult/History, Non-Fiction)
Farnham's Travels in the Great Western Prairies, etc., part 1, May 21-October 16, 1839 (Travel/History, Non-Fiction)
Henley's Twentieth Century Formulas, Recipes and Processes (Science/Reference, Non-Fiction)
History of the Philippine Islands (History, Non-Fiction)
The Life and Letters of Mary Wollstonecraft Shelley, Volume 1 (of 2) (Biography/Letters, Non-Fiction)
Armenia and the Armenians: A List of References in the New York Public Library (Reference/Bibliography, Non-Fiction)
The Officer's Manual: Napoleon's Maxims of War (Military/Strategy, Non-Fiction)
My own story (Autobiography, Non-Fiction)
Give Me Liberty or Give Me Death (History/Politics, Speech/Non-Fiction)
The Five Great Philosophies of Life (Philosophy, Non-Fiction)
Tacitus on Germany (History, Non-Fiction)
The Letters of Henry James (Vol. II) (Biography/Letters, Non-Fiction)
The Burgess Animal Book for Children (Children's Fiction, Educational/Animal Tales)
Damn! A Book of Calumny (Essays/Satire, Non-Fiction)
Charles Sumner: his complete works, volume 20 (of 20) (Politics/Essays, Non-Fiction)
Tales of King Arthur and the Round Table, Adapted from the Book of Romance (Children's Fiction, Arthurian Legend)
The Secrets of the Self (Asrar-i Khudi) — A Philosophical Poem (Poetry/Philosophy)
Heidi: (Gift Edition) (Children's Fiction, Novel) [Note: Variant of previous list entry]
My Life as an Author (Autobiography/Literary, Non-Fiction)
Criminal Man, According to the Classification of Cesare Lombroso (Criminology/Psychology, Non-Fiction)
The Stars and Stripes, Vol. 1, No. 1, February 8, 1918: The American Soldiers' Newspaper of World War I, 1918-1919 (History/Journalism, Non-Fiction)
Fine Books (History/Bibliography, Non-Fiction)
Martine's Hand-book of Etiquette, and Guide to True Politeness (Self-Help/Etiquette, Non-Fiction)
Winesburg, Ohio: A Group of Tales of Ohio Small Town Life (Fiction, Short Stories)
The inquisition in the Spanish dependencies: Sicily—Naples—Sardinia—Milan—the Canaries—Mexico—Peru—New Granada (History/Religion, Non-Fiction)
Wives of the Prime Ministers, 1844-1906 (Biography/History, Non-Fiction)
Astrology: How to Make and Read Your Own Horoscope (Occult/Guide, Non-Fiction)
A summer journey to Brazil (Travel, Non-Fiction)
The History of the Rise, Progress and Accomplishment of the Abolition of the African Slave-Trade, by the British Parliament (1839) (History, Non-Fiction)
Thomas Jefferson, the Apostle of Americanism (Biography/History, Non-Fiction)
The House on the Borderland (Horror Fiction, Novel)
Theodicy: Essays on the Goodness of God, the Freedom of Man and the Origin of Evil (Philosophy/Religion, Non-Fiction)
The Swiss Family Robinson; or Adventures in a Desert Island (Adventure Fiction, Novel) [Note: Variant of previous entry]
Fungi: Their Nature and Uses (Science/Botany, Non-Fiction)
Studies in the Psychology of Sex, Volume 2: Sexual Inversion (Psychology/Sexology, Non-Fiction)
Memoirs of the Life and Correspondence of Henry Reeve, C.B., D.C.L.: In Two Volumes. Volume II. (Biography/Letters, Non-Fiction)
Gregg's Commerce of the Prairies, 1831-1839, part 2 (Travel/History, Non-Fiction)
Gallio : $b or, The tyranny of science (Fiction, Novel)
Mark Twain: A Biography. Complete (Biography, Non-Fiction)
The Arts in the Middle Ages and at the Period of the Renaissance (Art/History, Non-Fiction)
My life in Sarawak (Autobiography/Travel, Non-Fiction)
The gradual acceptance of the Copernican theory of the universe (Science/History, Non-Fiction)
The Odes and Carmen Saeculare of Horace (Poetry Collection, Translation)
The Washington Historical Quarterly, Volume V, 1914 (History, Non-Fiction Journal)
Complete Short Works of George Meredith (Fiction, Short Stories Collection)
Life of Mozart, Vol. 1 (of 3) (Biography, Non-Fiction) [Note: Variant of previous list entry]
The Agrarian Problem in the Sixteenth Century (History/Economics, Non-Fiction)
The Autobiography of Goethe: Truth and Poetry: From My Own Life (Autobiography, Non-Fiction)
The Archaeology of the Yakima Valley (Archaeology/History, Non-Fiction)
Castles and Chateaux of Old Navarre and the Basque Provinces (History/Travel, Non-Fiction)
Amphitryo, Asinaria, Aulularia, Bacchides, Captivi: Amphitryon, The Comedy of Asses, The Pot of Gold, The Two; Bacchises, The Captives (Drama/Plays Collection)
The history of Sumatra : $b Containing an account of the government, laws, customs and manners of the native inhabitants (Travel/History, Non-Fiction)
Greener Than You Think (Science Fiction, Novel)
Endymion: A Poetic Romance (Poetry, Narrative Verse)
The Hand in the Dark (Mystery Fiction, Novel)
Engraving: Its Origin, Processes, and History (Art/History, Non-Fiction)
The Christ Myth (Religion/History, Non-Fiction)
An Apology for the Life of Mr. Colley Cibber, Volume 1 (of 2): Written by Himself. A New Edition with Notes and Supplement (Autobiography/Theater, Non-Fiction)
Portugal of the Portuguese (History/Cultural, Non-Fiction)
The Boy Mechanic, Book 2: 1000 Things for Boys to Do (Education/Crafts, Non-Fiction)
Superstition in all ages (Philosophy/Religion Criticism, Non-Fiction)
The Present Condition of Organic Nature: Lecture I. (of VI.), "Lectures to Working Men", at the Museum of Practical Geology, 1863, on Darwin's Work: "Origin of Species" (Science/Biology, Non-Fiction)
Knickerbocker's History of New York, Complete (History/Satire, Non-Fiction)
Hymns for Christian Devotion: Especially Adapted to the Universalist Denomination (Religion/Music, Non-Fiction)
The Alhambra (Travel/History, Non-Fiction)
Nonsense Books (Children's Poetry/Humor Collection)
French Book-plates (Art/History, Non-Fiction)
Japan: From the Japanese Government History (History/Cultural, Non-Fiction)
Frederick William Maitland, Downing Professor of the Laws of England: A Biographical Sketch (Biography/Law, Non-Fiction)
The Green Mummy (Mystery Fiction, Novel)
Contemporary American Composers: Being a Study of the Music of This Country, Its Present; Conditions and Its Future, with Critical Estimates and; Biographies of the Principal Living Composers; and an; Abundance of Portraits, Fac-simile Musical Autographs, and; Compositions (Music/Biography, Non-Fiction)
On the magnet, magnetick bodies also, and on the great magnet the earth: a new physiology, demonstrated by many arguments & experiments (Science/Physics, Non-Fiction)
The Man Who Did the Right Thing: A Romance (Fiction, Romance Novel)
The game of life and how to play it (Self-Help/Spirituality, Non-Fiction)
Jane Austen, Her Life and Letters: A Family Record (Biography/Literary, Non-Fiction)
The Czar's Spy: The Mystery of a Silent Love (Mystery Fiction, Novel)
The Satyricon — Volume 02: Dinner of Trimalchio (Fiction, Satirical Novel)
Sweden (History/Travel, Non-Fiction)
The Philippines: Past and Present (Volume 1 of 2) (History, Non-Fiction) [Note: Variant of previous list entry]
The American Diary of a Japanese Girl (Fiction, Diary Novel)
A Diplomat in Japan: The inner history of the critical years in the evolution of Japan when the ports were opened and the monarchy restored, recorded by a diplomatist who took an active part in the events of the time, with an account of his personal experiences during that period (History/Autobiography, Non-Fiction)
The Diamond Sutra (Chin-Kang-Ching) or Prajna-Paramita (Religion/Philosophy, Non-Fiction)
Morals and Dogma of the Ancient and Accepted Scottish Rite of Freemasonry (Occult/Philosophy, Non-Fiction)
Joseph Andrews, Vol. 1 (Fiction, Picaresque Novel)
The Works of Alexander Pope, Volume 1: Poetry - Volume 1 (Poetry Collection) [Note: Variant of previous list entry]
The Violoncello and Its History (Music/History, Non-Fiction)
The Wit and Humor of America, Volume IX (of X) (Humor/Fiction Collection) [Note: Variant of previous list entry]
The Revolutionary Movement of 1848-9 in Italy, Austria-Hungary, and Germany: With Some Examination of the Previous Thirty-three Years (History, Non-Fiction)
The Articles of Faith: A Series of Lectures on the Principal Doctrines of the Church of Jesus Christ of Latter-Day Saints (Religion/Theology, Non-Fiction)
The Landloper: The Romance of a Man on Foot (Fiction, Romance Novel)
Only a girl's love (Fiction, Romance Novel)
The Awakening of Spring: A Tragedy of Childhood (Drama/Play, Tragedy)
The Queen of Spades, and other stories (Fiction, Short Stories)
The Martian: A Novel (Science Fiction, Novel)
Main Currents in Nineteenth Century Literature - 6. Young Germany (Literary Criticism/History, Non-Fiction)
The Tragedies of Euripides, Volume I. (Drama/Plays Collection, Tragedy)
The Conjure Woman (Fiction, Short Stories/Folklore)
Copyright: Its History and Its Law (Law/History, Non-Fiction)
Ole Bull: A Memoir (Biography/Music, Non-Fiction)
The Spanish Tragedie (Drama/Play, Tragedy)
The Mysteries of Free Masonry: Containing All the Degrees of the Order Conferred in a Master's Lodge (Occult/Reference, Non-Fiction)
Micromegas (Fiction, Satirical Short Story)
Romain Rolland: The Man and His Work (Biography/Literary Criticism, Non-Fiction)
Christopher Columbus and His Monument Columbia: being a concordance of choice tributes to the great Genoese, his grand discovery, and his greatness of mind and purpose (History/Biography, Non-Fiction)
The Autobiography of Benjamin Franklin (Autobiography, Non-Fiction) [Note: Duplicate from previous list]
Studies in the Scriptures, Volume 7: The Finished Mystery (Religion/Commentary, Non-Fiction)
The Indolence of the Filipino (History/Politics, Non-Fiction Essay)
The Children of Odin: The Book of Northern Myths (Children's Fiction, Mythology Retellings)
Cleopatra's Needle: A History of the London Obelisk, with an Exposition of the Hieroglyphics (History/Archaeology, Non-Fiction)
A Literary and Historical Atlas of Asia (Reference/History, Non-Fiction)
Pamela, or Virtue Rewarded (Fiction, Epistolary Novel)
Historical records of the Buffs, East Kent Regiment (3rd Foot) : $b Formerly designated the Holland Regiment and Prince George of Denmark's Regiment (History/Military, Non-Fiction)
Records of a Girlhood (Autobiography, Non-Fiction)
The History of Woman Suffrage, Volume VI (History/Feminism, Non-Fiction)
A History of Rome to 565 A. D. (History, Non-Fiction)
Villani's Chronicle: Being Selections from the First Nine Books of the Croniche Fiorentine of Giovanni Villani (History, Non-Fiction)
The Story of the Sun: New York, 1833-1918 (History/Journalism, Non-Fiction)
Astronomy for Amateurs (Science/Astronomy, Non-Fiction)
The American Indian as Slaveholder and Seccessionist: An Omitted Chapter in the Diplomatic History of the Southern Confederacy (History, Non-Fiction)
Minute Mysteries [Detectograms] (Mystery Fiction, Puzzles/Short Stories)
The Home of the Blizzard: Being the Story of the Australasian Antarctic Expedition, 1911-1914 (Travel/Exploration, Non-Fiction)
Rasselas, Prince of Abyssinia (Fiction, Philosophical Novel)
English Verse: Specimens Illustrating its Principles and History (Poetry/Literary Criticism, Non-Fiction)
The Old English Herbals (History/Botany, Non-Fiction)
Traditions, Superstitions and Folk-lore: (Chiefly Lancashire and the North of England:) Their Affinity to Others in Widely-Distributed Localities; Their Eastern Origin and Mythical Significance. (Folklore, Non-Fiction)
Curious Epitaphs, Collected from the Graveyards of Great Britain and Ireland. (History/Folklore, Non-Fiction)
Letters of Two Brides (Fiction, Epistolary Novel)
French Idioms and Proverbs: A Companion to Deshumbert's "Dictionary of Difficulties" (Reference/Language, Non-Fiction)
Introduction to the History of Religions: Handbooks on the History of Religions, Volume IV (Religion/Anthropology, Non-Fiction)
Chippinge Borough (Fiction, Novel)
Nooks and Corners of Lancashire and Cheshire.: A Wayfarer's Notes in the Palatine Counties, Historical, Legendary, Genealogical, and Descriptive. (Travel/History, Non-Fiction)
The Secret of the Ninth Planet (Science Fiction, Novel)
News from Nowhere; Or, An Epoch of Rest: Being Some Chapters from a Utopian Romance (Fiction, Utopian Novel)
Paul Clifford — Complete (Fiction, Novel)
A Dictionary of the First or Oldest Words in the English Language: From the Semi-Saxon Period of A.D. 1250 to 1300 (Reference/Linguistics, Non-Fiction)
The Complete Works of Robert Burns: Containing his Poems, Songs, and Correspondence.: With a New Life of the Poet, and Notices, Critical and Biographical by Allan Cunningham (Poetry Collection)
Caravans to Santa Fe (History/Travel, Non-Fiction)
Gods and Fighting Men: The story of the Tuatha de Danaan and of the Fianna of Ireland, arranged and put into English by Lady Gregory (Folklore/Mythology, Non-Fiction)
The Cathedral Builders: The Story of a Great Masonic Guild (History/Architecture, Non-Fiction)
Canyons of the Colorado (Travel/Exploration, Non-Fiction)
East of Suez: Ceylon, India, China and Japan (Travel, Non-Fiction)
Lives of the Most Eminent Painters, Sculptors and Architects, Vol. 09 (of 10): Michelagnolo to the Flemings (Biography/Art History, Non-Fiction)
Selected Works of Voltairine de Cleyre (Essays/Politics, Non-Fiction)
Langley Memoir on Mechanical Flight, Parts I and II: Smithsonian Contributions to Knowledge, Volume 27 Number 3, Publication 1948, 1911 (Science/Aviation, Non-Fiction)
Translations of Shakuntala and Other Works (Drama/Literature, Translation)
The Love Letters of Mary Wollstonecraft to Gilbert Imlay (Biography/Letters, Non-Fiction)
Celtic Folk and Fairy Tales (Folklore/Fiction, Short Stories Collection)
That Which Hath Wings: A Novel of the Day (Fiction, Novel)
The United States Bill of Rights: The Ten Original Amendments to the Constitution of the United States (History/Politics, Non-Fiction)
Views of nature: or Contemplations on the sublime phenomena of creation: with scientific illustrations (Science/Nature, Non-Fiction)
Inspiration and Interpretation: Seven Sermons Preached Before the University of Oxford (Religion/Sermons, Non-Fiction)
Young People's Pride: A Novel (Fiction, Novel)
A Life of William Shakespeare: with portraits and facsimiles (Biography/Literary Criticism, Non-Fiction)
Curiosities of Great Britain: England and Wales Delineated Vol.1-11: Historical, Entertaining & Commercial; Alphabetically Arranged. 11 Volume set. (History/Reference, Non-Fiction)
Tales of Unrest (Fiction, Short Stories)
The Book of Dragons (Children's Fiction, Fantasy Short Stories)
The ugly-girl papers : $b or, Hints for the toilet (Self-Help/Beauty, Non-Fiction)
Jewish influences in American life : $b volume III of the International Jew, the world's foremost problem : being a reprint of a third selection from articles appearing in the Dearborn Independent (Politics/Antisemitic, Non-Fiction) [Note: Controversial and biased text]
The Journal of Negro History, Volume 7, 1922 (History, Non-Fiction Journal)
Nursing as Caring: A Model for Transforming Practice (Medicine/Nursing, Non-Fiction)
Palace and Mosque at Ukhaidir: A Study in Early Mohammadan Architecture (Architecture/History, Non-Fiction)
Legends, Tales and Poems (Poetry/Folklore Collection)
The Germ: Thoughts towards Nature in Poetry, Literature and Art (Literary Criticism/Poetry, Non-Fiction)
A Journey from Prince of Wales's Fort in Hudson's Bay to the Northern Ocean in the Years 1769, 1770, 1771, 1772: New Edition with Introduction, Notes, and Illustrations (Travel/Exploration, Non-Fiction)
The Dunwich horror (Horror Fiction, Short Story)
Indian Fairy Tales (Folklore/Fiction, Short Stories Collection)
The Epic of Gilgamish: A Fragment of the Gilgamish Legend in Old-Babylonian Cuneiform (Mythology/Poetry, Translation)
Primitive culture, vol. 1 (of 2) : $b Researches into the development of mythology, philosophy, religion, language, art, and custom (Anthropology, Non-Fiction)
The Legends and Myths of Hawaii: The fables and folk-lore of a strange people (Folklore/Mythology, Non-Fiction)
Poems by William Cullen Bryant (Poetry Collection)
The Story of the American Merchant Marine (History/Business, Non-Fiction)
Lands of the Slave and the Free; Or, Cuba, the United States, and Canada (Travel/History, Non-Fiction)
The new freedom : $b a call for the emancipation of the generous energies of a people (Politics, Non-Fiction)
Varney the Vampire; Or, the Feast of Blood (Horror Fiction, Novel)
The Athenian Constitution (History/Politics, Non-Fiction)
Villainage in England: Essays in English Mediaeval History (History, Non-Fiction)
The Complete Opera Book: The Stories of the Operas, together with 400 of the Leading Airs and Motives in Musical Notation (Music/Reference, Non-Fiction)
Simon the Jester (Fiction, Novel)
Annals of Music in America: A Chronological Record of Significant Musical Events (Music/History, Non-Fiction)
Chronological Retrospect of the History of Yarmouth and Neighbourhood: from A.D. 46 to 1884 (History, Non-Fiction)
Linnet: A Romance (Fiction, Romance Novel)
Discoveries Among the Ruins of Nineveh and Babylon (Archaeology/Travel, Non-Fiction)
My Experiences in Manipur and the Naga Hills (Autobiography/Travel, Non-Fiction)
The Travels of Sir John Mandeville (Travel/Fiction, Medieval)
The Renaissance: Studies in Art and Poetry (Art/Literary Criticism, Non-Fiction)
The Traditional Games of England, Scotland, and Ireland (Vol 2 of 2): With Tunes, Singing-Rhymes, and Methods of Playing etc. (Folklore/Games, Non-Fiction)
Shelters, Shacks and Shanties (Instruction/Outdoor, Non-Fiction)
The City of Auckland, New Zealand, 1840-1920 (History, Non-Fiction)
Fredericksburg and Its Many Points of Interest (History/Travel, Non-Fiction)
A Week's Tramp in Dickens-Land: Together with Personal Reminiscences of the 'Inimitable Boz' Therein Collected (Travel/Literary, Non-Fiction)
Strange Stories from a Chinese Studio, Vol. 1 (of 2) (Folklore/Fiction, Short Stories) [Note: Variant of previous list entry]
On the Spanish Main; Or, Some English forays on the Isthmus of Darien (History/Adventure, Non-Fiction)
Histories of two hundred and fifty-one divisions of the German army which participated in the war (1914-1918) (History/Military, Non-Fiction)
To Have and to Hold (Historical Fiction, Novel)
The Book of Buried Treasure: Being a True History of the Gold, Jewels, and Plate of Pirates, Galleons, etc., which are sought for to this day (History/Adventure, Non-Fiction)
A Guide to Health (Health/Self-Help, Non-Fiction)
Chess Strategy (Games/Strategy, Non-Fiction)
The Odes of Casimire, Translated by G. Hils (Poetry Collection, Translation)
Too old to fly (Fiction, Novel)
The Path to Rome (Travel/Philosophy, Non-Fiction)
The Damned Thing: 1898, From "In the Midst of Life" (Horror Fiction, Short Story)
Know the Truth: A Critique on the Hamiltonian Theory of Limitation: Including Some Strictures Upon the Theories of Rev. Henry L. Mansel and Mr. Herbert Spencer (Philosophy, Non-Fiction)
A Martian Odyssey (Science Fiction, Short Story)
Ancient Armour and Weapons in Europe: From the Iron Period of the Northern Nations to the End of the Thirteenth Century (History/Military, Non-Fiction)
The Dean of Lismore's Book: A Selection of Ancient Gaelic Poetry (Poetry Collection)
Mr. Honey's Banking Dictionary (German-English) (Reference/Dictionary, Non-Fiction)
Worcestershire in the Nineteenth Century: A Complete Digest of Facts Occuring in the County since the Commencement of the year 1800 (History, Non-Fiction)
History of the Forty-Eighth Regiment M. V. M. During the Civil War (History/Military, Non-Fiction)
Napoleon's Marshals (Biography/Military, Non-Fiction)
Lancelot of the Laik: A Scottish Metrical Romance (About 1490-1500 A. D.) (Poetry, Romance)
Stories of Old Greece and Rome (Children's Fiction, Mythology Retellings)
The Mechanical Properties of Wood: Including a Discussion of the Factors Affecting the Mechanical Properties, and Methods of Timber Testing (Science/Engineering, Non-Fiction)
The Reason Why: A Careful Collection of Many Hundreds of Reasons for Things Which, Though Generally Believed, Are Imperfectly Understood (Reference/Curiosities, Non-Fiction)
A Middle High German Primer: Third Edition (Education/Language, Non-Fiction)
The Romance of Words (4th ed.) (Linguistics/History, Non-Fiction)
How to Enjoy Paris in 1842: Intended to Serve as a Companion and Monitor, Containing; Historical, Political, Commercial, Artistical, Theatrical; And Statistical Information (Travel/Guide, Non-Fiction)
Nationalism (Politics/Philosophy, Non-Fiction)
The Literature of the Ancient Egyptians (Literary Criticism/History, Non-Fiction)
The Man with Two Left Feet, and Other Stories (Fiction, Short Stories)
Astounding Stories, July, 1931 (Science Fiction, Magazine/Short Stories)
Sagittulae, Random Verses (Poetry Collection)
Beowulf: An Introduction to the Study of the Poem with a Discussion of the Stories of Offa and Finn (Literary Criticism/Poetry, Non-Fiction)
Ireland under the Stuarts and During the Interregnum, Vol. 1 (of 3), 1603-1642 (History, Non-Fiction)
Southern Literature From 1579-1895: A comprehensive review, with copious extracts and criticisms; for the use of schools and the general reader (Literary Criticism/History, Non-Fiction)
An Etymological Dictionary of the Scottish Language: in which the words are explained in their different senses, authorized by the names of the writers by whom they are used, or the titles of the works in which they occur, and deduced from their originals (Reference/Linguistics, Non-Fiction)
Rambles and Recollections of an Indian Official (Travel/Autobiography, Non-Fiction)
Club Life of London, Vol. 1 (of 2): With Anecdotes of the Clubs, Coffee-Houses and Taverns of the Metropolis During the 17th, 18th, and 19th Centuries (History/Social, Non-Fiction)
The Invention of the Sewing Machine (History/Technology, Non-Fiction)
The Bible Story (Religion/Children's, Non-Fiction)
The History of Painting in Italy, Vol. 3 (of 6): From the Period of the Revival of the Fine Arts to the End of the Eighteenth Century (Art/History, Non-Fiction)
Korean Fairy Tales (Folklore/Fiction, Short Stories Collection)
History of circumcision from the earliest times to the present : $b Moral and physical reasons for its performance, with a history of eunuchism, hermaphrodism, etc., and of the different operations practiced upon the prepuce (Medicine/History, Non-Fiction)
Pagan Papers (Essays/Humor, Non-Fiction)
New lives for old (Fiction, Novel)
Anthony Trollope; His Work, Associates and Literary Originals (Biography/Literary Criticism, Non-Fiction)
Devil Stories: An Anthology (Fiction, Short Stories/Horror)
The Water-Babies: A Fairy Tale for a Land-Baby (Children's Fiction, Fantasy Novel)
Norfolk Annals, Vol. 2: A Chronological Record of Remarkable Events in the Nineteeth Century (History, Non-Fiction)
History of Brighthelmston; or, Brighton as I View it and Others Knew It: With a Chronological Table of Local Events (History, Non-Fiction)
The Coming Race (Science Fiction, Utopian Novel)
Memoir of Jane Austen (Biography/Literary, Non-Fiction)
Laurence Sterne in Germany: A Contribution to the Study of the Literary Relations of; England and Germany in the Eighteenth Century (Literary Criticism/History, Non-Fiction)
Narrative of the Circumnavigation of the Globe by the Austrian Frigate Novara, Volume II: (Commodore B. Von Wullerstorf-Urbair,) Undertaken by Order of the Imperial Government in the Years 1857, 1858, & 1859, Under the Immediate Auspices of His I. and R. Highness the Archduke Ferdinand Maximilian, Commander-In-Chief of the; Austrian Navy. (Travel/Exploration, Non-Fiction)
Old Greek Stories (Children's Fiction, Mythology Retellings)
A Budget of Paradoxes, Volume II (Mathematics/History, Non-Fiction)
The Japanese Spirit (Philosophy/Cultural, Non-Fiction)
History of the Wars, Books III and IV: The Vandalic War (History/Military, Non-Fiction)
A History of Matrimonial Institutions, Vol. 3 of 3 (History/Sociology, Non-Fiction)
Virginia's Attitude Toward Slavery and Secession (History, Non-Fiction)
The History of Modern Painting, Volume 3 (of 4): Revised edition continued by the author to the end of the XIX century (Art/History, Non-Fiction)
Alexandria: A History and a Guide (History/Travel, Non-Fiction)
The Life of Cesare Borgia (Biography/History, Non-Fiction)
Sailing Alone Around the World (Travel/Autobiography, Non-Fiction)
Emily Brontë (Biography/Literary Criticism, Non-Fiction)
Humorous Ghost Stories (Horror Fiction, Short Stories Collection)
Petrarch's Secret; or, the Soul's Conflict with Passion: Three Dialogues Between Himself and S. Augustine (Philosophy/Religion, Non-Fiction)
The Life and Work of Susan B. Anthony (Volume 1 of 2): Including Public Addresses, Her Own Letters and Many From Her Contemporaries During Fifty Years (Biography/Feminism, Non-Fiction)
The Biotic Associations of Cockroaches (Science/Entomology, Non-Fiction)
The Deipnosophists; or, Banquet of the Learned of Athenæus, Vol. 1 (of 3) (Philosophy/History, Non-Fiction)
Mad Shepherds, and Other Human Studies (Essays/Social, Non-Fiction)
A Critical Exposition of the Popular 'Jihád': Showing that all the Wars of Mohammad Were Defensive; and; that Aggressive War, or Compulsory Conversion, is not; Allowed in The Koran - 1885 (Religion/History, Non-Fiction)
A History of Horncastle, from the earliest period to the present time (History, Non-Fiction)
The Ornithology of Shakespeare: Critically examined, explained and illustrated (Literary Criticism/Nature, Non-Fiction)
The Three Brontës (Biography/Literary Criticism, Non-Fiction)
The Englishman in China During the Victorian Era, Vol. 2 (of 2): As Illustrated in the Career of Sir Rutherford Alcock, K.C.B., D.C.L., Many Years Consul and Minister in China and Japan (History/Biography, Non-Fiction)
A History of Magic and Experimental Science, Volume 1 (of 2): During the First Thirteen Centuries of Our Era (History/Science, Non-Fiction)
Lone Star Planet (Science Fiction, Novel)
Through South Africa: His Visit to Rhodesia, the Transvaal, Cape Colony, Natal (Travel/History, Non-Fiction)
The Initiates of the Flame (Occult/Philosophy, Non-Fiction)
Tea-Cup Reading and Fortune-Telling by Tea Leaves, by a Highland Seer (Occult/Guide, Non-Fiction)
The Ship of Fools, Volume 1 (Satire/Poetry)
The Veil Lifted: Modern Developments of Spirit Photography (Occult/Photography, Non-Fiction)
Mr. Honey's Medium Business Dictionary (German-English) (Reference/Dictionary, Non-Fiction)
Lillian's vow : $b or, The mystery of Raleigh House (Fiction, Novel)
Dainty's Cruel Rivals; Or, The Fatal Birthday (Fiction, Romance Novel)
The Melting-Pot (Drama/Play)
Clicking red heels (Fiction, Novel)
An Outline of English Speech-craft (Linguistics, Non-Fiction)
The Theory of the Leisure Class (Sociology/Economics, Non-Fiction)
Letters on England (Essays/Travel, Non-Fiction)
The Oxford Reformers: John Colet, Erasmus, and Thomas More (Biography/History, Non-Fiction)
The Modern Regime, Volume 1 (History/Politics, Non-Fiction)
Handbook of Birmingham: Prepared for the Members of the British Association, 1886 (History/Guide, Non-Fiction)
The Conspiracy of Gianluigi Fieschi, or, Genoa in the sixteenth century. (History, Non-Fiction)
The rat and its relation to the public health (Science/Medicine, Non-Fiction)
A Pocket Dictionary: Welsh-English (Reference/Dictionary, Non-Fiction)
"""

# Build dictionary
genre_map = {}
for line in titles_with_genres.splitlines():
    line = line.strip()
    if line.endswith(")"):  # only process lines with parentheses
        try:
            title, genre = line.rsplit("(", 1)   # split only at the last "("
            genre = genre.strip(") ")            # remove closing parenthesis
            genre_map[title.strip()] = genre.strip()
        except ValueError:
            pass  # skip malformed lines

# Print in nice dict format
print("# Define the mapping of title → genre")
print("genre_map = {")
for title, genre in genre_map.items():
    print(f'    "{title}": "{genre}",')
print("}")

# Define the mapping of title → genre
genre_map = {
    "The History of Modern Painting, Volume 1 (of 4): Revised edition continued by the author to the end of the XIX century": "Art/History, Non-Fiction",
    "Lays of Ancient Rome": "Poetry, Historical Verse",
    "Jerusalem Explored, Volume 1—Text: Being a Description of the Ancient and Modern City, with Numerous Illustrations Consisting of Views, Ground Plans and Sections": "History/Travel, Non-Fiction",
    "The Gutenberg Webster's Unabridged Dictionary: Section X, Y, and Z": "Reference/Dictionary, Non-Fiction",
    "The Box-Car Children": "Children's Fiction, Adventure Novel",
    "Sketches and Studies in Italy and Greece, Complete: Series I, II, and III": "Travel/Essays, Non-Fiction",
    "The Hacker Crackdown: Law and Disorder on the Electronic Frontier": "Technology/History, Non-Fiction",
    "A collection of Latin maxims and phrases literally translated: Intended for the use of students for all legal examinations": "Reference/

In [8]:
import pandas as pd

# === Load your dataset ===
df = pd.read_csv("author_identification_dataset_final.csv")

# === Define the mapping of title → genre ===
# Define the mapping of title → genre
genre_map = {
    "Lincoln Letters": "History/Politics, Non-Fiction",
    "Lincoln's First Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Gettysburg Address, given November 19, 1863": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Inaugurals, Addresses and Letters (Selections)": "History/Politics, Non-Fiction Collection",
    "Lincoln's Second Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Speeches and Letters of Abraham Lincoln, 1832-1865": "History/Politics, Non-Fiction Collection",
    "State of the Union Addresses": "History/Politics, Non-Fiction",
    "The Emancipation Proclamation": "History/Politics, Non-Fiction",
    "The Life and Public Service of General Zachary Taylor: An Address": "Biography/History, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 1: 1832-1843": "History/Politics, Non-Fiction Collection",
    "The Writings of Abraham Lincoln, Volume 2: 1843-1858": "History/Politics, Non-Fiction Collection",
    "The Writings of Abraham Lincoln, Volume 3": "History/Politics, Non-Fiction Collection",
    "The Mysterious Affair at Styles": "Mystery Fiction, Detective Novel",
    "The Secret Adversary": "Mystery Fiction, Adventure Novel",
    "Relativity: The Special and General Theory": "Science/Physics, Non-Fiction",
    "Sidelights on Relativity": "Science/Physics, Non-Fiction",
    "Crome Yellow": "Fiction, Satirical Novel",
    "Mortal Coils": "Fiction, Short Stories",
    "An Essay on Criticism": "Poetry, Literary Criticism",
    "Essay on Man": "Poetry, Philosophical Verse",
    "The Poetical Works of Alexander Pope, Volume 1": "Poetry Collection",
    "The Poetical Works of Alexander Pope, Volume 2": "Poetry Collection",
    "The Rape of the Lock and Other Poems": "Poetry, Satirical Verse",
    "The Works of Alexander Pope, Volume 1": "Poetry/Literary Collection",
    "Darwinism": "Science/Biology, Non-Fiction Essays",
    "Is Mars Habitable?": "Science/Astronomy, Non-Fiction",
    "Island Life": "Science/Biology, Non-Fiction",
    "An Occurrence at Owl Creek Bridge": "Fiction, Short Story/War",
    "Can Such Things Be?": "Horror/Supernatural Fiction, Short Stories",
    "Cobwebs From an Empty Skull": "Fiction, Satirical Fables",
    "Fantastic Fables": "Fiction, Fables/Satire",
    "Present at a Hanging and Other Ghost Stories": "Horror/Supernatural Fiction, Short Stories",
    "Shapes of Clay": "Poetry Collection",
    "The Collected Works of Ambrose Bierce, Volume 1": "Fiction/Non-Fiction Collection, Satire",
    "The Collected Works of Ambrose Bierce, Volume 2": "Fiction/Non-Fiction Collection, Satire",
    "A Collection of Ballads": "Poetry/Folklore Collection",
    "A Monk of Fife": "Historical Fiction, Novel",
    "A Short History of Scotland": "History, Non-Fiction",
    "Aaron Trow": "Fiction, Short Story",
    "An Eye for an Eye": "Fiction, Novel",
    "Autobiography of Anthony Trollope": "Autobiography, Non-Fiction",
    "A Bride of the Plains": "Historical Fiction, Romance Novel",
    "Castles in the Air": "Fiction, Short Stories",
    "El Dorado": "Historical Fiction, Adventure Novel",
    "A Collection of Beatrix Potter Stories": "Children's Fiction, Animal Tales",
    "The Great Big Treasury of Beatrix Potter": "Children's Fiction, Animal Tales Collection",
    "The Story of Miss Moppet": "Children's Fiction, Picture Book",
    "The Tailor of Gloucester": "Children's Fiction, Animal Tale",
    "The Tale of Ginger and Pickles": "Children's Fiction, Animal Tale",
    "The Tale of Jemima Puddle-Duck": "Children's Fiction, Animal Tale",
    "The Tale of Johnny Town-Mouse": "Children's Fiction, Animal Tale",
    "The Tale of Mr. Tod": "Children's Fiction, Animal Tale",
    "The Tale Of Peter Rabbit": "Children's Fiction, Animal Tale",
    "The Tale of Samuel Whiskers": "Children's Fiction, Animal Tale",
    "The Tale of the Flopsy Bunnies": "Children's Fiction, Animal Tale",
    "The Tale of the Pie and the Patty Pan": "Children's Fiction, Animal Tale",
    "The Tale of Tom Kitten": "Children's Fiction, Animal Tale",
    "Alroy": "Historical Fiction, Novel",
    "Coningsby": "Fiction, Political Novel",
    "Endymion": "Fiction, Political Novel",
    "Autobiography of Benjamin Franklin, Version 1": "Autobiography, Non-Fiction",
    "Autobiography of Benjamin Franklin, Version 2": "Autobiography, Non-Fiction",
    "Autobiography of Benjamin Franklin, Version 4": "Autobiography, Non-Fiction",
    "Mysticism and Logic and Other Essays": "Philosophy/Essays, Non-Fiction",
    "Political Ideals": "Philosophy/Politics, Non-Fiction",
    "Proposed Roads To Freedom": "Philosophy/Politics, Non-Fiction",
    "The Analysis of Mind": "Philosophy/Psychology, Non-Fiction",
    "The Practice and Theory of Bolshevism": "History/Politics, Non-Fiction",
    "Dracula's Guest": "Horror Fiction, Short Stories",
    "The Jewel of Seven Stars": "Horror Fiction, Novel",
    "The Lady of the Shroud": "Horror Fiction, Novel",
    "A First Family of Tasajara": "Fiction, Western Novel",
    "A Millionaire of Rough-and-Ready": "Fiction, Western Short Story",
    "A Phyllis of the Sierras": "Fiction, Western Novel",
    "A Sappho of Green Springs": "Fiction, Western Short Stories",
    "A Ward of the Golden Gate": "Fiction, Western Novel",
    "A Monograph on the Sub-class Cirripedia (Volume 1 of 2)": "Science/Biology, Non-Fiction",
    "A Naturalist's Voyage Round the World": "Science/Travel, Non-Fiction",
    "A Child's History of England": "History, Non-Fiction for Children",
    "A Christmas Carol": "Fiction, Ghost Story/Novella",
    "A Tale of Two Cities": "Historical Fiction, Novel",
    "Alexandria and her Schools": "History/Philosophy, Non-Fiction",
    "All Saints' Day and Other Sermons": "Religion/Sermons, Non-Fiction",
    "Andromeda and Other Poems": "Poetry Collection",
    "At Last": "Science/Travel, Non-Fiction",
    "Daily Thoughts": "Essays/Philosophy, Non-Fiction",
    "Jane Eyre": "Fiction, Gothic Romance Novel",
    "Shirley": "Fiction, Social Novel",
    "A Book of Golden Deeds": "History/Biography Collection, Non-Fiction",
    "A Modern Telemachus": "Fiction, Adventure Novel",
    "A Reputed Changeling": "Fiction, Historical Novel",
    "Abbeychurch": "Fiction, Domestic Novel",
    "Bay": "Poetry Collection",
    "England, My England": "Fiction, Short Stories",
    "Fantasia of the Unconscious": "Philosophy/Psychology, Non-Fiction",
    "Look! We Have Come Through!": "Poetry Collection",
    "New Poems": "Poetry Collection",
    "Sea and Sardinia": "Travel/Non-Fiction",
    "Sons and Lovers": "Fiction, Novel",
    "A Journal of the Plague Year": "Historical Fiction, Novel",
    "A New Voyage Round the World by a Course Never Sailed Before": "Fiction, Adventure Novel",
    "A True Relation of the Apparition of one Mrs. Veal": "Fiction, Ghost Story",
    "A Vindication of the Press": "Essays/Politics, Non-Fiction",
    "An Answer to a Question that Nobody thinks of, viz., But what if the Queen should Die?": "Essays/Politics, Non-Fiction",
    "An Appeal to Honour and Justice, Though It Be of His Worst Enemies": "Essays/Politics, Non-Fiction",
    "An Essay Upon Projects": "Essays/Economics, Non-Fiction",
    "And What if the Pretender should Come?": "Essays/Politics, Non-Fiction",
    "Atalantis Major": "Fiction, Satirical Novel",
    "Augusta Triumphans": "Essays/Social Commentary, Non-Fiction",
    "Edgar Allan Poe's Complete Poetical Works": "Poetry Collection",
    "Eureka": "Philosophy/Science, Non-Fiction Prose Poem",
    "The Cask of Amontillado": "Horror Fiction, Short Story",
    "The Fall of the House of Usher": "Horror Fiction, Short Story",
    "The Raven": "Poetry",
    "The Works of Edgar Allan Poe Volume 1": "Fiction/Poetry Collection, Horror/Mystery",
    "The Works of Edgar Allan Poe Volume 3": "Fiction/Poetry Collection, Horror/Mystery",
    "A Princess of Mars": "Science Fiction, Adventure Novel",
    "At the Earth's Core": "Science Fiction, Adventure Novel",
    "Jungle Tales of Tarzan": "Adventure Fiction, Short Stories",
    "Pellucidar": "Science Fiction, Adventure Novel",
    "Tarzan and the Jewels of Opar": "Adventure Fiction, Novel",
    "Tarzan of the Apes": "Adventure Fiction, Novel",
    "Burke's Speech on Conciliation with America": "History/Politics, Speech/Non-Fiction",
    "Selections from the Speeches and Writings of Edmund Burke": "History/Politics, Non-Fiction Collection",
    "A Lost Leader": "Poetry",
    "A Maker of History": "Fiction, Spy Novel",
    "A Monk of Cruta": "Fiction, Adventure Novel",
    "American Boy's Life of Theodore Roosevelt": "Biography, Non-Fiction for Youth",
    "Boys of The Fort": "Adventure Fiction, Youth Novel",
    "Dave Porter and His Double": "Adventure Fiction, Youth Novel",
    "Dave Porter and the Runaways": "Adventure Fiction, Youth Novel",
    "'He Giveth His Beloved Sleep'": "Poetry",
    "Sonnets from the Portuguese": "Poetry Collection",
    "The Poetical Works of Elizabeth Barrett Browning Volume I": "Poetry Collection",
    "Wuthering Heights": "Fiction, Gothic Romance Novel",
    "Anna Christie": "Drama/Play",
    "The First Man": "Drama/Play",
    "The Hairy Ape": "Drama/Play",
    "The Straw": "Drama/Play",
    "Certain Noble Plays of Japan": "Drama/Plays, Translation",
    "Hugh Selwyn Mauberley": "Poetry",
    "A Chosen Few": "Fiction, Short Stories",
    "A Jolly Fellowship": "Adventure Fiction, Youth Novel",
    "Amos Kilbright; His Adscititious Experiences": "Fantasy Fiction, Short Stories",
    "Buccaneers and Pirates of Our Coasts": "History/Adventure, Non-Fiction",
    "John Gayther's Garden and the Stories Told Therein": "Fiction, Short Stories",
    "A Short History of England": "History, Non-Fiction",
    "All Things Considered": "Essays/Humor, Non-Fiction",
    "Appreciations and Criticisms of the Works of Charles Dickens": "Literary Criticism, Non-Fiction",
    "George Bernard Shaw": "Biography/Literary Criticism, Non-Fiction",
    "A Chapter of Adventures": "Adventure Fiction, Youth Novel",
    "A Final Reckoning": "Adventure Fiction, Novel",
    "A Girl of the Commune": "Historical Fiction, Novel",
    "A Jacobite Exile": "Historical Fiction, Novel",
    "A Treatise on Parents and Children": "Essays/Education, Non-Fiction",
    "An Unsocial Socialist": "Fiction, Satirical Novel",
    "Androcles and the Lion": "Drama/Play",
    "Arms and the Man": "Drama/Play",
    "Caesar and Cleopatra": "Drama/Play",
    "Candida": "Drama/Play",
    "Adam Bede": "Fiction, Novel",
    "Brother Jacob": "Fiction, Short Story",
    "Daniel Deronda": "Fiction, Novel",
    "Charles Darwin": "Biography, Non-Fiction",
    "Falling in Love": "Science/Essays, Non-Fiction",
    "Hilda Wade": "Mystery Fiction, Novel",
    "A Daughter of the Middle Border": "Autobiography, Non-Fiction",
    "A Son of the Middle Border": "Autobiography, Non-Fiction",
    "A Spoil of Office": "Fiction, Political Novel",
    "A Prairie Courtship": "Fiction, Romance Novel",
    "Blake's Burden": "Fiction, Adventure Novel",
    "Household Papers and Stories": "Essays/Fiction Collection",
    "Lady Byron Vindicated": "Biography/Defense, Non-Fiction",
    "Oldtown Fireside Stories": "Fiction, Short Stories/Folklore",
    "The Chronicles of Clovis": "Fiction, Satirical Short Stories",
    "The Toys of Peace": "Fiction, Satirical Short Stories",
    "The Unbearable Bassington": "Fiction, Satirical Novel",
    "When William Came": "Fiction, Satirical Novel",
    "A Plea for Captain John Brown": "Essays/Politics, Non-Fiction",
    "Cape Cod": "Travel/Nature, Non-Fiction",
    "Excursions": "Essays/Nature, Non-Fiction",
    "On the Duty of Civil Disobedience": "Philosophy/Politics, Non-Fiction Essay",
    "A Passionate Pilgrim": "Fiction, Short Story",
    "A Little Tour in France, Version 1": "Travel, Non-Fiction",
    "A Little Tour in France, Version 2": "Travel, Non-Fiction",
    "A Small Boy and Others": "Autobiography, Non-Fiction",
    "Allan and the Holy Flower": "Adventure Fiction, Novel",
    "Allan's Wife": "Adventure Fiction, Short Stories",
    "A Modern Utopia": "Science Fiction, Utopian Novel",
    "A Short History of the World": "History, Non-Fiction",
    "Ann Veronica": "Fiction, Social Novel",
    "Essays on Education and Kindred Subjects": "Essays/Education, Non-Fiction",
    "Essays: Scientific, Political, & Speculative, Volume I": "Essays/Science/Politics, Non-Fiction",
    "The Right To Ignore The State": "Philosophy/Politics, Non-Fiction",
    "Battle-Pieces and Aspects of the War": "Poetry Collection",
    "I and My Chimney": "Fiction, Short Story",
    "Israel Potter": "Historical Fiction, Novel",
    "John Marr and Other Poems": "Poetry Collection",
    "Mardi: and A Voyage Thither, Volume 1": "Fiction, Allegorical Novel",
    "Howard Pyle's Book of Pirates": "Adventure Fiction, Illustrated Stories",
    "Men of Iron": "Historical Fiction, Youth Novel",
    "Pepper & Salt": "Children's Fiction, Fairy Tales",
    "Stolen Treasure": "Adventure Fiction, Short Stories",
    "The Merry Adventures of Robin Hood": "Adventure Fiction, Novel/Folklore",
    "The Rose of Paradise": "Adventure Fiction, Novel",
    "Youth": "Fiction, Novella",
    "A Daughter of the Snows": "Adventure Fiction, Novel",
    "Brown Wolf and Other Jack London Stories": "Fiction, Short Stories",
    "Burning Daylight": "Adventure Fiction, Novel",
    "Alexander the Great": "Biography/History, Non-Fiction",
    "Caleb in the Country": "Children's Fiction, Moral Tale",
    "Cleopatra": "Biography/History, Non-Fiction",
    "Forests of Maine": "Children's Fiction, Nature Tale",
    "Genghis Khan": "Biography/History, Non-Fiction",
    "Goblin Tales of Lancashire": "Folklore/Fiction, Short Stories",
    "Afloat And Ashore": "Adventure Fiction, Novel",
    "Autobiography of a Pocket-Hankerchief": "Fiction, Short Story",
    "A Portrait of the Artist as a Young Man": "Fiction, Bildungsroman Novel",
    "Chamber Music": "Poetry Collection",
    "Dubliners": "Fiction, Short Stories",
    "A Holiday in Bed and Other Sketches": "Essays/Humor, Non-Fiction",
    "A Window in Thrums": "Fiction, Novel",
    "Alice Sit-By-The-Fire": "Drama/Play",
    "Auld Licht Idylls": "Fiction, Short Stories",
    "Courage": "Essays/Inspirational, Non-Fiction",
    "Dear Brutus": "Drama/Play",
    "Margaret Ogilvy": "Biography, Non-Fiction",
    "My Lady Nicotine": "Essays/Humor, Non-Fiction",
    "A District Messenger Boy and a Necktie Party": "Adventure Fiction, Youth Stories",
    "An Amateur Fireman": "Adventure Fiction, Youth Novel",
    "Aunt Hannah and Seth": "Fiction, Youth Novel",
    "Commodore Barney's Young Spies": "Historical Fiction, Youth Novel",
    "Corporal 'Lige's Recruit": "Historical Fiction, Youth Novel",
    "Defending the Island": "Adventure Fiction, Youth Novel",
    "Dick in the Desert": "Adventure Fiction, Youth Novel",
    "Among My Books, First Series": "Literary Criticism, Non-Fiction",
    "Among My Books, Second Series": "Literary Criticism, Non-Fiction",
    "My Garden Acquaintance": "Essays/Nature, Non-Fiction",
    "Poems of James Russell Lowell": "Poetry Collection",
    "Emma": "Fiction, Romance Novel",
    "Lady Susan": "Fiction, Epistolary Novel",
    "Love And Freindship And Other Early Works": "Fiction, Satirical Stories",
    "Mansfield Park": "Fiction, Romance Novel",
    "All Roads Lead to Calvary": "Fiction, Novel",
    "Evergreens": "Poetry Collection",
    "Fanny and the Servant Problem": "Drama/Play",
    "Idle Thoughts of an Idle Fellow": "Essays/Humor, Non-Fiction",
    "John Ingerfield and Other Stories": "Fiction, Short Stories",
    "Mrs. Korner Sins Her Mercies": "Fiction, Short Story",
    "Novel Notes": "Fiction, Humorous Novel",
    "The Heavenly Footman": "Religion/Allegory, Non-Fiction",
    "The Holy War": "Religion/Allegory, Fiction",
    "The Jerusalem Sinner Saved": "Religion/Sermon, Non-Fiction",
    "The Pharisee And The Publican": "Religion/Sermon, Non-Fiction",
    "The Pilgrim's Progress": "Religion/Allegory, Fiction",
    "Discourses on Satire and Epic Poetry": "Literary Criticism, Non-Fiction",
    "Dryden's Works Vol. 16 (of 18)": "Poetry/Drama Collection",
    "Dryden's Works Vol. 2 (of 18)": "Poetry/Drama Collection",
    "A Family Man": "Drama/Play",
    "Essays Concerning Letters": "Essays/Literary, Non-Fiction",
    "Essays on Censorship and Art": "Essays/Politics, Non-Fiction",
    "Five Tales": "Fiction, Short Stories",
    "Fraternity": "Fiction, Social Novel",
    "Keats: Poems Published in 1820": "Poetry Collection",
    "Lamia": "Poetry",
    "Letters of John Keats to His Family and Friends": "Biography/Letters, Non-Fiction",
    "Poems 1817": "Poetry Collection",
    "An Essay Concerning Humane Understanding, Volume 1": "Philosophy, Non-Fiction",
    "An Essay Concerning Humane Understanding, Volume 2": "Philosophy, Non-Fiction",
    "Second Treatise of Government": "Philosophy/Politics, Non-Fiction",
    "The Economic Consequences of the Peace": "Economics/History, Non-Fiction",
    "L'Allegro, Il Penseroso, Comus, and Lycidas": "Poetry Collection",
    "Milton's Comus": "Poetry/Drama",
    "Minor Poems by Milton": "Poetry Collection",
    "Paradise Lost": "Poetry, Epic",
    "Paradise Regained": "Poetry, Epic",
    "Burke": "Biography, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 1, Robespierre": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 2, Carlyle": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 3, Byron": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 4, Macaulay": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 1, Vauvenargues": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 2, Turgot": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 3, Condorcet": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 4, Joseph de Maistre": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 1, On Popular Culture": "Literary Criticism/Essays, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 10, Auguste Comte": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 4, The Life of George Eliot": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 5, On Pattison's Memoirs": "Literary Criticism/Biography, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 7, A Sketch": "Literary Criticism/Essays, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 8, France in the Eighteenth Century": "Literary Criticism/History, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 9, The Expansion of England": "Literary Criticism/History, Non-Fiction",
    "Ariadne Florentina": "Art Criticism, Non-Fiction",
    "Hortus Inclusus": "Letters/Nature, Non-Fiction",
    "Lectures on Architecture and Painting": "Art Criticism, Non-Fiction",
    "A Tale of a Tub": "Fiction, Satire",
    "Gulliver's Travels": "Fiction, Satirical Adventure Novel",
    "The Battle of the Books": "Fiction, Satire",
    "The Journal to Stella": "Letters/Biography, Non-Fiction",
    "A Personal Record": "Autobiography, Non-Fiction",
    "A Set of Six": "Fiction, Short Stories",
    "Almayer's Folly": "Fiction, Adventure Novel",
    "Amy Foster": "Fiction, Short Story",
    "An Outcast of the Islands": "Fiction, Adventure Novel",
    "Captain Sword and Captain Pen": "Poetry, Satirical",
    "Stories from the Italian Poets: With Lives of the Writers, Volume 1": "Poetry/Biography, Non-Fiction",
    "A Tangled Tale": "Fiction, Mathematical Puzzles",
    "Eight or Nine Wise Words about Letter-Writing": "Essays/Humor, Non-Fiction",
    "Rhyme? And Reason?": "Poetry/Humor Collection",
    "The Game of Logic": "Philosophy/Mathematics, Non-Fiction",
    "The Hunting of the Snark": "Poetry, Nonsense Verse",
    "Through the Looking-Glass": "Children's Fiction, Fantasy Novel",
    "Childe Harold's Pilgrimage": "Poetry, Epic",
    "Don Juan": "Poetry, Epic Satire",
    "Fugitive Pieces": "Poetry Collection",
    "Becket and other plays": "Drama/Plays",
    "Enoch Arden, &c": "Poetry Collection",
    "Idylls of the King": "Poetry, Epic",
    "Lady Clare": "Poetry",
    "Queen Mary and Harold": "Drama/Plays",
    "The Princess": "Poetry",
    "The Suppressed Poems of Alfred Lord Tennyson": "Poetry Collection",
    "Aunt Jo's Scrap-Bag": "Fiction, Short Stories for Youth",
    "Behind A Mask, Or A Woman's Power": "Fiction, Gothic Novella",
    "Anne Of Avonlea": "Fiction, Youth Novel",
    "Anne Of Green Gables": "Fiction, Youth Novel",
    "Anne Of The Island": "Fiction, Youth Novel",
    "Anne's House of Dreams": "Fiction, Youth Novel",
    "American Fairy Tales": "Children's Fiction, Fairy Tales",
    "Aunt Jane's Nieces Abroad": "Adventure Fiction, Youth Novel",
    "Aunt Jane's Nieces and Uncle John": "Adventure Fiction, Youth Novel",
    "Aunt Jane's Nieces at Millville": "Adventure Fiction, Youth Novel",
    "Aunt Jane's Nieces at Work": "Adventure Fiction, Youth Novel",
    "1601": "Fiction, Satirical Short Story",
    "A Connecticut Yankee in King Arthur's Court, Complete": "Fiction, Satirical Fantasy Novel",
    "A Dog's Tale": "Fiction, Animal Story",
    "A Tramp Abroad": "Travel/Humor, Non-Fiction",
    "Adventures of Huckleberry Finn, Complete": "Adventure Fiction, Novel",
    "Frankenstein": "Science Fiction, Gothic Novel",
    "Mathilda": "Fiction, Gothic Novella",
    "Notes to the Complete Poetical Works of Percy Bysshe Shelley": "Literary Criticism, Non-Fiction",
    "Proserpine and Midas": "Drama/Mythological Plays",
    "The Last Man": "Science Fiction, Apocalyptic Novel",
    "Mariposilla": "Fiction, Novel",
    "Experimental Researches in Electricity, Volume 1": "Science/Physics, Non-Fiction",
    "A Book of Autographs": "Essays/Literary, Non-Fiction",
    'A Rill from the Town Pump (From "Twice Told Tales")': "Fiction, Short Story",
    'A Select Party (From "Mosses From An Old Manse")': "Fiction, Short Story",
    "A Virtuoso's Collection (From \"Mosses From An Old Manse\")": "Fiction, Short Story",
    "A Wonder Book and Tanglewood Tales": "Children's Fiction, Mythology Retellings",
    "A Wonder Book for Girls & Boys": "Children's Fiction, Mythology Retellings",
    "An Old Woman's Tale": "Fiction, Short Story",
    "Buds and Bird Voices (From \"Mosses From An Old Manse\")": "Essays/Nature, Non-Fiction",
    "Chippings With A Chisel (From \"Twice Told Tales\")": "Fiction, Short Story",
    "Doctor Grimshawe's Secret": "Fiction, Novel Fragment",
    "Dr. Bullivant": "Fiction, Short Story",
    "Earth's Holocaust (From \"Mosses From An Old Manse\")": "Fiction, Allegorical Short Story",
    "Edward Fane's Rosebud (From \"Twice Told Tales\")": "Fiction, Short Story",
    "Fancy's Show-Box (From \"Twice Told Tales\")": "Fiction, Short Story",
    "Fanshawe": "Fiction, Novel",
    "Heart of the West": "Fiction, Western Short Stories",
    "Options": "Fiction, Short Stories",
    "A Critic in Pall Mall": "Literary Criticism, Non-Fiction",
    "A House of Pomegranates": "Children's Fiction, Fairy Tales",
    "A Woman of No Importance": "Drama/Play",
    "An Ideal Husband": "Drama/Play",
    "Essays and Lectures": "Essays/Literary, Non-Fiction",
    "Intentions": "Essays/Literary, Non-Fiction",
    "Lord Arthur Savile's Crime and Other Stories": "Fiction, Short Stories",
    "A Vindication of Natural Diet": "Essays/Health, Non-Fiction",
    "Adonais": "Poetry, Elegy",
    "The Complete Poetical Works of Percy Bysshe Shelley": "Poetry Collection",
    "A Damsel in Distress": "Fiction, Humorous Novel",
    "A Prefect's Uncle": "Fiction, School Story",
    "A Wodehouse Miscellany": "Fiction/Essays Collection",
    "Jill the Reckless": "Fiction, Humorous Novel",
    "Noto, An Unexplored Corner of Japan": "Travel, Non-Fiction",
    "The Soul of the Far East": "Philosophy/Cultural, Non-Fiction",
    "Beyond Lies the Wub": "Science Fiction, Short Story",
    "Beyond the Door": "Science Fiction, Short Story",
    "Mr. Spaceship": "Science Fiction, Short Story",
    "Piper in the Woods": "Science Fiction, Short Story",
    "Second Variety": "Science Fiction, Short Story",
    "The Crystal Crypt": "Science Fiction, Short Story",
    "The Eyes Have It": "Science Fiction, Short Story",
    "The Gun": "Science Fiction, Short Story",
    "The Variable Man": "Science Fiction, Novella",
    "Away in the Wilderness": "Adventure Fiction, Youth Novel",
    "Battles with the Sea": "Adventure Fiction, Non-Fiction Elements",
    "Black Ivory": "Adventure Fiction, Novel",
    "Blown to Bits": "Adventure Fiction, Novel",
    "Blue Lights": "Adventure Fiction, Novel",
    "Captain Blood": "Adventure Fiction, Pirate Novel",
    "Essays, First Series": "Essays/Philosophy, Non-Fiction",
    "Essays, Second Series": "Essays/Philosophy, Non-Fiction",
    "Essays": "Essays/Philosophy, Non-Fiction",
    "Nature": "Essays/Philosophy, Non-Fiction",
    "Poems": "Poetry Collection",
    "Representative Men": "Biography/Essays, Non-Fiction",
    "Scarborough and the Critic": "Drama/Play",
    "St. Patrick's Day": "Drama/Play",
    "The Duenna": "Drama/Play",
    "The Rivals": "Drama/Play",
    "The School For Scandal": "Drama/Play",
    "Christmas Eve": "Poetry",
    "Dramatic Romances": "Poetry Collection",
    "Men and Women": "Poetry Collection",
    "Selections from the Poems and Plays of Robert Browning": "Poetry/Drama Collection",
    "The Pied Piper of Hamelin": "Poetry, Narrative Verse",
    "Poems And Songs Of Robert Burns": "Poetry Collection",
    "Tam O'Shanter": "Poetry",
    "The Letters of Robert Burns": "Biography/Letters, Non-Fiction",
    "A Boy's Will": "Poetry Collection",
    "Mountain Interval": "Poetry Collection",
    "North of Boston": "Poetry Collection",
    "Micrographia": "Science/Biology, Non-Fiction",
    "A Child's Garden of Verses, Verse 130": "Poetry, Children's",
    "A Child's Garden of Verses, Verse 154": "Poetry, Children's",
    "A Child's Garden of Verses, Verse 158": "Poetry, Children's",
    "A Child's Garden of Verses, Version 2": "Poetry Collection, Children's",
    "A Christmas Sermon": "Religion/Sermon, Non-Fiction",
    "A Footnote to History": "History, Non-Fiction",
    "Across The Plains": "Travel, Non-Fiction",
    "An Inland Voyage": "Travel, Non-Fiction",
    "Ballads": "Poetry Collection",
    "Catriona": "Adventure Fiction, Novel",
    "Poems, 1799": "Poetry Collection",
    "A Diversity of Creatures": "Fiction, Short Stories",
    "A Song of the English": "Poetry",
    "Actions and Reactions": "Fiction, Short Stories",
    "Barrack-Room Ballads": "Poetry Collection",
    "France At War": "History/War, Non-Fiction",
    "From Sea to Sea": "Travel, Non-Fiction",
    "Biographia Epistolaris, Volume 1": "Biography/Letters, Non-Fiction",
    "Biographia Literaria": "Literary Criticism, Non-Fiction",
    "Coleridge's Ancient Mariner and Select Poems": "Poetry Collection",
    "Coleridge's Literary Remains, Volume 1": "Literary Criticism, Non-Fiction",
    "Babbitt": "Fiction, Satirical Novel",
    "Free Air": "Fiction, Road Novel",
    "A Desert Drama": "Adventure Fiction, Novel",
    "A Study In Scarlet": "Mystery Fiction, Detective Novel",
    "A Visit to Three Fronts": "History/War, Non-Fiction",
    "Beyond the City": "Fiction, Novel",
    "Danger! and Other Stories": "Fiction, Short Stories",
    "His Last Bow": "Mystery Fiction, Short Stories",
    "Memoirs of Sherlock Holmes": "Mystery Fiction, Short Stories",
    "Micah Clarke": "Historical Fiction, Novel",
    "Finger Prints": "Science/Forensics, Non-Fiction",
    "Inquiries into Human Faculty and Its Development": "Science/Psychology, Non-Fiction",
    "The Art of Travel": "Travel/Guide, Non-Fiction",
    "Opticks": "Science/Physics, Non-Fiction",
    "The Chronology of Ancient Kingdoms Amended": "History, Non-Fiction",
    "Himalayan Journals": "Travel/Science, Non-Fiction",
    "First Footsteps in East Africa": "Travel/Adventure, Non-Fiction",
    "Bride of Lammermoor": "Historical Fiction, Novel",
    "Chronicles of the Canongate": "Historical Fiction, Short Stories",
    "Fifty Bab Ballads": "Poetry/Humor Collection",
    "Songs of a Savoyard": "Poetry/Humor Collection",
    "The Bab Ballads": "Poetry/Humor Collection",
    "Liberalism and the Social Problem": "Politics/Essays, Non-Fiction",
    "London to Ladysmith via Pretoria": "History/War, Non-Fiction",
    "The Story of the Malakand Field Force": "History/Military, Non-Fiction",
    "Adventurers of the Far North": "History/Exploration, Non-Fiction",
    "Arcadian Adventures with the Idle Rich": "Fiction, Satirical Short Stories",
    "Behind the Beyond": "Fiction, Short Stories",
    "Frenzied Fiction": "Fiction, Humorous Short Stories",
    "Further Foolishness": "Fiction, Humorous Short Stories",
    "Literary Lapses": "Fiction, Humorous Short Stories",
    "My Discovery of England": "Essays/Humor, Non-Fiction",
    "Nonsense Novels": "Fiction, Parody Novels",
    "Eeldrop and Appleplex": "Fiction, Short Story",
    "Ezra Pound: His Metric and Poetry": "Literary Criticism, Non-Fiction",
    "Prufrock and Other Observations": "Poetry Collection",
    "History of Friedrich II of Prussia, Volume 10": "History/Biography, Non-Fiction",
    "History of Friedrich II of Prussia, Volume 11": "History/Biography, Non-Fiction",
    "History of Friedrich II of Prussia, Volume 12": "History/Biography, Non-Fiction",
    "History of Friedrich II of Prussia, Volume 13": "History/Biography, Non-Fiction",
    "A Changed Man and Other Tales": "Fiction, Short Stories",
    "A Group of Noble Dames": "Fiction, Short Stories",
    "A Laodicean": "Fiction, Novel",
    "A Pair of Blue Eyes": "Fiction, Novel",
    'A Critical Examination Of The Position Of Mr. Darwin\'s Work, "On The Origin Of Species"': "Science/Criticism, Non-Fiction",
    "American Addresses, with a Lecture on the Study of Biology": "Science/Lectures, Non-Fiction",
    "Aphorisms and Reflections from the Works of T. H. Huxley": "Philosophy/Science, Non-Fiction",
    "Collected Essays, Volume V": "Essays/Science, Non-Fiction",
    "Criticisms on \"The Origin of Species\"": "Science/Criticism, Non-Fiction",
    "Critiques and Addresses": "Essays/Science, Non-Fiction",
    "An Essay on the Principle of Population": "Economics/Sociology, Non-Fiction",
    "Nature and Progress of Rent": "Economics, Non-Fiction",
    "The Grounds of an Opinion on the Policy of Restricting the Importation of Foreign Corn": "Economics/Politics, Non-Fiction",
    "Blacky the Crow": "Children's Fiction, Animal Tale",
    "Bowser The Hound": "Children's Fiction, Animal Tale",
    "Happy Jack": "Children's Fiction, Animal Tale",
    "Mother West Wind 'Why' Stories": "Children's Fiction, Animal Tales",
    "Mother West Wind \"Where\" Stories": "Children's Fiction, Animal Tales",
    "Mother West Wind's Animal Friends": "Children's Fiction, Animal Tales",
    "Mother West Wind's Children": "Children's Fiction, Animal Tales",
    "Mrs. Peter Rabbit": "Children's Fiction, Animal Tale",
    "Old Granny Fox": "Children's Fiction, Animal Tale",
    "Old Mother West Wind": "Children's Fiction, Animal Tales",
    "The Adventures of Buster Bear": "Children's Fiction, Animal Tale",
    "The Adventures of Chatterer the Red Squirrel": "Children's Fiction, Animal Tale",
    "The Adventures of Danny Meadow Mouse": "Children's Fiction, Animal Tale",
    "The Adventures of Grandfather Frog": "Children's Fiction, Animal Tale",
    "The Adventures of Jerry Muskrat": "Children's Fiction, Animal Tale",
    "Letters of Ulysses S. Grant": "Biography/Letters, Non-Fiction",
    "Personal Memoirs of U. S. Grant": "Autobiography, Non-Fiction",
    "State of the Union Addresses of Ulysses S. Grant": "History/Politics, Non-Fiction",
    "Jacob's Room": "Fiction, Modernist Novel",
    "Monday or Tuesday": "Fiction, Short Stories",
    "Night and Day": "Fiction, Novel",
    "The Voyage Out": "Fiction, Novel",
    "Drum Taps": "Poetry Collection",
    "Collected Poems 1901-1918 in Two Volumes Volume 1": "Poetry Collection",
    "Collected Poems 1901-1918 in Two Volumes Volume 2": "Poetry Collection",
    "Down-Adown-Derry": "Poetry, Children's Collection",
    "Henry Brocken": "Fiction, Fantasy Novel",
    "Memoirs of a Midget": "Fiction, Fantasy Novel",
    "Peacock Pie, A Book of Rhymes": "Poetry, Children's Collection",
    "Songs of Childhood": "Poetry, Children's Collection",
    "The Listeners and Other Poems": "Poetry Collection",
    "The Return": "Fiction, Supernatural Novel",
    "The Three Mulla-mulgars": "Children's Fiction, Fantasy Novel",
    "Abbotsford and Newstead Abbey": "Travel/Biography, Non-Fiction",
    "Bracebridge Hall, or The Humorists": "Fiction, Sketches",
    "Chronicle of the Conquest of Granada": "History, Non-Fiction",
    "A Rogue's Life": "Fiction, Novel",
    "Antonina": "Historical Fiction, Novel",
    "Poems of William Blake": "Poetry Collection",
    "Songs of Innocence and Songs of Experience": "Poetry Collection",
    "Discoveries": "Drama/Play",
    "Four Years": "Autobiography, Non-Fiction",
    "Ideas of Good and Evil": "Essays/Literary, Non-Fiction",
    "In The Seven Woods": "Poetry Collection",
    "Per Amica Silentia Lunae": "Essays/Philosophy, Non-Fiction",
    "Responsibilities": "Poetry Collection",
    "Reveries over Childhood and Youth": "Autobiography, Non-Fiction",
    "Seven Poems and a Fragment": "Poetry Collection",
    "Synge And The Ireland Of His Time": "Literary Criticism, Non-Fiction",
    "The Celtic Twilight": "Folklore/Essays, Non-Fiction",
    "The Green Helmet and Other Poems": "Poetry Collection",
    "The Land Of Heart's Desire": "Drama/Play",
    "The Secret Rose": "Fiction, Short Stories/Folklore",
    "A Belated Guest": "Essays/Literary, Non-Fiction",
    "A Boy's Town": "Autobiography, Non-Fiction",
    "A Fearful Responsibility and Other Stories": "Fiction, Short Stories",
    "A Foregone Conclusion": "Fiction, Novel",
    "A Hazard of New Fortunes": "Fiction, Social Novel",
    "On Books and the Housing of Them": "Essays/Literary, Non-Fiction",
    "A Crystal Age": "Science Fiction, Utopian Novel",
    "A Little Boy Lost": "Children's Fiction, Adventure",
    "A Shepherd's Life": "Autobiography/Nature, Non-Fiction",
    "A Traveller in Little Things": "Essays/Nature, Non-Fiction",
    "English Literature": "Literary Criticism/History, Non-Fiction",
    "A Brief Account of the Rise and Progress of the People Called Quakers": "Religion/History, Non-Fiction",
    "A Sermon Preached at the Quaker's Meeting House": "Religion/Sermon, Non-Fiction",
    "Caesar's Wife": "Drama/Play",
    "Liza of Lambeth": "Fiction, Novel",
    "Of Human Bondage": "Fiction, Bildungsroman Novel",
    "Orientations": "Fiction, Short Stories",
    "Lyrical Ballads, With Other Poems, 1800, Volume 2": "Poetry Collection",
    "Poems in Two Volumes, Volume 1": "Poetry Collection",
    "Poems in Two Volumes, Volume 2": "Poetry Collection",
    "The Poetical Works of William Wordsworth, Volume 3": "Poetry Collection",
    "A Golden Venture, The Lady of the Barge and Others, Part 11": "Fiction, Short Story",
    "A Master of Craft": "Fiction, Humorous Novel",
    "A Spirit of Avarice, Odd Craft, Part 11": "Fiction, Short Story",
    "Admiral Peters, Odd Craft, Part 14": "Fiction, Short Story",
    "An Adulteration Act, The Lady of the Barge and Others, Part 10": "Fiction, Short Story",
    "At Sunwich Port": "Fiction, Novel",
    "Back to Back, Night Watches, Part 1": "Fiction, Short Story",
    "Bedridden and The Winter Offensive, Deep Waters, Part 8": "Fiction, Short Story",
    "Bill's Lapse, Odd Craft, Part 4": "Fiction, Short Story",
    "Bill's Paper Chase, Lady of the Barge and Others, Part 3": "Fiction, Short Story",
    "Bob's Redemption, Captains All, Book 5": "Fiction, Short Story",
    "Captain Rogers, The Lady of the Barge and Others, Part 7": "Fiction, Short Story",
    "Captains All": "Fiction, Short Stories Collection",
    "Cupboard Love, The Lady of the Barge and Others, Part 5": "Fiction, Short Story",
    "Deep Waters": "Fiction, Short Stories Collection",
    "Deserted, Sailor's Knots, Part 1": "Fiction, Short Story",
    "Dialstone Lane": "Fiction, Novel",
    "Dirty Work, Deep Waters, Part 11": "Fiction, Short Story",
    "Dixon's Return, Odd Craft, Part 10": "Fiction, Short Story",
    "Dual Control, Ship's Company, Part 8": "Fiction, Short Story",
    "Establishing Relations, Odd Craft, Part 7": "Fiction, Short Story",
    "A Far Country, Complete": "Fiction, Novel",
    "A Traveller in War-Time": "Travel/War, Non-Fiction",
    "An Essay On The American Contribution And The Democratic Idea": "Essays/Politics, Non-Fiction",
    "Betty Zane": "Historical Fiction, Romance Novel",
    "Romeo and Juliet": "Drama/Play, Tragedy",
    "Pride and Prejudice": "Fiction, Romance Novel",
    "The Complete Works of William Shakespeare": "Drama/Plays Collection, Poetry",
    "Middlemarch": "Fiction, Novel",
    "Little Women; Or, Meg, Jo, Beth, and Amy": "Fiction, Coming-of-Age Novel",
    "The Blue Castle: a novel": "Fiction, Romance Novel",
    "How to Observe: Morals and Manners": "Sociology/Etiquette, Non-Fiction",
    "The Enchanted April": "Fiction, Novel",
    "Cranford": "Fiction, Social Novel",
    "The Expedition of Humphry Clinker": "Fiction, Epistolary Novel",
    "The Adventures of Ferdinand Count Fathom — Complete": "Fiction, Picaresque Novel",
    "History of Tom Jones, a Foundling": "Fiction, Picaresque Novel",
    "The Picture of Dorian Gray": "Fiction, Gothic Novel",
    "Leviathan": "Philosophy/Politics, Non-Fiction",
    "Society in America, Volume 1 (of 2)": "Sociology/Travel, Non-Fiction",
    "The Strange Case of Dr. Jekyll and Mr. Hyde": "Horror Fiction, Novella",
    "Metamorphosis": "Fiction, Absurdist Novella",
    "Right Ho, Jeeves": "Fiction, Humorous Novel",
    "The Brothers Karamazov": "Fiction, Philosophical Novel",
    "Project Gutenberg Compilation of Short Stories by Chekhov": "Fiction, Short Stories Collection",
    "The Scarlet Letter": "Fiction, Historical Novel",
    "Thus Spake Zarathustra: A Book for All and None": "Philosophy, Non-Fiction",
    "The Iliad": "Poetry, Epic",
    "The Count of Monte Cristo": "Adventure Fiction, Novel",
    "Grimms' Fairy Tales": "Children's Fiction, Fairy Tales Collection",
    "War and Peace": "Historical Fiction, Novel",
    "Simple Sabotage Field Manual": "Non-Fiction, Military/Instructional",
    "A Doll's House : a play": "Drama/Play",
    "A Modest Proposal: For preventing the children of poor people in Ireland, from being a burden on their parents or country, and for making them beneficial to the publick": "Satire/Essays, Non-Fiction",
    "Rip Van Winkle": "Fiction, Short Story/Folklore",
    "The Prince": "Philosophy/Politics, Non-Fiction",
    "The Importance of Being Earnest: A Trivial Comedy for Serious People": "Drama/Play, Comedy",
    "The Adventures of Tom Sawyer, Complete": "Adventure Fiction, Youth Novel",
    "The divine comedy": "Poetry, Epic",
    "On Liberty": "Philosophy/Politics, Non-Fiction",
    "We": "Science Fiction, Dystopian Novel",
    "Les Bijoux Indiscrets, or, The Indiscreet Toys": "Fiction, Erotic Satire Novel",
    "R.U.R. (Rossum's Universal Robots): A Fantastic Melodrama in Three Acts and an Epilogue": "Drama/Play, Science Fiction",
    "St. John's College, Cambridge": "History/Education, Non-Fiction",
    "The Republic": "Philosophy, Non-Fiction",
    "Great Expectations": "Fiction, Bildungsroman Novel",
    "Beyond Good and Evil": "Philosophy, Non-Fiction",
    "The Kama Sutra of Vatsyayana: Translated From the Sanscrit in Seven Parts With Preface, Introduction and Concluding Remarks": "Philosophy/Erotica, Non-Fiction",
    "A passage to India": "Fiction, Novel",
    "The Yellow Wallpaper": "Fiction, Short Story/Horror",
    "The Wars of Religion in France 1559-1576: The Huguenots, Catherine de Medici and Philip II": "History, Non-Fiction",
    "Society as I Have Found It": "Autobiography/Social Commentary, Non-Fiction",
    "The Odyssey: Rendered into English prose for the use of those who cannot read the original": "Poetry/Epic, Translation",
    "The lesser Key of Solomon, Goetia, the book of evil spirits : $b contains two hundred diagrams and seals for invocation and convocation of spirits, necromancy, witchcraft and black art": "Occult/Religion, Non-Fiction",
    "More Letters of Charles Darwin — Volume 1: A Record of His Work in a Series of Hitherto Unpublished Letters": "Biography/Letters, Non-Fiction",
    "Chambers's Twentieth Century Dictionary (part 1 of 4: A-D)": "Reference/Dictionary, Non-Fiction",
    "The Romance of Lust: A classic Victorian erotic novel": "Fiction, Erotic Novel",
    "Auguste Comte and Positivism": "Philosophy, Non-Fiction",
    "The Comic Poems of Thomas Hood: A New and Complete Edition": "Poetry/Humor Collection",
    "The Modern Traveller": "Poetry/Satire",
    "Narrative of the Life of Frederick Douglass, an American Slave": "Autobiography, Non-Fiction",
    "Peter Pan : $b [Peter and Wendy]": "Children's Fiction, Fantasy Novel",
    "Justice": "Philosophy/Non-Fiction",
    "Meditations": "Philosophy/Autobiography, Non-Fiction",
    "Don't Marry; or, Advice on How, When and Who to Marry": "Self-Help/Advice, Non-Fiction",
    "Manual of Classical Erotology (De figuris Veneris)": "Erotica/Philosophy, Non-Fiction",
    "Cookery and Dining in Imperial Rome": "Cookbook/History, Non-Fiction",
    "The Mahabharata of Krishna-Dwaipayana Vyasa, Volume 1: Books 1, 2 and 3": "Poetry/Epic, Mythology/Religion",
    "The Thousand and One Nights, Vol. I.: Commonly Called the Arabian Nights' Entertainments": "Folklore/Fiction, Short Stories Collection",
    "On the Origin of Species By Means of Natural Selection: Or, the Preservation of Favoured Races in the Struggle for Life": "Science/Biology, Non-Fiction",
    "The Wonderful Wizard of Oz": "Children's Fiction, Fantasy Novel",
    "The Convent School; Or, Early Experiences of a Young Flagellant": "Fiction, Erotic Novel",
    "The Legend of Sleepy Hollow": "Horror Fiction, Short Story",
    "Confidential Chats with Girls": "Self-Help/Advice, Non-Fiction",
    "The King James Version of the Bible": "Religion/Scripture, Non-Fiction",
    "Pascal's Pensées": "Philosophy/Religion, Non-Fiction Essays",
    "The Twilight of the Idols; or, How to Philosophize with the Hammer. The Antichrist: Complete Works, Volume Sixteen": "Philosophy, Non-Fiction",
    "Shakespeare's family": "Biography/History, Non-Fiction",
    "Heart of Darkness": "Fiction, Novella/Adventure",
    "The Philippine Islands, 1493-1898, Volume 33, 1519-1522: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century": "History/Exploration, Non-Fiction",
    "Le Morte d'Arthur: Volume 1": "Fiction, Arthurian Legend/Romance",
    "An Essay on Laughter: Its Forms, Its Causes, Its Development and Its Value": "Philosophy/Psychology, Non-Fiction",
    "Carmilla": "Horror Fiction, Vampire Novella",
    "Treasure Island": "Adventure Fiction, Novel",
    "The sexual life of woman in its physiological, pathological and hygienic aspects": "Science/Medicine, Non-Fiction",
    "The Ethics of Aristotle": "Philosophy, Non-Fiction",
    "The History of the Peloponnesian War": "History, Non-Fiction",
    "The King in Yellow": "Horror Fiction, Short Stories",
    "The Waterloo Roll Call: With Biographical Notes and Anecdotes": "History/Military, Non-Fiction",
    "A Journey to the Centre of the Earth": "Science Fiction, Adventure Novel",
    "Candide": "Fiction, Satirical Novel",
    "Sense and Sensibility": "Fiction, Romance Novel",
    "The Innocents Abroad": "Travel/Humor, Non-Fiction",
    "Faust [part 1]. Translated Into English in the Original Metres": "Drama/Poetry, Tragedy",
    "The Time Machine": "Science Fiction, Novel",
    "The Life and Adventures of Robinson Crusoe": "Adventure Fiction, Novel",
    "The Tragical History of Doctor Faustus: From the Quarto of 1604": "Drama/Play, Tragedy",
    "Lady Chatterley's lover": "Fiction, Romance Novel/Erotic",
    "Winnie-the-Pooh": "Children's Fiction, Animal Tales",
    "Don Quixote": "Fiction, Satirical Novel",
    "Democracy in America — Volume 1": "Politics/Sociology, Non-Fiction",
    "Antiquities of the Jews": "History/Religion, Non-Fiction",
    "Harriet Martineau": "Biography, Non-Fiction",
    "The Art of War": "Military/Philosophy, Non-Fiction",
    "The Philippines a Century Hence": "History/Politics, Non-Fiction Essay",
    "A Complete Guide to Heraldry": "History/Reference, Non-Fiction",
    "The Jungle Book": "Children's Fiction, Animal Tales/Short Stories",
    "Twenty Thousand Leagues under the Sea": "Science Fiction, Adventure Novel",
    "Legends of the Bastille": "History, Non-Fiction",
    "The Philosophy of Auguste Comte": "Philosophy, Non-Fiction",
    "A Complete Dictionary of Synonyms and Antonyms: or, Synonyms and Words of Opposite Meaning": "Reference/Dictionary, Non-Fiction",
    "The Decameron of Giovanni Boccaccio": "Fiction, Short Stories Collection",
    "The Genealogy of Morals: The Complete Works, Volume Thirteen, edited by Dr. Oscar Levy.": "Philosophy, Non-Fiction",
    "The History of the Decline and Fall of the Roman Empire: Table of Contents with links in the HTML file to the two Project Gutenberg editions (12 volumes)": "History, Non-Fiction",
    "Aesop's Fables; a new translation": "Children's Fiction, Fables",
    "An Inquiry into the Nature and Causes of the Wealth of Nations": "Economics, Non-Fiction",
    "The Hound of the Baskervilles": "Mystery Fiction, Detective Novel",
    "The Rámáyan of Válmíki, translated into English verse": "Poetry/Epic, Mythology",
    "On War": "Military/Strategy, Non-Fiction",
    "Rizal's own story of his life": "Autobiography, Non-Fiction",
    "Poems by Emily Dickinson, Three Series, Complete": "Poetry Collection",
    "The Elementary Forms of the Religious Life": "Sociology/Religion, Non-Fiction",
    "Complete Original Short Stories of Guy De Maupassant": "Fiction, Short Stories Collection",
    "Goethe's Theory of Colours": "Science/Art, Non-Fiction",
    "Tess of the d'Urbervilles: A Pure Woman": "Fiction, Novel",
    "Three men in a boat (to say nothing of the dog)": "Fiction, Humorous Novel",
    "The Anabasis of Alexander : $b or, The history of the wars and conquests of Alexander the Great": "History/Biography, Non-Fiction",
    "Spenser's The Faerie Queene, Book I": "Poetry, Epic",
    "Struwwelpeter: Merry Stories and Funny Pictures": "Children's Fiction, Illustrated Rhymes",
    "The Story of Beowulf, Translated from Anglo-Saxon into Modern English Prose": "Poetry/Epic, Translation",
    "Notes and Queries, Number 82, May 24, 1851: A Medium of Inter-communication for Literary Men, Artists, Antiquaries, Genealogists, etc.": "Literary Criticism/History, Non-Fiction Journal",
    "The History of Herodotus — Volume 1": "History, Non-Fiction",
    "The Birth of Tragedy; or, Hellenism and Pessimism": "Philosophy/Literary Criticism, Non-Fiction",
    "Ethics": "Philosophy, Non-Fiction",
    "The Marriage of Heaven and Hell": "Poetry/Philosophy",
    "The History of the Standard Oil Company": "History/Business, Non-Fiction",
    "The Sorrows of Young Werther": "Fiction, Epistolary Novel",
    "The Sins of the Cities of the Plain; or, The Recollections of a Mary-Ann: with Short Essays on Sodomy and Tribadism": "Fiction, Erotic Memoir",
    "Around the World in Eighty Days": "Adventure Fiction, Novel",
    "Doctrina Christiana: The first book printed in the Philippines, Manila, 1593.": "Religion/History, Non-Fiction",
    "The Ten Books on Architecture": "Architecture/Science, Non-Fiction",
    "A Dictionary of Cebuano Visayan": "Reference/Dictionary, Non-Fiction",
    "My Life and Loves, Vol. 1 (of 4)": "Autobiography/Erotic, Non-Fiction",
    "Lives of Fair and Gallant Ladies. Vol 1": "Biography/Erotic History, Non-Fiction",
    "A serious proposal to the Ladies, for the advancement of their true and greatest interest (In Two Parts)": "Essays/Feminism, Non-Fiction",
    "The Sign of the Four": "Mystery Fiction, Detective Novel",
    "The Sun Also Rises": "Fiction, Novel",
    "Reminiscences of a Stock Operator": "Autobiography/Business, Non-Fiction",
    "The Beautiful and Damned": "Fiction, Novel",
    "The Lady of the Lake": "Poetry, Narrative Verse",
    "The Enchiridion": "Philosophy, Non-Fiction",
    "The Travels of Marco Polo — Volume 1": "Travel/History, Non-Fiction",
    "Little Brother": "Science Fiction, Novel",
    "The Erotic Motive in Literature": "Literary Criticism/Psychology, Non-Fiction",
    "The Devil's Dictionary": "Humor/Satire, Non-Fiction Reference",
    "Titan: A Romance. v. 1 (of 2)": "Fiction, Romance Novel",
    "Fifteen sonnets of Petrarch": "Poetry Collection",
    "Revelations of Divine Love": "Religion/Mysticism, Non-Fiction",
    "Uncle Tom's Cabin": "Fiction, Anti-Slavery Novel",
    "The City of God, Volume I": "Religion/Philosophy, Non-Fiction",
    "The Mummy! A Tale of the Twenty-Second Century": "Science Fiction, Novel",
    "Vanity Fair": "Fiction, Satirical Novel",
    "The art of money getting : $b or, golden rules for making money": "Self-Help/Business, Non-Fiction",
    "The Phantom of the Opera": "Fiction, Gothic Novel",
    "Just so stories": "Children's Fiction, Short Stories",
    "Philippine Folk Tales": "Folklore/Fiction, Short Stories Collection",
    "Mrs. Dalloway": "Fiction, Modernist Novel",
    "A Dictionary of Slang, Cant, and Vulgar Words: Used at the Present Day in the Streets of London; the Universities of Oxford and Cambridge; the Houses of Parliament; the Dens of St. Giles; and the Palaces of St. James.": "Reference/Dictionary, Non-Fiction",
    "Princes and Poisoners: Studies of the Court of Louis XIV": "History/Biography, Non-Fiction",
    "A History of Greek Economic Thought": "History/Economics, Non-Fiction",
    "Plays": "Drama/Plays Collection",
    "Phaedrus": "Philosophy, Non-Fiction Dialogue",
    "The war drama of the Eagles : $b Napoleon's standard-bearers on the battlefield in victory and defeat from Austerlitz to Waterloo, a record of hard fighting, heroism, and adventure": "History/Military, Non-Fiction",
    "Omens and Superstitions of Southern India": "Folklore/Anthropology, Non-Fiction",
    "The Turn of the Screw": "Horror Fiction, Novella",
    "Discourse on the Method of Rightly Conducting One's Reason and of Seeking Truth in the Sciences": "Philosophy/Science, Non-Fiction",
    "Golden Days for Boys and Girls, Vol. XII, Jan. 3, 1891": "Children's Fiction, Magazine/Short Stories",
    "Heath's Modern Language Series: The Spanish American Reader": "Language/Education, Non-Fiction",
    "Amusements in Mathematics": "Mathematics/Puzzles, Non-Fiction",
    "Degeneration": "Sociology/Cultural Criticism, Non-Fiction",
    "A short history of the printing press and of the improvements in printing machinery from the time of Gutenberg up to the present day": "History/Technology, Non-Fiction",
    "Spoon River Anthology": "Poetry Collection",
    "The Lives of the Twelve Caesars, Complete": "History/Biography, Non-Fiction",
    "Forty-Eighth Annual Report of the Bureau of American Ethnology to the Secretary of the Smithsonian Institution, 1930-1931, Government Printing Office, Washington, 1933.": "Anthropology/History, Non-Fiction Report",
    "The Philippine Islands, 1493-1898 — Volume 07 of 55: 1588-1591; Explorations by Early Navigators, Descriptions of the Islands and Their Peoples, Their History and Records of the Catholic Missions, as Related in Contemporaneous Books and Manuscripts, Showing the Political, Economic, Commercial and Religious Conditions of Those Islands from Their Earliest Relations with European Nations to the Close of the Nineteenth Century": "History/Exploration, Non-Fiction",
    "Reliques of Ancient English Poetry, Volume 2 (of 3): Consisting of Old Heroic Ballads, Songs and Other Pieces of Our Earlier Poets Together With Some Few of Later Date": "Poetry/Folklore Collection",
    "The Complete Herbal: To which is now added, upwards of one hundred additional herbs, with a display of their medicinal and occult qualities physically applied to the cure of all disorders incident to mankind: to which are now first annexed, the English physician enlarged, and key to Physic.": "Medicine/Herbalism, Non-Fiction",
    "2 B R 0 2 B": "Science Fiction, Short Story",
    "Memoirs of Extraordinary Popular Delusions and the Madness of Crowds": "History/Psychology, Non-Fiction",
    "The Pursuit of God": "Religion/Spirituality, Non-Fiction",
    "All About Coffee": "History/Food, Non-Fiction",
    "Plutarch's Morals": "Philosophy/Ethics, Non-Fiction",
    "Helps to Latin Translation at Sight": "Education/Language, Non-Fiction",
    "A Pickle for the Knowing Ones": "Humor/Satire, Non-Fiction",
    "The reader's guide to the Encyclopaedia Britannica : $b A handbook containing sixty-six courses of systematic study or occasional reading": "Reference/Education, Non-Fiction",
    "Gorgias": "Philosophy, Non-Fiction Dialogue",
    "The Diary of Samuel Pepys — Complete": "Autobiography/Diary, Non-Fiction",
    "Brief Lives, Vol. 1": "Biography Collection, Non-Fiction",
    "The Red Record: Tabulated Statistics and Alleged Causes of Lynching in the United States": "History/Social Justice, Non-Fiction",
    "Highways and Byways in Cambridge and Ely": "Travel/History, Non-Fiction",
    "The Awakening, and Selected Short Stories": "Fiction, Short Stories/Novel",
    "Poems, with The Ballad of Reading Gaol": "Poetry Collection",
    "International Law. A Treatise. Volume 1 (of 2): Peace. Second Edition": "Law/Politics, Non-Fiction",
    "Scenes in the Life of Harriet Tubman": "Biography/History, Non-Fiction",
    "Death in Venice": "Fiction, Novella",
    "Southern Horrors: Lynch Law in All Its Phases": "History/Social Justice, Non-Fiction",
    "The Book of the Thousand Nights and a Night — Volume 02 (of 10)": "Folklore/Fiction, Short Stories",
    "The Song Celestial; Or, Bhagavad-Gîtâ (from the Mahâbhârata): Being a discourse between Arjuna, Prince of India, and the Supreme Being under the form of Krishna": "Poetry/Religion, Translation",
    "Anabasis": "History/Military, Non-Fiction",
    "Narrative of the Captivity and Restoration of Mrs. Mary Rowlandson": "Autobiography/History, Non-Fiction",
    "Narrative and Critical History of America, Vol. 2 (of 8): Spanish Explorations and Settlements in America from the Fifteenth to the Seventeenth Century": "History, Non-Fiction",
    "The Slayer of Souls": "Fiction, Occult Novel",
    "Reliques of Ancient English Poetry, Volume 1 (of 3): Consisting of Old Heroic Ballads, Songs and Other Pieces of Our Earlier Poets Together With Some Few of Later Date": "Poetry/Folklore Collection",
    "The Glories of Ireland": "History/Cultural, Non-Fiction",
    'Encyclopaedia Britannica, 11th Edition, "Cincinnatus" to "Cleruchy": Volume 6, Slice 4': "Reference/Encyclopedia, Non-Fiction",
    "The Nursery Rhymes of England": "Children's Poetry, Collection",
    "Swann's Way": "Fiction, Novel",
    "The Aeneid": "Poetry, Epic",
    "The History of Rome, Books 01 to 08": "History, Non-Fiction",
    "Staffordshire pottery and its history": "History/Art, Non-Fiction",
    "A History of Witchcraft in England from 1558 to 1718": "History/Occult, Non-Fiction",
    "Bradford's History of 'Plimoth Plantation': From the Original Manuscript. With a Report of the Proceedings Incident to the Return of the Manuscript to Massachusetts": "History, Non-Fiction",
    "The Flowers of Evil": "Poetry Collection",
    "Napoleon's Letters to Josephine, 1796-1812: For the First Time Collected and Translated, with Notes Social, Historical, and Chronological, from Contemporary Sources": "Biography/Letters, Non-Fiction",
    "The Methods of Ethics": "Philosophy/Ethics, Non-Fiction",
    "A Popular History of Astronomy During the Nineteenth Century: Fourth Edition": "Science/History, Non-Fiction",
    "Chess Fundamentals": "Games/Strategy, Non-Fiction",
    "Black Beauty": "Children's Fiction, Animal Tale",
    "Democracy in America — Volume 2": "Politics/Sociology, Non-Fiction",
    "The Divine Comedy of Dante Alighieri: The Inferno": "Poetry, Epic",
    "The House of the Wolfings: A Tale of the House of the Wolfings and All the Kindreds of the Mark Written in Prose and in Verse": "Fiction, Fantasy Novel",
    "In Praise of Folly: Illustrated with Many Curious Cuts": "Satire/Philosophy, Non-Fiction",
    "The Ivory Gate, a new edition": "Fiction, Novel",
    "Eugene Oneguine [Onegin]: A Romance of Russian Life in Verse": "Poetry, Novel in Verse",
    "George Cruikshank": "Biography/Art, Non-Fiction",
    "Apology": "Philosophy, Non-Fiction Dialogue",
    "Encyclopaedia Britannica, 11th Edition, \"Gichtel, Johann\" to \"Glory\": Volume 12, Slice 1": "Reference/Encyclopedia, Non-Fiction",
    "Bartleby, the Scrivener: A Story of Wall-Street": "Fiction, Short Story",
    "The Satires of Juvenal, Persius, Sulpicia, and Lucilius: Literally translated into English prose, with notes, chronological tables, arguments, &c.": "Poetry/Satire, Translation",
    "The girl from Scotland Yard": "Mystery Fiction, Novel",
    "Pen-portraits of literary women : $b by themselves and others, Volume 2 (of 2)": "Biography/Literary Criticism, Non-Fiction",
    "On Heroes, Hero-Worship, and the Heroic in History": "Essays/Philosophy, Non-Fiction",
    "De Officiis": "Philosophy/Ethics, Non-Fiction",
    "The Life of Ludwig van Beethoven, Volume I": "Biography, Non-Fiction",
    "Anthem": "Science Fiction, Dystopian Novella",
    "The Ancient Stone Implements, Weapons and Ornaments, of Great Britain: Second Edition, Revised": "Archaeology/History, Non-Fiction",
    "Demonology and Devil-lore": "Folklore/Occult, Non-Fiction",
    "Some Jewish Witnesses For Christ": "Religion/Biography, Non-Fiction",
    "Up from Slavery: An Autobiography": "Autobiography, Non-Fiction",
    "The Man Who Was Thursday: A Nightmare": "Fiction, Philosophical Novel",
    "Poems on various subjects, religious and moral": "Poetry Collection",
    "History of the Royal Regiment of Artillery, Vol. 1: Compiled from the Original Records": "History/Military, Non-Fiction",
    "The Canterbury Tales, and Other Poems": "Poetry Collection",
    "Twelve Years a Slave: Narrative of Solomon Northup, a Citizen of New-York, Kidnapped in Washington City in 1841, and Rescued in 1853, from a Cotton Plantation near the Red River in Louisiana": "Autobiography/History, Non-Fiction",
    "Curious Facts in the History of Insects; Including Spiders and Scorpions.: A Complete Collection of the Legends, Superstitions, Beliefs, and Ominous Signs Connected with Insects; Together with Their Uses in Medicine, Art, and as Food; and a Summary of Their Remarkable Injuries and Appearances.": "Science/Folklore, Non-Fiction",
    "The Monkey's Paw: The Lady of the Barge and Others, Part 2.": "Horror Fiction, Short Story",
    "The Online World": "Technology/Reference, Non-Fiction",
    "Principles of Political Economy: Abridged with Critical, Bibliographical, and Explanatory Notes, and a Sketch of the History of Political Economy": "Economics, Non-Fiction",
    "The Vampyre; a Tale": "Horror Fiction, Short Story",
    "Portuguese literature": "Literary Criticism/History, Non-Fiction",
    "My Studio Neighbors": "Essays/Nature, Non-Fiction",
    "A Short History of Freethought Ancient and Modern, Volume 1 of 2: Third edition, Revised and Expanded, in two volumes": "History/Philosophy, Non-Fiction",
    "The Satyricon — Complete": "Fiction, Satirical Novel",
    "The Social Cancer: A Complete English Version of Noli Me Tangere": "Fiction, Social Novel",
    "The Trial": "Fiction, Absurdist Novel",
    "New Latin Grammar": "Education/Language, Non-Fiction",
    "The Extraordinary Adventures of Arsène Lupin, Gentleman-Burglar": "Mystery Fiction, Short Stories",
    "The Outline of History: Being a Plain History of Life and Mankind": "History, Non-Fiction",
    "An Enquiry Concerning Human Understanding": "Philosophy, Non-Fiction",
    "The Works of Edgar Allan Poe, The Raven Edition: Table Of Contents And Index Of The Five Volumes": "Fiction/Poetry Collection, Horror/Mystery",
    "Medea of Euripides": "Drama/Play, Tragedy",
    "Bacon's Essays, and Wisdom of the Ancients": "Essays/Philosophy, Non-Fiction",
    "Ivanhoe: A Romance": "Historical Fiction, Novel",
    "The Germany and the Agricola of Tacitus": "History, Non-Fiction",
    "The Medicinal Plants of the Philippines": "Science/Botany, Non-Fiction",
    "The Wit and Humor of America, Volume X (of X)": "Humor/Fiction Collection",
    "The Anatomy of Melancholy": "Philosophy/Medicine, Non-Fiction",
    "The Marching Morons": "Science Fiction, Short Story",
    "The Mediterranean : $b Seaports and sea routes including Madeira, the Canary Islands, the coast of Morocco, Algeria, and Tunisia; handbook for travellers": "Travel/Guide, Non-Fiction",
    "Ancient Pagan and Modern Christian Symbolism: With an Essay on Baal Worship, on the Assyrian Sacred \"Grove,\" and Other Allied Symbols": "Religion/History, Non-Fiction",
    "Historic Paris": "History/Travel, Non-Fiction",
    "Mesopotamian Archaeology: An introduction to the archaeology of Mesopotamia and Assyria": "Archaeology/History, Non-Fiction",
    "A Concise Anglo-Saxon Dictionary: For the Use of Students": "Reference/Dictionary, Non-Fiction",
    "The Survey of London": "History/Architecture, Non-Fiction",
    "North and South": "Fiction, Social Novel",
    "Susan B. Anthony: Rebel, Crusader, Humanitarian": "Biography, Non-Fiction",
    "Roget's Thesaurus": "Reference/Dictionary, Non-Fiction",
    "The World as Will and Idea (Vol. 1 of 3)": "Philosophy, Non-Fiction",
    "Showell's Dictionary of Birmingham: A History and Guide, Arranged Alphabetically": "History/Guide, Non-Fiction",
    "The Philippine Islands, 1493-1898—Volume 34 of 55, 1519-1522; 1280-1605: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century": "History/Exploration, Non-Fiction",
    "De Re Metallica, Translated from the First Latin Edition of 1556": "Science/Mining, Non-Fiction",
    "Devotions Upon Emergent Occasions; Together with Death's Duel": "Religion/Meditation, Non-Fiction",
    "How France Built Her Cathedrals: A Study in the Twelfth and Thirteenth Centuries": "Architecture/History, Non-Fiction",
    "Shakespearean Tragedy: Lectures on Hamlet, Othello, King Lear, Macbeth": "Literary Criticism, Non-Fiction",
    "The Essays of Arthur Schopenhauer: the Wisdom of Life": "Philosophy/Essays, Non-Fiction",
    "A Comprehensive History of Norwich": "History, Non-Fiction",
    "The Younger Edda; Also called Snorre's Edda, or The Prose Edda": "Mythology/Folklore, Non-Fiction",
    "The Progress of Invention in the Nineteenth Century.": "History/Technology, Non-Fiction",
    "The Decoration of Houses": "Architecture/Design, Non-Fiction",
    "Modern Spanish Lyrics": "Poetry Collection",
    "Webster's Unabridged Dictionary": "Reference/Dictionary, Non-Fiction",
    "Facts and Speculations on the Origin and History of Playing Cards": "History/Games, Non-Fiction",
    "The Geography of Strabo, Volume 3 (of 3): Literally Translated, with Notes": "Geography/History, Non-Fiction",
    "The Adventures of Pinocchio": "Children's Fiction, Fantasy Novel",
    "Man and Superman: A Comedy and a Philosophy": "Drama/Play, Comedy",
    "Stories from Tagore": "Fiction, Short Stories",
    "Herland": "Science Fiction, Utopian Novel",
    "The Practice and Science of Drawing": "Art/Instruction, Non-Fiction",
    "Etiquette": "Self-Help/Social, Non-Fiction",
    "Fifteen Thousand Useful Phrases": "Reference/Language, Non-Fiction",
    "Friday, the Thirteenth: A Novel": "Fiction, Novel",
    "The Golden Bough: A Study in Comparative Religion (Vol. 1 of 2)": "Anthropology/Religion, Non-Fiction",
    "The Natural History of Pliny, Volume 1 (of 6)": "Science/Natural History, Non-Fiction",
    "The Garden Party, and Other Stories": "Fiction, Short Stories",
    "The Ballad of the White Horse": "Poetry, Narrative Verse",
    "The Life of Florence Nightingale, vol. 2 of 2": "Biography, Non-Fiction",
    "Brief Lives, Vol. 2": "Biography Collection, Non-Fiction",
    "The Pilgrim's Progress by John Bunyan Every Child Can Read": "Religion/Allegory, Fiction/Children's Adaptation",
    "Modern English biography, volume 2 (of 4), I-Q": "Biography/Reference, Non-Fiction",
    "Utopia": "Fiction, Utopian Satire",
    "Shell-shock and other neuropsychiatric problems : $b Presented in five hundred and eighty-nine case histories from the war literature, 1914-1918": "Medicine/Psychology, Non-Fiction",
    "Geographical etymology: a dictionary of place-names giving their derivations": "Reference/Geography, Non-Fiction",
    "A Treatise on Painting": "Art/Instruction, Non-Fiction",
    "Autobiography of a Yogi": "Autobiography/Spirituality, Non-Fiction",
    "I, Mars": "Science Fiction, Short Story",
    "Moby-Dick; or, The Whale": "Adventure Fiction, Novel",
    "A farewell to arms": "Fiction, War Novel",
    "Bible Myths and their Parallels in other Religions: Being a Comparison of the Old and New Testament Myths and Miracles with those of the Heathen Nations of Antiquity Considering also their Origin and Meaning": "Religion/Mythology, Non-Fiction",
    "The Expositor's Bible: The Book of the Twelve Prophets, Vol. 1: Commonly Called the Minor": "Religion/Commentary, Non-Fiction",
    "Haunted London": "History/Occult, Non-Fiction",
    "One Thousand Ways to Make Money": "Self-Help/Business, Non-Fiction",
    "A Defence of Poesie and Poems": "Literary Criticism/Poetry, Non-Fiction",
    "Kant's Critique of Judgement": "Philosophy, Non-Fiction",
    "The Castle of Otranto": "Fiction, Gothic Novel",
    "The History of Cuba, vol. 4": "History, Non-Fiction",
    "The Roll of Honour, Volume 1 : $b A biographical record of all members of His Majesty's naval and military forces who have fallen in the war": "History/Military, Non-Fiction",
    "A vocabulary of criminal slang : $b with some examples of common usages": "Reference/Language, Non-Fiction",
    "The Influence of Sea Power Upon History, 1660-1783": "History/Military, Non-Fiction",
    "Three hundred Aesop’s fables: Translated by George Fyler Townsend": "Children's Fiction, Fables",
    "The Handbook to English Heraldry": "History/Reference, Non-Fiction",
    "A general history of the pyrates : $b from their first rise and settlement in the island of Providence, to the present time": "History/Adventure, Non-Fiction",
    "Principia Ethica": "Philosophy/Ethics, Non-Fiction",
    "Letters of Abelard and Heloise: To which is prefix'd a particular account of their lives, amours, and misfortunes": "Biography/Letters, Non-Fiction",
    "The Balkan Wars: 1912-1913: Second Edition": "History/Military, Non-Fiction",
    "A history of the Peninsular War, Vol. 3, Sep. 1809-Dec. 1810 : $b Ocaña, Cadiz, Bussaco, Torres Vedras": "History/Military, Non-Fiction",
    "The Tenniel Illustrations for Carroll's Alice in Wonderland": "Art/Illustration, Non-Fiction",
    "The Life of Lazarillo de Tormes: His Fortunes & Adversities; with a Notice of the Mendoza Family, a Short Life of the Author, Don Diego Hurtado De Mendoza, a Notice of the Work, and Some Remarks on the Character of Lazarillo de Tormes": "Fiction, Picaresque Novel",
    "The History of Orange County New York": "History, Non-Fiction",
    "Gargantua and Pantagruel": "Fiction, Satirical Novel",
    "The Valleys of Tirol: Their traditions and customs and how to visit them": "Travel/Folklore, Non-Fiction",
    "The Nature of Animal Light": "Science/Biology, Non-Fiction",
    "Lineage, Life and Labors of José Rizal, Philippine Patriot": "Biography/History, Non-Fiction",
    "Mr. Honey's Large Business Dictionary (English-German)": "Reference/Dictionary, Non-Fiction",
    "Penrod": "Fiction, Youth Novel",
    "The South of France—East Half": "Travel/Guide, Non-Fiction",
    "The Tribes and Castes of the Central Provinces of India, Volume 3": "Anthropology/History, Non-Fiction",
    "Pragmatism: A New Name for Some Old Ways of Thinking": "Philosophy, Non-Fiction",
    "A Boy's Voyage Round the World": "Travel/Adventure, Non-Fiction",
    "The Golden Bough: A Study of Magic and Religion": "Anthropology/Religion, Non-Fiction",
    "History of the Spanish Conquest of Yucatan and of the Itzas: Papers of the Peabody Museum of American Archaeology and Ethnology, Hard University. Vol. VII.": "History/Exploration, Non-Fiction",
    "A Journal of the First Voyage of Vasco da Gama 1497-1499": "History/Exploration, Non-Fiction",
    "A Treatise of Human Nature": "Philosophy, Non-Fiction",
    "The American Language: A Preliminary Inquiry into the Development of English in the United States": "Linguistics, Non-Fiction",
    "The Alchemist": "Drama/Play, Comedy",
    "The Problem of China": "History/Politics, Non-Fiction",
    "Triplanetary": "Science Fiction, Novel",
    "A Short History of French Literature": "Literary Criticism/History, Non-Fiction",
    "McGuffey's Fourth Eclectic Reader": "Education/Reader, Non-Fiction",
    "The Story of Little Black Sambo, and The Story of Little Black Mingo": "Children's Fiction, Tales",
    "The Haciendas of Mexico: An Artist's Record": "History/Art, Non-Fiction",
    "Plato and the Other Companions of Sokrates, 3rd ed. Volume 3": "Philosophy, Non-Fiction",
    "Journal of a Residence on a Georgian Plantation: 1838-1839": "Autobiography/History, Non-Fiction",
    "The Argonautica": "Poetry, Epic",
    "Voltaire's Philosophical Dictionary": "Philosophy/Essays, Non-Fiction",
    "Surgical Anatomy": "Medicine/Science, Non-Fiction",
    "An anthology of German literature": "Literary Collection/Anthology",
    "A Concise Dictionary of Middle English from A.D. 1150 to 1580": "Reference/Dictionary, Non-Fiction",
    "The Art of Public Speaking": "Self-Help/Communication, Non-Fiction",
    "Folk-Tales of Bengal": "Folklore/Fiction, Short Stories",
    "Mother Goose's Nursery Rhymes: A Collection of Alphabets, Rhymes, Tales, and Jingles": "Children's Poetry, Collection",
    "Women in the fine arts, from the Seventh Century B.C. to the Twentieth Century A.D.": "History/Biography, Non-Fiction",
    "The Woman in White": "Mystery Fiction, Novel",
    "The Sign of the Seven Sins": "Fiction, Novel",
    "The World as Will and Idea (Vol. 3 of 3)": "Philosophy, Non-Fiction",
    "The symbolism of Freemasonry : $b Illustrating and explaining its science and philosophy, its legends, myths and symbols": "Occult/History, Non-Fiction",
    "The Philippines: Past and Present (Volume 2 of 2)": "History, Non-Fiction",
    "Psychological Warfare": "Military/Psychology, Non-Fiction",
    "Plutarch: Lives of the noble Grecians and Romans": "Biography/History, Non-Fiction",
    "The Orbis Pictus": "Education/Children's, Non-Fiction",
    "The origin and development of the moral ideas": "Philosophy/Anthropology, Non-Fiction",
    "Familiar Quotations: A Collection of Passages, Phrases, and Proverbs Traced to; Their Sources in Ancient and Modern Literature": "Reference/Literary, Non-Fiction",
    "The Sailor's Word-Book: An Alphabetical Digest of Nautical Terms, including Some More Especially Military and Scientific, but Useful to Seamen; as well as Archaisms of Early Voyagers, etc.": "Reference/Nautical, Non-Fiction",
    "Heidi": "Children's Fiction, Novel",
    "The Last of the Mohicans; A narrative of 1757": "Historical Fiction, Adventure Novel",
    "The Principles of Psychology, Volume 1 (of 2)": "Psychology, Non-Fiction",
    "The Declaration of Independence of the United States of America": "History/Politics, Non-Fiction",
    "The Well of Saint Clare": "Fiction, Short Stories",
    "The Complete Poetic and Dramatic Works of Robert Browning: Cambridge Edition": "Poetry/Drama Collection",
    "Tragic Sense Of Life": "Philosophy/Religion, Non-Fiction",
    "A Short Biographical Dictionary of English Literature": "Biography/Reference, Non-Fiction",
    "The best short stories of 1920, and the yearbook of the American short story": "Fiction, Short Stories Collection",
    "A Short History of Freethought Ancient and Modern, Volume 2 of 2: Third edition, Revised and Expanded, in two volumes": "History/Philosophy, Non-Fiction",
    "Myths and Legends of China": "Folklore/Mythology, Non-Fiction",
    "The Rise of the Mediaeval Church: And Its Influence on the Civilization of Western Europe from the First to the Thirteenth Century": "History/Religion, Non-Fiction",
    "The battles of the British Army : $b being a popular account of all the principal engagements during the last hundred years": "History/Military, Non-Fiction",
    "Domestic Manners of the Americans": "Travel/Sociology, Non-Fiction",
    "Kidnapped": "Adventure Fiction, Novel",
    "Boy Scouts Handbook: The First Edition, 1911": "Education/Guide, Non-Fiction",
    "Complete Prose Works: Specimen Days and Collect, November Boughs and Goodbye My Fancy": "Essays/Autobiography, Non-Fiction",
    "The Antiquarian Magazine & Bibliographer; Vol. 4, July-Dec 1884": "Literary Criticism/History, Non-Fiction Journal",
    "Democracy and Education: An Introduction to the Philosophy of Education": "Education/Philosophy, Non-Fiction",
    "On the Duties of the Clergy": "Religion/Theology, Non-Fiction",
    "Diccionario Ingles-Español-Tagalog: Con partes de la oracion y pronunciacion figurada": "Reference/Dictionary, Non-Fiction",
    "The Works of Lord Byron. Vol. 4": "Poetry Collection",
    "Nil Darpan; or, The Indigo Planting Mirror, A Drama.: Translated from the Bengali by a Native.": "Drama/Play, Social Commentary",
    "The Interpretation of Dreams": "Psychology, Non-Fiction",
    "The Legends of King Arthur and His Knights": "Fiction, Arthurian Legend",
    "The Journal of Negro History, Volume 8, 1923": "History, Non-Fiction Journal",
    "The House of Mirth": "Fiction, Social Novel",
    "The Gay Cockade": "Fiction, Short Stories",
    "Politics: A Treatise on Government": "Philosophy/Politics, Non-Fiction",
    "Letters on the equality of the sexes, and the condition of woman": "Essays/Feminism, Non-Fiction",
    "Letters to His Son, Complete: On the Fine Art of Becoming a Man of the World and a Gentleman": "Essays/Advice, Non-Fiction",
    "The Danish History, Books I-IX": "History/Mythology, Non-Fiction",
    "The Wonders of the Invisible World: Being an Account of the Tryals of Several Witches Lately Executed in New-England, to which is added A Farther Account of the Tryals of the New-England Witches": "History/Occult, Non-Fiction",
    "Dio's Rome, Volume 1: An Historical Narrative Originally Composed in Greek during the Reigns of Septimius Severus, Geta and Caracalla, Macrinus, Elagabalus and Alexander Severus: and Now Presented in English Form": "History, Non-Fiction",
    "The history of magic : $b including a clear and precise exposition of its procedure, its rites and its mysteries": "Occult/History, Non-Fiction",
    "A Literary History of the English People, from the Origins to the Renaissance": "Literary Criticism/History, Non-Fiction",
    "St. Benedict's Rule for Monasteries": "Religion/Guide, Non-Fiction",
    "John F. Kennedy's Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Pollyanna": "Children's Fiction, Novel",
    "The Hindu-Arabic Numerals": "History/Mathematics, Non-Fiction",
    "The Oriental Rug: A Monograph on Eastern Rugs and Carpets, Saddle-Bags, Mats & Pillows, with a Consideration of Kinds and Classes, Types, Borders, Figures, Dyes, Symbols, etc. Together with Some Practical Advice to Collectors.": "Art/History, Non-Fiction",
    "How to sing [Meine Gesangskunst]": "Music/Instruction, Non-Fiction",
    "Strange Stories from a Chinese Studio (Volumes 1 and 2)": "Folklore/Fiction, Short Stories",
    "The Meditations of the Emperor Marcus Aurelius Antoninus: A new rendering based on the Foulis translation of 1742": "Philosophy, Non-Fiction",
    "The Sketch-Book of Geoffrey Crayon": "Fiction, Essays/Short Stories",
    "On the Nature of Things": "Philosophy/Poetry",
    "The Every Day Book of History and Chronology: Embracing the Anniversaries of Memorable Persons and Events in Every Period and State of the World, from the Creation to the Present Time": "History/Reference, Non-Fiction",
    "On the Sublime": "Literary Criticism, Non-Fiction",
    "Old and New London, Volume I: A Narrative of Its History, Its People, and Its Places": "History/Travel, Non-Fiction",
    "The Churches and Modern Thought: An inquiry into the grounds of unbelief and an appeal for candour": "Religion/Philosophy, Non-Fiction",
    "The Nibelungenlied: Translated into Rhymed English Verse in the Metre of the Original": "Poetry, Epic",
    "The Instruction of Ptah-Hotep and the Instruction of Ke'Gemni: The Oldest Books in the World": "Philosophy/Wisdom Literature, Non-Fiction",
    "Marmion: A Tale Of Flodden Field": "Poetry, Narrative Verse",
    "Travels in the Central Parts of Indo-China (Siam), Cambodia, and Laos (Vol. 1 of 2): During the Years 1858, 1859, and 1860": "Travel/History, Non-Fiction",
    "Every Day Life in the Massachusetts Bay Colony": "History, Non-Fiction",
    "Memoirs of General William T. Sherman — Complete": "Autobiography/Military, Non-Fiction",
    "History of the Reformation in the Sixteenth Century, Volume 1": "History/Religion, Non-Fiction",
    "The Bible, King James Version, Complete Contents": "Religion/Scripture, Non-Fiction",
    "The Frontier in American History": "History, Non-Fiction",
    "The new science of space speech": "Science/Non-Fiction, Speculative",
    "The Cook's Decameron: A Study in Taste, Containing over Two Hundred Recipes for Italian Dishes": "Cookbook, Non-Fiction",
    "A Brief Account of the Destruction of the Indies: Or, a faithful NARRATIVE OF THE Horrid and Unexampled Massacres, Butcheries, and all manner of Cruelties, that Hell and Malice could invent, committed by the Popish Spanish Party on the inhabitants of West-India, TOGETHER With the Devastations of several Kingdoms in America by Fire and Sword, for the space of Forty and Two Years, from the time of its first Discovery by them.": "History, Non-Fiction",
    "The Seven Lamps of Architecture": "Architecture/Art Criticism, Non-Fiction",
    "Familiar Letters: The Writings of Henry David Thoreau, Volume 06 (of 20)": "Letters/Essays, Non-Fiction",
    "Types of News Writing": "Journalism/Education, Non-Fiction",
    "The Duchess of Malfi": "Drama/Play, Tragedy",
    "Castes and Tribes of Southern India. Vol. 7 of 7": "Anthropology/History, Non-Fiction",
    "The French Revolution: A History": "History, Non-Fiction",
    "Life on a Mediaeval Barony: A Picture of a Typical Feudal Community in the Thirteenth Century": "History, Non-Fiction",
    "A Wanderer in Paris": "Travel, Non-Fiction",
    "The Squire's Daughter: Being the First Book in the Chronicles of the Clintons": "Fiction, Novel",
    "The Blue Fairy Book": "Children's Fiction, Fairy Tales Collection",
    "Poems Every Child Should Know: The What-Every-Child-Should-Know-Library": "Poetry Collection, Children's",
    "The Catacombs of Rome, and Their Testimony Relative to Primitive Christianity": "History/Religion, Non-Fiction",
    "Cyrano de Bergerac": "Drama/Play, Comedy",
    "Red Nails": "Fantasy Fiction, Short Story",
    "Famous Modern Ghost Stories": "Horror Fiction, Short Stories Collection",
    "Stage-coach and Tavern Days": "History/Social, Non-Fiction",
    "A Classification and Subject Index for Cataloguing and Arranging the Books and Pamphlets of a Library": "Library Science/Reference, Non-Fiction",
    "The Life of Charles Dickens, Vol. I-III, Complete": "Biography, Non-Fiction",
    "Memoirs of Napoleon Bonaparte — Complete": "Biography/History, Non-Fiction",
    "Fifty Years In The Northwest: With An Introduction And Appendix Containing Reminiscences, Incidents And Notes": "History/Autobiography, Non-Fiction",
    "Laughter: An Essay on the Meaning of the Comic": "Philosophy/Humor, Non-Fiction",
    "The Letters of Wolfgang Amadeus Mozart — Volume 01": "Biography/Letters, Non-Fiction",
    "The Fables of La Fontaine: Translated into English Verse by Walter Thornbury and Illustrated by Gustave Doré": "Poetry/Fables, Translation",
    "The Cumulative Book Review Digest, Volume 1, 1905: Complete in a single alphabet": "Literary Criticism/Reference, Non-Fiction",
    "The History of Sulu": "History, Non-Fiction",
    "Archæology and the Bible": "Archaeology/Religion, Non-Fiction",
    "Miss Eden's Letters": "Biography/Letters, Non-Fiction",
    "Giordano Bruno": "Biography/Philosophy, Non-Fiction",
    "Poirot Investigates": "Mystery Fiction, Short Stories",
    "The Anglo-Saxon Chronicle": "History, Non-Fiction",
    "The Boy Mechanic, Volume 1: 700 Things for Boys to Do": "Education/Crafts, Non-Fiction",
    "Cowboy Songs, and Other Frontier Ballads": "Poetry/Folklore Collection",
    "The New Testament of our Lord and Savior Jesus Christ.: The common English version, corrected by the final committee of the American Bible Union.": "Religion/Scripture, Non-Fiction",
    "Boswell's Life of Johnson: Abridged and edited, with an introduction by Charles Grosvenor Osgood": "Biography, Non-Fiction",
    "The Rising Tide of Color Against White World-Supremacy": "Politics/Sociology, Non-Fiction",
    "Wine, Women, and Song: Mediaeval Latin Students' songs; Now first translated into English verse": "Poetry Collection, Translation",
    "Nobody": "Fiction, Novel",
    "The carnal god": "Fiction, Novel",
    "The Book of Household Management": "Home Economics/Guide, Non-Fiction",
    "The Bronze Age and the Celtic World": "Archaeology/History, Non-Fiction",
    "The Central Eskimo: Sixth Annual Report of the Bureau of Ethnology to the Secretary of the Smithsonian Institution, 1884-1885, Government Printing Office, Washington, 1888, pages 399-670": "Anthropology, Non-Fiction",
    "An Account of Egypt": "History/Travel, Non-Fiction",
    "Argot and slang : $b a new French and English dictionary of the cant words, quaint expressions, slang terms and flash phrases used in the high and low life of old and new Paris": "Reference/Dictionary, Non-Fiction",
    "A Short View of the Immorality, and Profaneness of the English Stage: Together with the Sense of Antiquity on this Argument": "Literary Criticism, Non-Fiction",
    "Pals: Young Australians in Sport and Adventure": "Adventure Fiction, Youth Novel",
    "British Goblins: Welsh Folk-lore, Fairy Mythology, Legends and Traditions": "Folklore, Non-Fiction",
    "Russian Fairy Tales: A Choice Collection of Muscovite Folk-lore": "Folklore/Fiction, Short Stories",
    "The Congo, and Other Poems": "Poetry Collection",
    "Poems of American History": "Poetry Collection",
    "Annals and Antiquities of Rajasthan, v. 1 of 3: or the Central and Western Rajput States of India": "History, Non-Fiction",
    "Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume XIII, Oklahoma Narratives": "History/Oral History, Non-Fiction",
    "The Swastika, the Earliest Known Symbol, and Its Migration: With Observations on the Migration of Certain Industries in Prehistoric Times": "History/Symbolism, Non-Fiction",
    "Scurvy, Past and Present": "Medicine/History, Non-Fiction",
    "The Way of the World": "Drama/Play, Comedy",
    "The Mill on the Floss": "Fiction, Novel",
    "Feudal England: Historical Studies on the Eleventh and Twelfth Centuries": "History, Non-Fiction",
    "The Scarlet Pimpernel": "Adventure Fiction, Novel",
    "Volpone; Or, The Fox": "Drama/Play, Comedy",
    "The Moon and Sixpence": "Fiction, Novel",
    "Woodworking Tools 1600-1900": "History/Technology, Non-Fiction",
    "Le Morte d'Arthur: Volume 2": "Fiction, Arthurian Legend/Romance",
    "The letters of Queen Victoria, vol. 1, 1837-1843 : $b A selection from Her Majesty's correspondence between the years 1837 and 1861.": "Biography/Letters, Non-Fiction",
    "Roster of the Rainbow division (Forty-Second) Major General Wm. A. Mann commanding": "History/Military, Non-Fiction",
    "The Wendigo": "Horror Fiction, Short Story",
    "Church History (Volumes 1-3)": "Religion/History, Non-Fiction",
    "Little Masterpieces of American Wit and Humor, Volume I": "Humor/Fiction Collection",
    "Man a Machine": "Philosophy/Science, Non-Fiction",
    "Domesday Book and Beyond: Three Essays in the Early History of England": "History, Non-Fiction",
    "Myths and Legends of Our Own Land — Complete": "Folklore/Fiction, Short Stories",
    "Franklin's Way to Wealth; or, 'Poor Richard Improved'": "Self-Help/Economics, Non-Fiction",
    "Rubáiyát of Omar Khayyam, Rendered into English Verse": "Poetry Collection",
    "London and the Kingdom - Volume 2: A History Derived Mainly from the Archives at Guildhall in the Custody of the Corporation of the City of London.": "History, Non-Fiction",
    "The Book of the Thousand Nights and a Night — Volume 10 (of 10)": "Folklore/Fiction, Short Stories",
    "The Red and the Black: A Chronicle of 1830": "Fiction, Novel",
    "The Conflict of Religions in the Early Roman Empire": "History/Religion, Non-Fiction",
    "Prayers of the Early Church": "Religion/Prayer, Non-Fiction",
    "The Angel in the House": "Poetry, Narrative Verse",
    "The History of the First West India Regiment": "History/Military, Non-Fiction",
    "British Flags: Their Early History, and Their Development at Sea: With an Account of the Origin of the Flag as a National Device": "History, Non-Fiction",
    "The Friars in the Philippines": "History/Religion, Non-Fiction",
    "Anarchism": "Politics/Philosophy, Non-Fiction",
    "Letters of Pliny": "Biography/Letters, Non-Fiction",
    "The History of Signboards, from the Earliest times to the Present Day": "History/Cultural, Non-Fiction",
    "The Ego and His Own": "Philosophy/Anarchism, Non-Fiction",
    "Fairy and Folk Tales of the Irish Peasantry": "Folklore/Fiction, Short Stories",
    "Ars Amatoria; or, The Art Of Love: Literally Translated into English Prose, with Copious Notes": "Poetry/Erotica, Translation",
    "The Amenities of Book-Collecting and Kindred Affections": "Essays/Literary, Non-Fiction",
    "Memoirs of Louis XIV and His Court and of the Regency — Complete": "Autobiography/History, Non-Fiction",
    "The Letters of Amerigo Vespucci, and Other Documents Illustrative of His Career": "History/Letters, Non-Fiction",
    "The Art of Perfumery, and Methods of Obtaining the Odors of Plants: With Instructions for the Manufacture of Perfumes for the Handkerchief, Scented Powders, Odorous Vinegars, Dentifrices, Pomatums, Cosmetics, Perfumed Soap, Etc., to which is Added an Appendix on Preparing Artificial Fruit-Essences, Etc.": "Science/Instruction, Non-Fiction",
    "Frederick Chopin, as a Man and Musician — Complete": "Biography, Non-Fiction",
    "Crimes of Preachers in the United States and Canada": "History/Social, Non-Fiction",
    "Principles of Political Economy, Vol. 2": "Economics, Non-Fiction",
    "The Russian Army and the Japanese War, Vol. 2 (of 2): Being Historical and Critical Comments on the Military Policy and Power of Russia and on the Campaign in the Far East": "History/Military, Non-Fiction",
    "An Essay on Man; Moral Essays and Satires": "Poetry, Philosophical Verse",
    "The Jargon File, Version 4.2.2, 20 Aug 2000": "Reference/Technology, Non-Fiction",
    "Myths of the Norsemen: From the Eddas and Sagas": "Mythology/Folklore, Non-Fiction",
    "West African Folk-Tales": "Folklore/Fiction, Short Stories",
    "Historic Jamaica": "History, Non-Fiction",
    "The Sufistic Quatrains of Omar Khayyam": "Poetry Collection",
    "History, Manners, and Customs of the Indian Nations Who Once Inhabited Pennsylvania and the Neighbouring States.": "Anthropology/History, Non-Fiction",
    "The Devil is an Ass": "Drama/Play, Comedy",
    "Self Help; with Illustrations of Conduct and Perseverance": "Self-Help, Non-Fiction",
    "The Book of Tea": "Philosophy/Cultural, Non-Fiction",
    "Life of Harriet Beecher Stowe: Compiled From Her Letters and Journals by Her Son Charles Edward Stowe": "Biography, Non-Fiction",
    "The Letters of Cassiodorus: Being a Condensed Translation of the Variae Epistolae of Magnus Aurelius Cassiodorus Senator": "History/Letters, Non-Fiction",
    "Humour, Wit, & Satire of the Seventeenth Century": "Humor/Literary Collection",
    "Life's Handicap: Being Stories of Mine Own People": "Fiction, Short Stories",
    "Young's Night Thoughts: With Life, Critical Dissertation and Explanatory Notes": "Poetry, Philosophical Verse",
    "The house at Pooh Corner": "Children's Fiction, Animal Tales",
    "The Gentleman: A Romance of the Sea": "Fiction, Adventure Novel",
    "The Bābur-nāma in English (Memoirs of Bābur)": "Autobiography/History, Non-Fiction",
    "Cyropaedia: The Education of Cyrus": "History/Philosophy, Non-Fiction",
    "The English Lakes": "Travel/Nature, Non-Fiction",
    "The Spectator, Volumes 1, 2 and 3: With Translations and Index for the Series": "Essays/Literary, Non-Fiction",
    "A Practical Guide to Self-Hypnosis": "Self-Help/Psychology, Non-Fiction",
    "English Fairy Tales": "Children's Fiction, Fairy Tales Collection",
    "An Englishman in Paris: Notes and Recollections": "Autobiography/Travel, Non-Fiction",
    "The Chronicle of the Discovery and Conquest of Guinea. Vol. I": "History/Exploration, Non-Fiction",
    "The New Life (La Vita Nuova)": "Poetry/Autobiography",
    "Hidden Symbolism of Alchemy and the Occult Arts": "Occult/Philosophy, Non-Fiction",
    "Parish Priests and Their People in the Middle Ages in England": "History/Religion, Non-Fiction",
    "A dictionary of American authors": "Reference/Biography, Non-Fiction",
    "Entretiens / Interviews / Entrevistas": "Interviews/Literary, Non-Fiction",
    "Jane's All the World's Aircraft. 1913": "Aviation/History, Non-Fiction",
    "The Katipunan; or, The Rise and Fall of the Filipino Commune": "History/Politics, Non-Fiction",
    "Johann Sebastian Bach: His Life, Art, and Work": "Biography, Non-Fiction",
    "The Geography of Strabo, Volume 2 (of 3): Literally Translated, with Notes": "Geography/History, Non-Fiction",
    "Flatland: A Romance of Many Dimensions": "Science Fiction, Satirical Novel",
    "Tradition, Principally with Reference to Mythology and the Law of Nations": "History/Law, Non-Fiction",
    "The Itinerary of Benjamin of Tudela": "Travel/History, Non-Fiction",
    "The Philippine Islands, 1493-1898; Volume 46, 1721-1739: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century": "History/Exploration, Non-Fiction",
    "Now we are six": "Children's Poetry, Collection",
    "Printers' Marks: A Chapter in the History of Typography": "History/Art, Non-Fiction",
    "The Chronicle of the Discovery and Conquest of Guinea. Vol. II": "History/Exploration, Non-Fiction",
    "A Visit to the Philippine Islands": "Travel/History, Non-Fiction",
    "The Jesuit Relations and Allied Documents, Vol. 1: Acadia, 1610-1613": "History/Religion, Non-Fiction",
    "Tales From Jókai": "Fiction, Short Stories",
    "The Arabian Nights: Their Best-known Tales": "Folklore/Fiction, Short Stories Collection",
    "Chaucer's Works, Volume 1 — Romaunt of the Rose; Minor Poems": "Poetry Collection",
    "The Great God Pan": "Horror Fiction, Novella",
    "Petrarch's Letters to Classical Authors": "Biography/Letters, Non-Fiction",
    "Argonauts of the Western Pacific: An Account of Native Enterprise and Adventure in the Archipelagoes of Melanesian New Guinea": "Anthropology, Non-Fiction",
    "A History of Literary Criticism in the Renaissance: With special reference to the influence of Italy in the formation and development of modern classicism": "Literary Criticism/History, Non-Fiction",
    "Fifty Famous Stories Retold": "Children's Fiction, Short Stories",
    "England, Picturesque and Descriptive: A Reminiscence of Foreign Travel": "Travel/History, Non-Fiction",
    "The Bondage and Travels of Johann Schiltberger, a Native of Bavaria, in Europe, Asia, and Africa, 1396-1427": "Travel/Autobiography, Non-Fiction",
    "Institutes of the Christian Religion (Vol. 1 of 2)": "Religion/Theology, Non-Fiction",
    "Struggles and Triumphs: or, Forty Years' Recollections of P. T. Barnum": "Autobiography, Non-Fiction",
    "Ireland under the Tudors, with a Succinct Account of the Earlier History. Vol. 3 (of 3)": "History, Non-Fiction",
    "The Witch-cult in Western Europe: A Study in Anthropology": "Anthropology/Occult, Non-Fiction",
    "Manifesto of the Communist Party": "Politics/Philosophy, Non-Fiction",
    "Horton Genealogy: or Chronicles of the Descendants of Barnabas Horton, of Southold, L. I., 1640.": "Genealogy/History, Non-Fiction",
    "An Illustrated Dictionary of Words used in Art and Archaeology: Explaining terms frequently used in works on architecture, arms, bronzes, Christian art, colour, costume, decoration, devices, emblems, heraldry, lace, personal ornaments, pottery, painting, sculpture, &c., with their derivations.": "Reference/Art, Non-Fiction",
    "An Essay on the Development of Christian Doctrine": "Religion/Theology, Non-Fiction",
    "Japanese Girls and Women: Revised and Enlarged Edition": "Sociology/Cultural, Non-Fiction",
    "Lucian's True History": "Fiction, Satirical Fantasy",
    "The Wonderful 'One-Hoss-Shay', and Other Poems": "Poetry Collection",
    "My Bondage and My Freedom": "Autobiography, Non-Fiction",
    "The Ancient Irish Epic Tale Táin Bó Cúalnge": "Poetry/Epic, Folklore",
    "Japanese Fairy Tales": "Folklore/Fiction, Short Stories",
    "The Writings of Thomas Paine, Complete: With Index to Volumes I - IV": "Politics/Essays, Non-Fiction",
    "The Tenant of Wildfell Hall": "Fiction, Novel",
    "Women's Bathing and Swimming Costume in the United States": "History/Fashion, Non-Fiction",
    "The Kādambarī of Bāṇa": "Fiction, Romance Novel",
    "The Poems of John Donne, Volume 1 (of 2): Edited from the Old Editions and Numerous Manuscripts": "Poetry Collection",
    "Heimskringla; Or, The Chronicle of the Kings of Norway": "History/Mythology, Non-Fiction",
    "Castes and Tribes of Southern India. Vol. 1 of 7": "Anthropology/History, Non-Fiction",
    "Letters of Samuel Taylor Coleridge, Vol. 2 (of 2)": "Biography/Letters, Non-Fiction",
    "The Post Office": "Drama/Play",
    "The Planters of Colonial Virginia": "History, Non-Fiction",
    "History of Lace": "History/Fashion, Non-Fiction",
    "Twas the Night before Christmas: A Visit from St. Nicholas": "Poetry, Children's",
    "The Poor Plutocrats": "Fiction, Novel",
    "Cole's Funny Picture Book No. 1": "Children's Humor, Illustrated",
    "Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume XVI, Texas Narratives, Part 1": "History/Oral History, Non-Fiction",
    "Argentina, Legend and History": "History, Non-Fiction",
    "Every Man in His Humor": "Drama/Play, Comedy",
    "Slang and its analogues past and present, volume 3 (of 7) : $b A dictionary, historical and comparative, of the heterodox speech of all classes of society for more than three hundred years. With synonyms in English, French, German, Italian, etc.": "Reference/Dictionary, Non-Fiction",
    "Dio's Rome, Volume 5: An Historical Narrative Originally Composed in Greek During The Reigns of Septimius Severus, Geta and Caracalla, Macrinus, Elagabalus and Alexander Severus: and Now Presented in English Form": "History, Non-Fiction",
    "An Enquiry Concerning the Principles of Morals": "Philosophy, Non-Fiction",
    "The Mantle, and Other Stories": "Fiction, Short Stories",
    "Harriet, the Moses of Her People": "Biography/History, Non-Fiction",
    "The Advancement of Learning": "Philosophy/Education, Non-Fiction",
    "Henry of Ofterdingen: A Romance.": "Fiction, Romance Novel",
    "The Home Book of Verse — Volume 4": "Poetry Collection",
    "The Curiosities of Heraldry": "History/Reference, Non-Fiction",
    "The Queen of Hearts": "Fiction, Novel",
    "Isis unveiled, Volume 1 (of 2), Science : $b A master-key to mysteries of ancient and modern science and theology": "Occult/Religion, Non-Fiction",
    "The Expositor's Bible: The Book of the Twelve Prophets, Vol. 2: Commonly Called the Minor": "Religion/Commentary, Non-Fiction",
    "The Inhabitants of the Philippines": "History/Anthropology, Non-Fiction",
    "She": "Adventure Fiction, Novel",
    "Philippine progress prior to 1898 : $b a source book of Philippine history to supply a fairer view of Filipino participation and supplement the defective Spanish accounts": "History, Non-Fiction",
    "A Cyclopaedia of Canadian Biography: Being Chiefly Men of the Time: A Collection of Persons Distinguished in Professional and Political Life, Leaders in the Commerce and Industry of Canada, and Successful Pioneers": "Biography/Reference, Non-Fiction",
    "The Phoenix and the Carpet": "Children's Fiction, Fantasy Novel",
    "Doña Perfecta": "Fiction, Novel",
    "History of Linn County Iowa: From Its Earliest Settlement to the Present Time": "History, Non-Fiction",
    "Poems of James Russell Lowell: With biographical sketch by Nathan Haskell Dole": "Poetry Collection",
    "The Fairy Mythology: Illustrative of the Romance and Superstition of Various Countries": "Folklore, Non-Fiction",
    "The Education of Henry Adams": "Autobiography, Non-Fiction",
    "Pushing to the Front": "Self-Help, Non-Fiction",
    "The Civilisation of the Renaissance in Italy": "History/Cultural, Non-Fiction",
    "Slave Narratives: A Folk History of Slavery in the United States from Interviews with Former Slaves, Volume III, Florida Narratives": "History/Oral History, Non-Fiction",
    "Twice-told tales": "Fiction, Short Stories",
    "The Underground Railroad: A Record of Facts, Authentic Narratives, Letters, &c., Narrating the Hardships, Hair-Breadth Escapes and Death Struggles of the Slaves in Their Efforts for Freedom, As Related by Themselves and Others, or Witnessed by the Author.": "History, Non-Fiction",
    "The Life of John Marshall, Volume 2: Politician, diplomatist, statesman, 1789-1801": "Biography/History, Non-Fiction",
    "Henry IV, Makers of History": "Biography/History, Non-Fiction",
    "Astounding Stories of Super-Science February 1930": "Science Fiction, Magazine/Short Stories",
    "Seneca myths and folk tales": "Folklore, Non-Fiction",
    "An Introduction to Entomology: Vol. 1: or Elements of the Natural History of the Insects": "Science/Entomology, Non-Fiction",
    "Ireland in Fiction: A Guide to Irish Novels, Tales, Romances, and Folk-lore": "Literary Criticism/Reference, Non-Fiction",
    "Plotinos: Complete Works, v. 4: In Chronological Order, Grouped in Four Periods": "Philosophy, Non-Fiction",
    "The History of Prostitution: Its Extent, Causes, and Effects throughout the World": "History/Sociology, Non-Fiction",
    "The Philippine Islands: A Political, Geographical, Ethnographical, Social and Commercial History of the Philippine Archipelago, Embracing the Whole Period of Spanish Rule": "History, Non-Fiction",
    "A Selection from the Works of Frederick Locker": "Poetry Collection",
    "Galicia, the Switzerland of Spain": "Travel/History, Non-Fiction",
    "Thoughts of Marcus Aurelius Antoninus": "Philosophy, Non-Fiction",
    "The Part Borne by the Dutch in the Discovery of Australia 1606-1765": "History/Exploration, Non-Fiction",
    "Discourses on the First Decade of Titus Livius": "History/Politics, Non-Fiction",
    "Practical taxidermy : $b a manual of instruction to the amateur in collecting, preserving, and setting up natural history specimens of all kinds. To which is added a chapter upon the pictorial arrangement of museums. With additional instructions in modelling and artistic taxidermy.": "Science/Instruction, Non-Fiction",
    "The Great Illusion: A Study of the Relation of Military Power to National Advantage": "Politics/Economics, Non-Fiction",
    "Plant Lore, Legends, and Lyrics: Embracing the Myths, Traditions, Superstitions, and Folk-Lore of the Plant Kingdom": "Folklore/Botany, Non-Fiction",
    "Mythology among the Hebrews and Its Historical Development": "Mythology/History, Non-Fiction",
    "The Younger Set": "Fiction, Novel",
    "Mount Everest, the Reconnaissance, 1921": "Travel/Exploration, Non-Fiction",
    "Letters to the young from the Old World : $b Notes of travel": "Travel/Letters, Non-Fiction",
    "The Blue Goose": "Fiction, Novel",
    "The call of Cthulhu": "Horror Fiction, Short Story",
    "Household Tales by Brothers Grimm": "Children's Fiction, Fairy Tales Collection",
    "The Lusiad; Or, The Discovery of India, an Epic Poem": "Poetry, Epic",
    "Secret Service Under Pitt": "History/Espionage, Non-Fiction",
    "Man and Nature; Or, Physical Geography as Modified by Human Action": "Geography/Environment, Non-Fiction",
    "The Hour of the Dragon": "Fantasy Fiction, Novel",
    "A Cyclopædia of Canadian Biography: Brief biographies of persons distinguished in the professional, military and political life, and the commerce and industry of Canada, in the twentieth century": "Biography/Reference, Non-Fiction",
    "The Letters of Cicero, Volume 1: The Whole Extant Correspodence in Chronological Order": "Biography/Letters, Non-Fiction",
    "A Collection of Essays and Fugitiv Writings: On Moral, Historical, Political, and Literary Subjects": "Essays/Politics, Non-Fiction",
    "Mr. Punch's Golf Stories": "Humor/Fiction, Short Stories",
    "The whole truth and nothing but": "Autobiography, Non-Fiction",
    "The Nuttall Encyclopædia: Being a Concise and Comprehensive Dictionary of General Knowledge": "Reference/Encyclopedia, Non-Fiction",
    "The History of Modern Painting, Volume 1 (of 4): Revised edition continued by the author to the end of the XIX century": "Art/History, Non-Fiction",
    "Lays of Ancient Rome": "Poetry, Historical Verse",
    "Jerusalem Explored, Volume 1—Text: Being a Description of the Ancient and Modern City, with Numerous Illustrations Consisting of Views, Ground Plans and Sections": "History/Travel, Non-Fiction",
    "The Gutenberg Webster's Unabridged Dictionary: Section X, Y, and Z": "Reference/Dictionary, Non-Fiction",
    "The Box-Car Children": "Children's Fiction, Adventure Novel",
    "Sketches and Studies in Italy and Greece, Complete: Series I, II, and III": "Travel/Essays, Non-Fiction",
    "The Hacker Crackdown: Law and Disorder on the Electronic Frontier": "Technology/History, Non-Fiction",
    "A collection of Latin maxims and phrases literally translated: Intended for the use of students for all legal examinations": "Reference/Law, Non-Fiction",
    "Richard III: His Life & Character, Reviewed in the Light of Recent Research": "Biography/History, Non-Fiction",
    "A guide book of art, architecture, and historic interests in Pennsylvania": "Art/History/Guide, Non-Fiction",
    "History of Mexico, Volume 1, 1516-1521": "History, Non-Fiction",
    "Ernest Maltravers — Complete": "Fiction, Novel",
    "The Idea of a University Defined and Illustrated: In Nine Discourses Delivered to the Catholics of Dublin": "Education/Philosophy, Non-Fiction",
    "The Surprising Adventures of Baron Munchausen": "Fiction, Satirical Adventure Novel",
    "History of Civilization in England, Vol. 2 of 3": "History, Non-Fiction",
    "The Organism as a Whole, from a Physicochemical Viewpoint": "Science/Biology, Non-Fiction",
    "Narrative and Critical History of America, Vol. 3 (of 8): English Explorations and Settlements in North America 1497-1689": "History/Exploration, Non-Fiction",
    "A Japanese Boy": "Autobiography/Cultural, Non-Fiction",
    "Tremendous Trifles": "Essays/Humor, Non-Fiction",
    "Plotinos: Complete Works, v. 3: In Chronological Order, Grouped in Four Periods": "Philosophy, Non-Fiction",
    "Cautionary Tales for Children": "Children's Poetry, Satirical Verse",
    "Hegel's Lectures on the History of Philosophy: Volume 1 (of 3)": "Philosophy/History, Non-Fiction",
    "Lilith: A Romance": "Fiction, Fantasy Novel",
    "Hector Berlioz: A Romantic Tragedy": "Biography, Non-Fiction",
    "History, Gazetteer, and Directory of Shropshire": "History/Reference, Non-Fiction",
    "Lives of the Most Eminent Painters Sculptors and Architects, Vol. 01 (of 10): Cimabue to Agnolo Gaddi": "Biography/Art History, Non-Fiction",
    "Hossfeld's Spanish Dialogues, and Idiomatic Phrases Indispensible for a Rapid Acquisition of the Spanish Language": "Education/Language, Non-Fiction",
    "The Story of Mankind": "History, Non-Fiction for Youth",
    "The Works of the Right Honourable Edmund Burke, Vol. 03 (of 12)": "Politics/Essays, Non-Fiction",
    "Lady Audley's Secret": "Mystery Fiction, Novel",
    "The Monk: A Romance": "Fiction, Gothic Novel",
    "Alchemy: Ancient and Modern: Being a Brief Account of the Alchemistic Doctrines, and Their Relations, to Mysticism on the One Hand, and to Recent Discoveries in Physical Science on the Other Hand; Together with Some Particulars Regarding the Lives and Teachings of the Most Noted Alchemists": "Science/History/Occult, Non-Fiction",
    "The story hunter : $b or, Tales of the weird and wild": "Fiction, Short Stories/Horror",
    "The Princess and the Goblin": "Children's Fiction, Fantasy Novel",
    "The Molly Maguires and the detectives": "History/Crime, Non-Fiction",
    "Dickens and His Illustrators: Cruikshank, Seymour, Buss, \"Phiz,\" Cattermole, Leech, Doyle, Stanfield, Maclise, Tenniel, Frank Stone, Landseer, Palmer, Topham, Marcus Stone, and Luke Fildes; 2nd. Ed.": "Biography/Art History, Non-Fiction",
    "The origin of the family, private property, and the state": "Sociology/Philosophy, Non-Fiction",
    "Ancient Monuments of the Mississippi Valley: Smithsonian Contributions to Knowledge, Vol. I.": "Archaeology/History, Non-Fiction",
    "Flappers and Philosophers": "Fiction, Short Stories",
    "Human Nature and Conduct: An introduction to social psychology": "Psychology/Sociology, Non-Fiction",
    "The Negro in the United States; a selected bibliography. Compiled by Dorothy B. Porter": "Reference/History, Non-Fiction",
    "A Literary History of the Arabs": "Literary Criticism/History, Non-Fiction",
    "The Hyborian Age": "Fantasy Fiction, Essay/Short Story",
    "India: What can it teach us?: A Course of Lectures Delivered before the University Of Cambridge": "History/Cultural, Non-Fiction",
    "An Old Babylonian Version of the Gilgamesh Epic": "Mythology/Poetry, Translation",
    "Financial Crime and Corruption": "Economics/Crime, Non-Fiction",
    "The Lani People": "Science Fiction, Novel",
    "Bloodletting Instruments in the National Museum of History and Technology": "History/Medicine, Non-Fiction",
    "The Histories of Polybius, Vol. 2 (of 2)": "History, Non-Fiction",
    "The Hanging Stranger": "Science Fiction, Short Story",
    "The Four Million": "Fiction, Short Stories",
    "On the Old Road Vol. 1 (of 2): A Collection of Miscellaneous Essays and Articles on Art and Literature": "Essays/Literary Criticism, Non-Fiction",
    "Tamburlaine the Great — Part 1": "Drama/Play, Tragedy",
    "A History of Lancashire": "History, Non-Fiction",
    "Index of the Project Gutenberg Works of O. Henry": "Reference/Literary, Non-Fiction",
    "A History of Epidemics in Britain, Volume 1 (of 2): From A.D. 664 to the Extinction of Plague": "History/Medicine, Non-Fiction",
    "Aids to Reflection; and, The Confessions of an Inquiring Spirit": "Philosophy/Religion, Non-Fiction",
    "Montreal, 1535-1914. Vol. 2. Under British Rule, 1760-1914": "History, Non-Fiction",
    "Areopagitica: A Speech for the Liberty of Unlicensed Printing to the Parliament of England": "Essays/Politics, Non-Fiction",
    "The Virginian: A Horseman of the Plains": "Fiction, Western Novel",
    "The first voyage round the world, by Magellan : $b translated from the accounts of Pigafetta, and other contemporary writers": "History/Exploration, Non-Fiction",
    "How music grew, from prehistoric times to the present day": "Music/History, Non-Fiction",
    "The Marvelous Land of Oz": "Children's Fiction, Fantasy Novel",
    "How I found Livingstone : $b Travels, adventures, and discoveres in Central Africa, including an account of four months' residence with Dr. Livingstone": "Travel/Exploration, Non-Fiction",
    "A History of the Inquisition of Spain; vol. 1": "History/Religion, Non-Fiction",
    "Phantastes: A Faerie Romance for Men and Women": "Fiction, Fantasy Novel",
    "The Economist": "Economics/Non-Fiction",
    "The Fables of Phædrus: Literally translated into English prose with notes": "Poetry/Fables, Translation",
    "The Poems of Jonathan Swift, D.D., Volume 2": "Poetry Collection",
    "Letters of Asa Gray; Vol. 2": "Biography/Letters, Non-Fiction",
    "A Contribution to the Critique of Political Economy": "Economics/Philosophy, Non-Fiction",
    "Virginia": "Fiction, Novel",
    "On The Principles of Political Economy, and Taxation": "Economics, Non-Fiction",
    "Spanish Explorers in the Southern United States, 1528-1543.: The Narrative of Alvar Nunez Cabeca de Vaca. The Narrative of the Expedition of Hernando De Soto by the Gentleman of Elvas": "History/Exploration, Non-Fiction",
    "The Churches of Paris, from Clovis to Charles X": "History/Architecture, Non-Fiction",
    "A History of Matrimonial Institutions, Vol. 1 of 3": "History/Sociology, Non-Fiction",
    "A History of Inventions, Discoveries, and Origins, Volume 1 (of 2)": "History/Science, Non-Fiction",
    "Tarrano the Conqueror": "Science Fiction, Novel",
    "I. Beówulf: an Anglo-Saxon poem. II. The fight at Finnsburh: a fragment.": "Poetry/Epic",
    "A Budget of Paradoxes, Volume I": "Mathematics/History, Non-Fiction",
    "The letters of Hart Crane, 1916-1932": "Biography/Letters, Non-Fiction",
    "Chaucer and His England": "Biography/History, Non-Fiction",
    "Northern Nut Growers Association Report of the Proceedings at the 44th Annual Meeting: Rochester, N.Y. August 31 and September 1, 1953": "Science/Agriculture, Non-Fiction Report",
    "The Memoirs of François René Vicomte de Chateaubriand sometime Ambassador to England, Volume 1 (of 6): Mémoires d'outre-tombe, volume 1": "Autobiography/History, Non-Fiction",
    "The Christ: A Critical Review and Analysis of the Evidences of His Existence": "Religion/History, Non-Fiction",
    "The Imaginary Invalid": "Drama/Play, Comedy",
    "Shakespeare's environment": "Literary Criticism/History, Non-Fiction",
    "Tacitus: The Histories, Volumes I and II": "History, Non-Fiction",
    "Asser's Life of King Alfred": "Biography/History, Non-Fiction",
    "King Spruce, A Novel": "Fiction, Novel",
    "Zoological Mythology; or, The Legends of Animals, Volume 2 (of 2)": "Folklore/Mythology, Non-Fiction",
    "North Dakota: A Guide to the Northern Prairie State": "Travel/Guide, Non-Fiction",
    "London Labour and the London Poor, Vol. 1": "Sociology/History, Non-Fiction",
    "The seventh shot : $b A detective story": "Mystery Fiction, Novel",
    "The Agamemnon of Aeschylus: Translated into English Rhyming Verse with Explanatory Notes": "Drama/Play, Tragedy",
    "The Discoverie of Witchcraft": "Occult/History, Non-Fiction",
    "Round the Corner in Gay Street": "Fiction, Novel",
    "Plymouth memories of an octogenarian": "Autobiography/History, Non-Fiction",
    "Latin for Beginners": "Education/Language, Non-Fiction",
    "Nature Books Popular and Scientific from The Franklin Bookshop, 1910: Catalogue 24, 1910-11 Season": "Reference/Bibliography, Non-Fiction",
    "Handbook to the Mennonite Hymnary": "Religion/Music, Non-Fiction",
    "\"De Bello Gallico\" and Other Commentaries": "History/Military, Non-Fiction",
    "The Philippine Islands, 1493-1898, Volume 35, 1640-1649: Explorations by early navigators, descriptions of the islands and their peoples, their history and records of the Catholic missions, as related in contemporaneous books and manuscripts, showing the political, economic, commercial and religious conditions of those islands from their earliest relations with European nations to the close of the nineteenth century": "History/Exploration, Non-Fiction",
    "Christina of Denmark, Duchess of Milan and Lorraine, 1522-1590": "Biography/History, Non-Fiction",
    "The Metamorphoses of Ovid, Books VIII-XV": "Poetry/Mythology, Translation",
    "The language of flowers : $b The floral offering ; a token of affection and esteem ; comprising the language and poetry of flowers": "Reference/Cultural, Non-Fiction",
    "De Orbe Novo, Volume 1 (of 2): The Eight Decades of Peter Martyr D'Anghera": "History/Exploration, Non-Fiction",
    "The Bacchae of Euripides": "Drama/Play, Tragedy",
    "The American Joe Miller: A Collection of Yankee Wit and Humor": "Humor/Folklore, Non-Fiction Collection",
    "Nineteen Centuries of Drink in England: A History": "History/Cultural, Non-Fiction",
    "Home Geography for Primary Grades": "Education/Geography, Non-Fiction",
    "History of Woman Suffrage, Volume I": "History/Feminism, Non-Fiction",
    "The Hungry Stones, and Other Stories": "Fiction, Short Stories",
    "Against the Grain": "Fiction, Decadent Novel",
    "Raymond; or, Life and Death: With examples of the evidence for survival of memory and affection after death.": "Spirituality/Non-Fiction",
    "Ancient Nahuatl Poetry, Containing the Nahuatl Text of XXVII Ancient Mexican Poems: Brinton's Library of Aboriginal American Literature Number VII.": "Poetry Collection, Translation",
    "The Stones of Venice, Volume 2 (of 3),": "Art/Architecture Criticism, Non-Fiction",
    "Steam-ships : $b The story of their development to the present day": "History/Technology, Non-Fiction",
    "The Yeoman Adventurer": "Adventure Fiction, Novel",
    "Magic and Religion": "Anthropology/Religion, Non-Fiction",
    "France and England in North America, Part III: La Salle, Discovery of The Great West": "History/Exploration, Non-Fiction",
    "Lost Face": "Fiction, Short Stories",
    "France in the Nineteenth Century": "History, Non-Fiction",
    "The Riddle of the Sands": "Adventure Fiction, Spy Novel",
    "The Arabian Nights Entertainments": "Folklore/Fiction, Short Stories Collection",
    "Vanishing England": "History/Architecture, Non-Fiction",
    "Cicero: Letters to Atticus, Vol. 2 of 3": "Biography/Letters, Non-Fiction",
    "My Musical Life": "Autobiography/Music, Non-Fiction",
    "Erewhon; Or, Over the Range": "Fiction, Satirical Utopian Novel",
    "Nietzsche and Art": "Philosophy/Art Criticism, Non-Fiction",
    "The magnificent possession": "Fiction, Novel",
    "Journeys and Experiences in Argentina, Paraguay, and Chile: Including a Side Trip to the Source of the Paraguay River in the State of Matto Grosso, Brazil, and a Journey Across the Andes to the Rio Tambo in Peru": "Travel/Adventure, Non-Fiction",
    "Smithsonian Institution - United States National Museum - Bulletin 240: Contributions From the Museum of History and Technology; Papers 34-44 on Science and Technology": "Science/History, Non-Fiction",
    "Beyond the hills": "Fiction, Novel",
    "Inventions in the Century": "History/Technology, Non-Fiction",
    "The Story of Magellan and The Discovery of the Philippines": "History/Exploration, Non-Fiction",
    "The Sorrows of Satan: or, The Strange Experience of One Geoffrey Tempest, Millionaire: A Romance": "Fiction, Fantasy Novel",
    "Library of the World's Best Literature, Ancient and Modern — Volume 16": "Literary Anthology",
    "The decline of the West : $b Volume 1, Form and actuality": "Philosophy/History, Non-Fiction",
    "Annals of the Bodleian Library, Oxford, A.D. 1598-A.D. 1867: With a Preliminary Notice of the earlier Library founded in the Fourteenth Century": "History/Literary, Non-Fiction",
    "The German Terror in Belgium: An Historical Record": "History/War, Non-Fiction",
    "Curiosities of Literature, Vol. 2": "Literary Criticism/Essays, Non-Fiction",
    "Repertory of The Comedie Humaine, Complete, A — Z": "Reference/Literary, Non-Fiction",
    "The Old Inns of Old England, Volume 1 (of 2): A Picturesque Account of the Ancient and Storied Hostelries of Our Own Country": "History/Travel, Non-Fiction",
    "The Doctor's Dilemma: Preface on Doctors": "Drama/Play, Comedy",
    "Memoirs of the Reign of King George the Third, Volume 1 (of 4)": "History/Biography, Non-Fiction",
    "The Story of London": "History, Non-Fiction",
    "The Witness of the Stars": "Religion/Astrology, Non-Fiction",
    "The Tao Teh King, or the Tao and its Characteristics": "Philosophy/Religion, Non-Fiction",
    "The Formation of Christendom, Volume II": "Religion/History, Non-Fiction",
    "The Races of Man: An Outline of Anthropology and Ethnography": "Anthropology, Non-Fiction",
    "The massacre of St. Bartholomew : $b Preceded by a history of the religious wars in the reign of Charles IX": "History/Religion, Non-Fiction",
    "The Regimental Records of the British Army: A historical résumé chronologically arranged of titles, campaigns, honours, uniforms, facings, badges, nicknames, etc.": "History/Military, Non-Fiction",
    "The Book of the Damned": "Science/Occult, Non-Fiction",
    "The Life and Amours of the Beautiful, Gay and Dashing Kate Percival: The Belle of the Delaware": "Fiction, Erotic Memoir",
    "Vikram and the Vampire: Classic Hindu Tales of Adventure, Magic, and Romance": "Folklore/Fiction, Short Stories",
    "Blue eyes and diamonds": "Fiction, Novel",
    "The Middle English Poem, Erthe Upon Erthe": "Poetry, Medieval Verse",
    "Random Reminiscences of Men and Events": "Autobiography/Business, Non-Fiction",
    "Pan Tadeusz; or, The last foray in Lithuania: a story of life among Polish gentlefolk in the years 1811 and 1812": "Poetry, Epic",
    "The Romance of Plant Life: Interesting Descriptions of the Strange and Curious in the Plant World": "Science/Botany, Non-Fiction",
    "Lincoln's yarns and stories : $b A complete collection of the funny and witty anecdotes that made Lincoln famous as America's greatest story teller": "Humor/History, Non-Fiction Collection",
    "Guernsey Folk Lore: a collection of popular superstitions, legendary tales, peculiar customs, proverbs, weather sayings, etc., of the people of that island": "Folklore, Non-Fiction",
    "English Surnames: Their Sources and Significations": "History/Linguistics, Non-Fiction",
    "The Tale of Benjamin Bunny": "Children's Fiction, Animal Tale",
    "A Manual of American Literature": "Literary Criticism/History, Non-Fiction",
    "Buddhist birth stories; or, Jataka tales, Volume 1": "Folklore/Religion, Non-Fiction",
    "The Works of Horace": "Poetry Collection, Translation",
    "The Goddess of Atvatabar: Being the history of the discovery of the interior world and conquest of Atvatabar": "Science Fiction, Adventure Novel",
    "A Military Dictionary and Gazetteer: Comprising ancient and modern military technical terms, historical accounts of all North American Indians, as well as ancient warlike tribes; also notices of battles from the earliest period to the present time, with a concise explanation of terms used in heraldry and the offices thereof. The work also gives valuable geographical information. Compiled from the best authorities of all nations. With an appendix containing the Articles of war, etc.": "Reference/Military, Non-Fiction",
    "Æschylos Tragedies and Fragments": "Drama/Plays, Tragedy Collection",
    "The Romance of Tristan and Iseult": "Fiction, Arthurian Romance",
    "Complete Hypnotism, Mesmerism, Mind-Reading and Spiritualism: How to Hypnotize: Being an Exhaustive and Practical System of Method, Application, and Use": "Occult/Science, Non-Fiction",
    "A Popular History of France from the Earliest Times, Volume 4": "History, Non-Fiction",
    "The Luck of Roaring Camp and Other Tales: With Condensed Novels, Spanish and American Legends, and Earlier Papers": "Fiction, Short Stories",
    "Trial by water": "Fiction, Novel",
    "Line and Form (1900)": "Art/Instruction, Non-Fiction",
    "The Industrial Arts in Spain": "Art/History, Non-Fiction",
    "Howards End": "Fiction, Social Novel",
    "Bahaism and Its Claims: A Study of the Religion Promulgated by Baha Ullah and Abdul Baha": "Religion/History, Non-Fiction",
    "The censorship of the Church of Rome and its influence upon the production and distribution of literature, volume 2 (of 2)": "History/Religion, Non-Fiction",
    "'Everyman,' with other interludes, including eight miracle plays": "Drama/Plays Collection, Medieval",
    "Hazlitt on English Literature: An Introduction to the Appreciation of Literature": "Literary Criticism, Non-Fiction",
    "Ancient society : $b or, Researches in the lines of human progress from savagery, through barbarism to civilization": "Anthropology/Sociology, Non-Fiction",
    "Daisy Miller: A Study": "Fiction, Novella",
    "Confessions of an English Opium-Eater": "Autobiography, Non-Fiction",
    "The Rainbow": "Fiction, Novel",
    "The Expositor's Bible: The First Book of Kings": "Religion/Commentary, Non-Fiction",
    "Adventures and Reminiscences of a Volunteer; Or, A Drummer Boy from Maine": "Autobiography/War, Non-Fiction",
    "Great Astronomers": "Biography/Science, Non-Fiction",
    "The History of 'Punch'": "History/Literary, Non-Fiction",
    "The Mysteries of Udolpho": "Fiction, Gothic Novel",
    "The Suppression of the African Slave Trade to the United States of America: 1638-1870": "History, Non-Fiction",
    "Hawaiian folk tales : $b a collection of native legends": "Folklore/Fiction, Short Stories",
    "The Chinese Fairy Book": "Folklore/Fiction, Short Stories Collection",
    "Aeschylus' Prometheus Bound and the Seven Against Thebes": "Drama/Play, Tragedy",
    "Memoirs of the Dukes of Urbino, Volume 1 (of 3): Illustrating the Arms, Arts, and Literature of Italy, from 1440 To 1630.": "History/Biography, Non-Fiction",
    "An Advanced English Grammar with Exercises": "Education/Language, Non-Fiction",
    "The Story of Siena and San Gimignano": "History/Travel, Non-Fiction",
    "Biographical Anecdotes of William Hogarth, With a Catalogue of His Works": "Biography/Art History, Non-Fiction",
    "Some Principles of Maritime Strategy": "Military/Strategy, Non-Fiction",
    "A History of French Literature: Short Histories of the Literatures of the World: II.": "Literary Criticism/History, Non-Fiction",
    "The Ingoldsby Legends; or, Mirth and Marvels": "Poetry/Humor Collection",
    "Dramatic Technique": "Literary Criticism/Drama, Non-Fiction",
    "Kwaidan: Stories and Studies of Strange Things": "Folklore/Horror, Short Stories",
    "Italian Highways and Byways from a Motor Car": "Travel, Non-Fiction",
    "The nigger of the 'Narcissus' : $b A tale of the forecastle": "Fiction, Novella",
    "The Strange Adventures of Andrew Battell, of Leigh, in Angola and the Adjoining Regions": "Travel/Adventure, Non-Fiction",
    "How to Live on 24 Hours a Day": "Self-Help, Non-Fiction",
    "The Magic and Science of Jewels and Stones": "History/Occult, Non-Fiction",
    "Up the Orinoco and down the Magdalena": "Travel/Adventure, Non-Fiction",
    "The sound and the fury": "Fiction, Modernist Novel",
    "The eleventh hour in the life of Julia Ward Howe": "Biography, Non-Fiction",
    "The Inimitable Jeeves": "Fiction, Humorous Novel",
    "The Cavalier Songs and Ballads of England from 1642 to 1684": "Poetry/Folklore Collection",
    "The Fables of Aesop: Selected, Told Anew, and Their History Traced": "Children's Fiction, Fables",
    "Haviland's Chum": "Fiction, Novel",
    "The World's Greatest Books — Volume 02 — Fiction": "Literary Anthology/Fiction",
    "In the hollow of His hand": "Fiction, Novel",
    "The Man of the Forest": "Fiction, Western Novel",
    "Play-Making: A Manual of Craftsmanship": "Literary Criticism/Drama, Non-Fiction",
    "The Ballads and Songs of Yorkshire: Transcribed from Private Manuscripts, Rare Broadsides, and Scarce Publications; with Notes and a Glossary": "Poetry/Folklore Collection",
    "Mrs. Warren's Profession": "Drama/Play",
    "The Practice of the Presence of God": "Religion/Spirituality, Non-Fiction",
    "Old Christmas: from the Sketch Book of Washington Irving": "Fiction, Sketches/Holiday",
    "Waverley; or, 'Tis sixty years since": "Historical Fiction, Novel",
    "Notes on Nursing: What It Is, and What It Is Not": "Medicine/Health, Non-Fiction",
    "The Mediaeval Mind (Volume 2 of 2): A History of the Development of Thought and Emotion in the Middle Ages": "History/Philosophy, Non-Fiction",
    "Armenia, Travels and Studies (Volume 1 of 2): The Russian Provinces": "Travel/History, Non-Fiction",
    "The Forsyte Saga - Complete": "Fiction, Novel Series",
    "Renshaw Fanning's Quest: A Tale of the High Veldt": "Adventure Fiction, Novel",
    "A Century of Parody and Imitation": "Literary Criticism/Humor, Non-Fiction",
    "Life in the Iron-Mills; Or, The Korl Woman": "Fiction, Short Story",
    "The Classic Myths in English Literature and in Art (2nd ed.) (1911): Based Originally on Bulfinch's 'Age of Fable' (1855)": "Mythology/Literary Criticism, Non-Fiction",
    "The English and Scottish popular ballads, volume 1 (of 5)": "Poetry/Folklore Collection",
    "Lancashire Folk-lore: Illustrative of the Superstitious Beliefs and Practices, Local Customs and Usages of the People of the County Palatine": "Folklore, Non-Fiction",
    "The Sayings of Confucius: A New Translation of the Greater Part of the Confucian Analects": "Philosophy/Religion, Non-Fiction",
    "Ten Thousand Wonderful Things: Comprising whatever is marvellous and rare, curious, eccentric and extraordinary in all ages and nations": "Reference/Curiosities, Non-Fiction",
    "Giovanni Boccaccio, a Biographical Study": "Biography/Literary Criticism, Non-Fiction",
    "Ethnological results of the Point Barrow expedition: Ninth Annual Report of the Bureau of Ethnology to the Secretary of the Smithsonian Institution, 1887-1888, Government Printing Office, Washington, 1892, pages 3-442": "Anthropology, Non-Fiction",
    "With the Night Mail: A Story of 2000 A.D.: (Together with extracts from the comtemporary magazine in which it appeared)": "Science Fiction, Short Story",
    "The Book Review Digest, v. 16, 1920 : $b Sixteenth annual accumulation. Reviews of 1920 books": "Literary Criticism/Reference, Non-Fiction",
    "Aesthetical Essays of Friedrich Schiller": "Philosophy/Art Criticism, Non-Fiction",
    "Good Friday, and other poems": "Poetry Collection",
    "What Every Girl Should Know": "Self-Help/Health, Non-Fiction",
    "The La Chance Mine Mystery": "Mystery Fiction, Novel",
    "A Treatise on Wood Engraving, Historical and Practical": "Art/History, Non-Fiction",
    "The Velveteen Rabbit": "Children's Fiction, Fantasy Tale",
    "Curious Creatures in Zoology": "Science/Natural History, Non-Fiction",
    "History of the Colony and Ancient Dominion of Virginia": "History, Non-Fiction",
    "Father Goriot": "Fiction, Novel",
    "Babylonian and Assyrian Laws, Contracts and Letters": "History/Law, Non-Fiction",
    "The Brighton Road: The Classic Highway to the South": "Travel/History, Non-Fiction",
    "The History of Antiquity, Vol. 1 (of 6)": "History, Non-Fiction",
    "Character Sketches of Romance, Fiction, and the Drama, Vol. 4: A Revised American Edition of the Reader's Handbook": "Reference/Literary Criticism, Non-Fiction",
    "The inventions, researches and writings of Nikola Tesla: With special reference to his work in polyphase currents and high potential lighting": "Science/Biography, Non-Fiction",
    "A Complete History of Music: for Schools, Clubs, and Private Reading": "Music/History, Non-Fiction",
    "Soap-Making Manual: A Practical Handbook on the Raw Materials, Their Manipulation, Analysis and Control in the Modern Soap Plant.": "Science/Instruction, Non-Fiction",
    "Venice and Its Story": "History/Travel, Non-Fiction",
    "Lady Jane Grey and Her Times": "Biography/History, Non-Fiction",
    "Self-instructor in the art of hair work, dressing hair, making curls, switches, braids, and hair jewelry of every description.": "Instruction/Beauty, Non-Fiction",
    "History of the Jews in Russia and Poland, Volume 3 [of 3]: From the Accession of Nicholas II until the Present Day": "History, Non-Fiction",
    "Recollections of a Tour Made in Scotland A.D. 1803": "Travel/Autobiography, Non-Fiction",
    "Correspondence and Report from His Majesty's Consul at Boma Respecting the Administration of the Independent State of the Congo [and Further Correspondence]": "History/Politics, Non-Fiction",
    "Town Life in the Fifteenth Century, Volume 2 (of 2)": "History/Social, Non-Fiction",
    "The Project Gutenberg Encyclopedia, Volume 1 of 28": "Reference/Encyclopedia, Non-Fiction",
    "Prisoners of War in Britain 1756 to 1815: A record of their lives, their romance and their sufferings": "History/Military, Non-Fiction",
    "My past and thoughts, vol 1 (of 6) : $b The memoirs of Alexander Herzen": "Autobiography/History, Non-Fiction",
    "Creative Evolution": "Philosophy/Science, Non-Fiction",
    "A Gothic Grammar, with selections for reading and a glossary": "Education/Language, Non-Fiction",
    "The Voyages of Pedro Fernandez de Quiros, 1595 to 1606. Volume 1": "History/Exploration, Non-Fiction",
    "The First Blast of the Trumpet against the monstrous regiment of Women": "Politics/Religion, Non-Fiction",
    "Comrades: A Story of Social Adventure in California": "Fiction, Social Novel",
    "Beethoven's Letters 1790-1826, Volume 1": "Biography/Letters, Non-Fiction",
    "On Love": "Philosophy/Essays, Non-Fiction",
    "The Pencil of Nature": "Photography/Art, Non-Fiction",
    "The eternal moment, and other stories": "Fiction, Short Stories",
    "The Mysteries of All Nations: Rise and Progress of Superstition, Laws Against and Trials of Witches, Ancient and Modern Delusions Together with Strange Customs, Fables, and Tales": "Folklore/History, Non-Fiction",
    "Records of the Spanish Inquisition, Translated from the Original Manuscripts": "History/Religion, Non-Fiction",
    "Parzival: A Knightly Epic (vol. 1 of 2)": "Poetry, Epic Romance",
    "L. Annaeus Seneca on Benefits": "Philosophy/Ethics, Non-Fiction",
    "Historia Amoris: A History of Love, Ancient and Modern": "History/Cultural, Non-Fiction",
    "Human Animals": "Folklore/Occult, Non-Fiction",
    "The best British short stories of 1922": "Fiction, Short Stories Collection",
    "United States Presidents' Inaugural Speeches: From Washington to George W. Bush": "History/Politics, Non-Fiction Collection",
    "The Phase Rule and Its Applications": "Science/Chemistry, Non-Fiction",
    "One Woman's Life": "Fiction, Novel",
    "In Morocco": "Travel, Non-Fiction",
    "Autobiography of Andrew Carnegie": "Autobiography, Non-Fiction",
    "The best short stories of 1922, and the yearbook of the American short story": "Fiction, Short Stories Collection",
    "Commentary on Genesis, Vol. 1: Luther on the Creation": "Religion/Commentary, Non-Fiction",
    "Uncle Remus, his songs and his sayings": "Folklore/Fiction, Short Stories",
    "Jewellery": "Art/History, Non-Fiction",
    "Don Juan Tenorio": "Drama/Play",
    "Fools and mules : $b [A Shorty McKay story]": "Fiction, Short Story",
    "Little Eyebright and her pund o' care": "Fiction, Novel",
    "The Memoirs of François René Vicomte de Chateaubriand sometime Ambassador to England. volume 5 (of 6): Mémoires d'outre-tombe volume 5": "Autobiography/History, Non-Fiction",
    "The World's Sixteen Crucified Saviors; Or, Christianity Before Christ": "Religion/Mythology, Non-Fiction",
    "The Posthumous Works of Thomas De Quincey, Vol. 1": "Essays/Literary, Non-Fiction",
    "The Analects of Confucius (from the Chinese Classics)": "Philosophy/Religion, Non-Fiction",
    "Custom and Myth: New Edition": "Mythology/Anthropology, Non-Fiction",
    "Character Sketches of Romance, Fiction, and the Drama, Vol. 2: A Revised American Edition of the Reader's Handbook": "Reference/Literary Criticism, Non-Fiction",
    "Machiavelli, Volume I": "Politics/Philosophy, Non-Fiction",
    "Yachting, Vol. 2": "Sports/History, Non-Fiction",
    "My Ántonia": "Fiction, Novel",
    "Karl Grier : $b The strange story of a man with a sixth sense": "Fiction, Novel",
    "The Story of the Great Fire in St. John, N.B., June 20th, 1877": "History, Non-Fiction",
    "The United States Constitution": "History/Politics, Non-Fiction",
    "My Reminiscences": "Autobiography, Non-Fiction",
    "A Catalogue of Books in English Later than 1700, Vol. 1: Forming a portion of the library of Robert Hoe": "Reference/Bibliography, Non-Fiction",
    "Selected Poems": "Poetry Collection",
    "Childhood's Favorites and Fairy Stories: The Young Folks Treasury, Volume 1": "Children's Fiction, Anthology",
    "The Little Mother Goose": "Children's Poetry, Collection",
    "The Ethics of Medical Homicide and Mutilation": "Medicine/Ethics, Non-Fiction",
    "The Complete Plays of Gilbert and Sullivan": "Drama/Plays Collection",
    "Four Arthurian Romances": "Fiction, Arthurian Legend",
    "The Clicking of Cuthbert": "Fiction, Humorous Short Stories",
    "Tobacco Leaves: Being a Book of Facts for Smokers": "History/Health, Non-Fiction",
    "Fort Amity": "Historical Fiction, Novel",
    "A General View of Positivism: Or, Summary exposition of the System of Thought and Life": "Philosophy, Non-Fiction",
    "Memoirs": "Autobiography, Non-Fiction",
    "Language: Its Nature, Development and Origin": "Linguistics, Non-Fiction",
    "History of Florence and of the Affairs of Italy: From the Earliest Times to the Death of Lorenzo the Magnificent": "History, Non-Fiction",
    "The Great American Fraud: The Patent Medicine Evil": "History/Medicine, Non-Fiction",
    "A history of the Peninsular War, Vol. 5, Oct. 1811-Aug. 31, 1812 : $b Valencia, Ciudad Rodrigo, Badajoz, Salamanca, Madrid": "History/Military, Non-Fiction",
    "Architecture : $b nineteenth and twentieth centuries": "Architecture/History, Non-Fiction",
    "The movie boys in the jungle : $b or, Lively times among the wild beasts": "Adventure Fiction, Youth Novel",
    "The Task of Social Hygiene": "Sociology/Health, Non-Fiction",
    "The Diary of John Evelyn (Volume 2 of 2)": "Autobiography/Diary, Non-Fiction",
    "The history of fashion in France : $b or, the dress of women from the Gallo-Roman period to the present time": "History/Fashion, Non-Fiction",
    "Library of the World's Best Mystery and Detective Stories": "Mystery Fiction, Anthology",
    "The best short stories of 1917, and the yearbook of the American short story": "Fiction, Short Stories Collection",
    "The Journal of a Disappointed Man": "Autobiography, Non-Fiction",
    "A collection of short-stories": "Fiction, Short Stories Collection",
    "Unknown to History: A Story of the Captivity of Mary of Scotland": "Historical Fiction, Novel",
    "The history of steam navigation": "History/Technology, Non-Fiction",
    "Telling Fortunes By Tea Leaves: How to Read Your Fate in a Teacup": "Occult/Guide, Non-Fiction",
    "A Child's History of the World": "History, Non-Fiction for Children",
    "The Red Seal": "Mystery Fiction, Novel",
    "The hero of the Filipinos : $b The story of José Rizal, poet, patriot and martyr": "Biography/History, Non-Fiction",
    "Fictitious & Symbolic Creatures in Art: With Special Reference to Their Use in British Heraldry": "Art/History, Non-Fiction",
    "The Amores; or, Amours: Literally Translated into English Prose, with Copious Notes": "Poetry/Erotica, Translation",
    "Life and Letters of Thomas Cromwell, Vol. 1 of 2: Life, Letters to 1535": "Biography/Letters, Non-Fiction",
    "The Kipling Reader: Selections from the Books of Rudyard Kipling": "Literary Anthology",
    "Selections from the Table Talk of Martin Luther": "Religion/Philosophy, Non-Fiction",
    "Highways and Byways in Surrey": "Travel/History, Non-Fiction",
    "The History of the Thirteen Colonies of North America 1497-1763": "History, Non-Fiction",
    "The Georgics": "Poetry, Didactic Verse",
    "Colour in the flower garden": "Gardening/Guide, Non-Fiction",
    "The Principles of Masonic Law: A Treatise on the Constitutional Laws, Usages and Landmarks of Freemasonry": "Occult/History, Non-Fiction",
    "Farnham's Travels in the Great Western Prairies, etc., part 1, May 21-October 16, 1839": "Travel/History, Non-Fiction",
    "Henley's Twentieth Century Formulas, Recipes and Processes": "Science/Reference, Non-Fiction",
    "History of the Philippine Islands": "History, Non-Fiction",
    "The Life and Letters of Mary Wollstonecraft Shelley, Volume 1 (of 2)": "Biography/Letters, Non-Fiction",
    "Armenia and the Armenians: A List of References in the New York Public Library": "Reference/Bibliography, Non-Fiction",
    "The Officer's Manual: Napoleon's Maxims of War": "Military/Strategy, Non-Fiction",
    "My own story": "Autobiography, Non-Fiction",
    "Give Me Liberty or Give Me Death": "History/Politics, Speech/Non-Fiction",
    "The Five Great Philosophies of Life": "Philosophy, Non-Fiction",
    "Tacitus on Germany": "History, Non-Fiction",
    "The Letters of Henry James (Vol. II)": "Biography/Letters, Non-Fiction",
    "The Burgess Animal Book for Children": "Children's Fiction, Educational/Animal Tales",
    "Damn! A Book of Calumny": "Essays/Satire, Non-Fiction",
    "Charles Sumner: his complete works, volume 20 (of 20)": "Politics/Essays, Non-Fiction",
    "Tales of King Arthur and the Round Table, Adapted from the Book of Romance": "Children's Fiction, Arthurian Legend",
    "The Secrets of the Self (Asrar-i Khudi) — A Philosophical Poem": "Poetry/Philosophy",
    "My Life as an Author": "Autobiography/Literary, Non-Fiction",
    "Criminal Man, According to the Classification of Cesare Lombroso": "Criminology/Psychology, Non-Fiction",
    "The Stars and Stripes, Vol. 1, No. 1, February 8, 1918: The American Soldiers' Newspaper of World War I, 1918-1919": "History/Journalism, Non-Fiction",
    "Fine Books": "History/Bibliography, Non-Fiction",
    "Martine's Hand-book of Etiquette, and Guide to True Politeness": "Self-Help/Etiquette, Non-Fiction",
    "Winesburg, Ohio: A Group of Tales of Ohio Small Town Life": "Fiction, Short Stories",
    "The inquisition in the Spanish dependencies: Sicily—Naples—Sardinia—Milan—the Canaries—Mexico—Peru—New Granada": "History/Religion, Non-Fiction",
    "Wives of the Prime Ministers, 1844-1906": "Biography/History, Non-Fiction",
    "Astrology: How to Make and Read Your Own Horoscope": "Occult/Guide, Non-Fiction",
    "A summer journey to Brazil": "Travel, Non-Fiction",
    "The History of the Rise, Progress and Accomplishment of the Abolition of the African Slave-Trade, by the British Parliament (1839)": "History, Non-Fiction",
    "Thomas Jefferson, the Apostle of Americanism": "Biography/History, Non-Fiction",
    "The House on the Borderland": "Horror Fiction, Novel",
    "Theodicy: Essays on the Goodness of God, the Freedom of Man and the Origin of Evil": "Philosophy/Religion, Non-Fiction",
    "Fungi: Their Nature and Uses": "Science/Botany, Non-Fiction",
    "Studies in the Psychology of Sex, Volume 2: Sexual Inversion": "Psychology/Sexology, Non-Fiction",
    "Memoirs of the Life and Correspondence of Henry Reeve, C.B., D.C.L.: In Two Volumes. Volume II.": "Biography/Letters, Non-Fiction",
    "Gregg's Commerce of the Prairies, 1831-1839, part 2": "Travel/History, Non-Fiction",
    "Gallio : $b or, The tyranny of science": "Fiction, Novel",
    "Mark Twain: A Biography. Complete": "Biography, Non-Fiction",
    "The Arts in the Middle Ages and at the Period of the Renaissance": "Art/History, Non-Fiction",
    "My life in Sarawak": "Autobiography/Travel, Non-Fiction",
    "The gradual acceptance of the Copernican theory of the universe": "Science/History, Non-Fiction",
    "The Odes and Carmen Saeculare of Horace": "Poetry Collection, Translation",
    "The Washington Historical Quarterly, Volume V, 1914": "History, Non-Fiction Journal",
    "Complete Short Works of George Meredith": "Fiction, Short Stories Collection",
    "The Agrarian Problem in the Sixteenth Century": "History/Economics, Non-Fiction",
    "The Autobiography of Goethe: Truth and Poetry: From My Own Life": "Autobiography, Non-Fiction",
    "The Archaeology of the Yakima Valley": "Archaeology/History, Non-Fiction",
    "Castles and Chateaux of Old Navarre and the Basque Provinces": "History/Travel, Non-Fiction",
    "Amphitryo, Asinaria, Aulularia, Bacchides, Captivi: Amphitryon, The Comedy of Asses, The Pot of Gold, The Two; Bacchises, The Captives": "Drama/Plays Collection",
    "The history of Sumatra : $b Containing an account of the government, laws, customs and manners of the native inhabitants": "Travel/History, Non-Fiction",
    "Greener Than You Think": "Science Fiction, Novel",
    "Endymion: A Poetic Romance": "Poetry, Narrative Verse",
    "The Hand in the Dark": "Mystery Fiction, Novel",
    "Engraving: Its Origin, Processes, and History": "Art/History, Non-Fiction",
    "The Christ Myth": "Religion/History, Non-Fiction",
    "An Apology for the Life of Mr. Colley Cibber, Volume 1 (of 2): Written by Himself. A New Edition with Notes and Supplement": "Autobiography/Theater, Non-Fiction",
    "Portugal of the Portuguese": "History/Cultural, Non-Fiction",
    "The Boy Mechanic, Book 2: 1000 Things for Boys to Do": "Education/Crafts, Non-Fiction",
    "Superstition in all ages": "Philosophy/Religion Criticism, Non-Fiction",
    "The Present Condition of Organic Nature: Lecture I. (of VI.), 'Lectures to Working Men', at the Museum of Practical Geology, 1863, on Darwin's Work: 'Origin of Species'": "Science/Biology, Non-Fiction",
    "Knickerbocker's History of New York, Complete": "History/Satire, Non-Fiction",
    "Hymns for Christian Devotion: Especially Adapted to the Universalist Denomination": "Religion/Music, Non-Fiction",
    "The Alhambra": "Travel/History, Non-Fiction",
    "Nonsense Books": "Children's Poetry/Humor Collection",
    "French Book-plates": "Art/History, Non-Fiction",
    "Japan: From the Japanese Government History": "History/Cultural, Non-Fiction",
    "Frederick William Maitland, Downing Professor of the Laws of England: A Biographical Sketch": "Biography/Law, Non-Fiction",
    "The Green Mummy": "Mystery Fiction, Novel",
    "Contemporary American Composers: Being a Study of the Music of This Country, Its Present; Conditions and Its Future, with Critical Estimates and; Biographies of the Principal Living Composers; and an; Abundance of Portraits, Fac-simile Musical Autographs, and; Compositions": "Music/Biography, Non-Fiction",
    "On the magnet, magnetick bodies also, and on the great magnet the earth: a new physiology, demonstrated by many arguments & experiments": "Science/Physics, Non-Fiction",
    "The Man Who Did the Right Thing: A Romance": "Fiction, Romance Novel",
    "The game of life and how to play it": "Self-Help/Spirituality, Non-Fiction",
    "Jane Austen, Her Life and Letters: A Family Record": "Biography/Literary, Non-Fiction",
    "The Czar's Spy: The Mystery of a Silent Love": "Mystery Fiction, Novel",
    "The Satyricon — Volume 02: Dinner of Trimalchio": "Fiction, Satirical Novel",
    "Sweden": "History/Travel, Non-Fiction",
    "The American Diary of a Japanese Girl": "Fiction, Diary Novel",
    "A Diplomat in Japan: The inner history of the critical years in the evolution of Japan when the ports were opened and the monarchy restored, recorded by a diplomatist who took an active part in the events of the time, with an account of his personal experiences during that period": "History/Autobiography, Non-Fiction",
    "The Diamond Sutra (Chin-Kang-Ching) or Prajna-Paramita": "Religion/Philosophy, Non-Fiction",
    "Morals and Dogma of the Ancient and Accepted Scottish Rite of Freemasonry": "Occult/Philosophy, Non-Fiction",
    "Joseph Andrews, Vol. 1": "Fiction, Picaresque Novel",
    "The Violoncello and Its History": "Music/History, Non-Fiction",
    "The Revolutionary Movement of 1848-9 in Italy, Austria-Hungary, and Germany: With Some Examination of the Previous Thirty-three Years": "History, Non-Fiction",
    "The Articles of Faith: A Series of Lectures on the Principal Doctrines of the Church of Jesus Christ of Latter-Day Saints": "Religion/Theology, Non-Fiction",
    "The Landloper: The Romance of a Man on Foot": "Fiction, Romance Novel",
    "Only a girl's love": "Fiction, Romance Novel",
    "The Awakening of Spring: A Tragedy of Childhood": "Drama/Play, Tragedy",
    "The Queen of Spades, and other stories": "Fiction, Short Stories",
    "The Martian: A Novel": "Science Fiction, Novel",
    "Main Currents in Nineteenth Century Literature - 6. Young Germany": "Literary Criticism/History, Non-Fiction",
    "The Tragedies of Euripides, Volume I.": "Drama/Plays Collection, Tragedy",
    "The Conjure Woman": "Fiction, Short Stories/Folklore",
    "Copyright: Its History and Its Law": "Law/History, Non-Fiction",
    "Ole Bull: A Memoir": "Biography/Music, Non-Fiction",
    "The Spanish Tragedie": "Drama/Play, Tragedy",
    "The Mysteries of Free Masonry: Containing All the Degrees of the Order Conferred in a Master's Lodge": "Occult/Reference, Non-Fiction",
    "Micromegas": "Fiction, Satirical Short Story",
    "Romain Rolland: The Man and His Work": "Biography/Literary Criticism, Non-Fiction",
    "Christopher Columbus and His Monument Columbia: being a concordance of choice tributes to the great Genoese, his grand discovery, and his greatness of mind and purpose": "History/Biography, Non-Fiction",
    "Studies in the Scriptures, Volume 7: The Finished Mystery": "Religion/Commentary, Non-Fiction",
    "The Indolence of the Filipino": "History/Politics, Non-Fiction Essay",
    "The Children of Odin: The Book of Northern Myths": "Children's Fiction, Mythology Retellings",
    "Cleopatra's Needle: A History of the London Obelisk, with an Exposition of the Hieroglyphics": "History/Archaeology, Non-Fiction",
    "A Literary and Historical Atlas of Asia": "Reference/History, Non-Fiction",
    "Pamela, or Virtue Rewarded": "Fiction, Epistolary Novel",
    "Historical records of the Buffs, East Kent Regiment (3rd Foot) : $b Formerly designated the Holland Regiment and Prince George of Denmark's Regiment": "History/Military, Non-Fiction",
    "Records of a Girlhood": "Autobiography, Non-Fiction",
    "The History of Woman Suffrage, Volume VI": "History/Feminism, Non-Fiction",
    "A History of Rome to 565 A. D.": "History, Non-Fiction",
    "Villani's Chronicle: Being Selections from the First Nine Books of the Croniche Fiorentine of Giovanni Villani": "History, Non-Fiction",
    "The Story of the Sun: New York, 1833-1918": "History/Journalism, Non-Fiction",
    "Astronomy for Amateurs": "Science/Astronomy, Non-Fiction",
    "The American Indian as Slaveholder and Seccessionist: An Omitted Chapter in the Diplomatic History of the Southern Confederacy": "History, Non-Fiction",
    "Minute Mysteries [Detectograms]": "Mystery Fiction, Puzzles/Short Stories",
    "The Home of the Blizzard: Being the Story of the Australasian Antarctic Expedition, 1911-1914": "Travel/Exploration, Non-Fiction",
    "Rasselas, Prince of Abyssinia": "Fiction, Philosophical Novel",
    "English Verse: Specimens Illustrating its Principles and History": "Poetry/Literary Criticism, Non-Fiction",
    "The Old English Herbals": "History/Botany, Non-Fiction",
    "Traditions, Superstitions and Folk-lore: (Chiefly Lancashire and the North of England:) Their Affinity to Others in Widely-Distributed Localities; Their Eastern Origin and Mythical Significance.": "Folklore, Non-Fiction",
    "Curious Epitaphs, Collected from the Graveyards of Great Britain and Ireland.": "History/Folklore, Non-Fiction",
    "Letters of Two Brides": "Fiction, Epistolary Novel",
    "French Idioms and Proverbs: A Companion to Deshumbert's 'Dictionary of Difficulties'": "Reference/Language, Non-Fiction",
    "Introduction to the History of Religions: Handbooks on the History of Religions, Volume IV": "Religion/Anthropology, Non-Fiction",
    "Chippinge Borough": "Fiction, Novel",
    "Nooks and Corners of Lancashire and Cheshire.: A Wayfarer's Notes in the Palatine Counties, Historical, Legendary, Genealogical, and Descriptive.": "Travel/History, Non-Fiction",
    "The Secret of the Ninth Planet": "Science Fiction, Novel",
    "News from Nowhere; Or, An Epoch of Rest: Being Some Chapters from a Utopian Romance": "Fiction, Utopian Novel",
    "Paul Clifford — Complete": "Fiction, Novel",
    "A Dictionary of the First or Oldest Words in the English Language: From the Semi-Saxon Period of A.D. 1250 to 1300": "Reference/Linguistics, Non-Fiction",
    "The Complete Works of Robert Burns: Containing his Poems, Songs, and Correspondence.: With a New Life of the Poet, and Notices, Critical and Biographical by Allan Cunningham": "Poetry Collection",
    "Caravans to Santa Fe": "History/Travel, Non-Fiction",
    "Gods and Fighting Men: The story of the Tuatha de Danaan and of the Fianna of Ireland, arranged and put into English by Lady Gregory": "Folklore/Mythology, Non-Fiction",
    "The Cathedral Builders: The Story of a Great Masonic Guild": "History/Architecture, Non-Fiction",
    "Canyons of the Colorado": "Travel/Exploration, Non-Fiction",
    "East of Suez: Ceylon, India, China and Japan": "Travel, Non-Fiction",
    "Lives of the Most Eminent Painters, Sculptors and Architects, Vol. 09 (of 10): Michelagnolo to the Flemings": "Biography/Art History, Non-Fiction",
    "Selected Works of Voltairine de Cleyre": "Essays/Politics, Non-Fiction",
    "Langley Memoir on Mechanical Flight, Parts I and II: Smithsonian Contributions to Knowledge, Volume 27 Number 3, Publication 1948, 1911": "Science/Aviation, Non-Fiction",
    "Translations of Shakuntala and Other Works": "Drama/Literature, Translation",
    "The Love Letters of Mary Wollstonecraft to Gilbert Imlay": "Biography/Letters, Non-Fiction",
    "Celtic Folk and Fairy Tales": "Folklore/Fiction, Short Stories Collection",
    "That Which Hath Wings: A Novel of the Day": "Fiction, Novel",
    "The United States Bill of Rights: The Ten Original Amendments to the Constitution of the United States": "History/Politics, Non-Fiction",
    "Views of nature: or Contemplations on the sublime phenomena of creation: with scientific illustrations": "Science/Nature, Non-Fiction",
    "Inspiration and Interpretation: Seven Sermons Preached Before the University of Oxford": "Religion/Sermons, Non-Fiction",
    "Young People's Pride: A Novel": "Fiction, Novel",
    "A Life of William Shakespeare: with portraits and facsimiles": "Biography/Literary Criticism, Non-Fiction",
    "Curiosities of Great Britain: England and Wales Delineated Vol.1-11: Historical, Entertaining & Commercial; Alphabetically Arranged. 11 Volume set.": "History/Reference, Non-Fiction",
    "Tales of Unrest": "Fiction, Short Stories",
    "The Book of Dragons": "Children's Fiction, Fantasy Short Stories",
    "The ugly-girl papers : $b or, Hints for the toilet": "Self-Help/Beauty, Non-Fiction",
    "The Journal of Negro History, Volume 7, 1922": "History, Non-Fiction Journal",
    "Nursing as Caring: A Model for Transforming Practice": "Medicine/Nursing, Non-Fiction",
    "Palace and Mosque at Ukhaidir: A Study in Early Mohammadan Architecture": "Architecture/History, Non-Fiction",
    "Legends, Tales and Poems": "Poetry/Folklore Collection",
    "The Germ: Thoughts towards Nature in Poetry, Literature and Art": "Literary Criticism/Poetry, Non-Fiction",
    "A Journey from Prince of Wales's Fort in Hudson's Bay to the Northern Ocean in the Years 1769, 1770, 1771, 1772: New Edition with Introduction, Notes, and Illustrations": "Travel/Exploration, Non-Fiction",
    "The Dunwich horror": "Horror Fiction, Short Story",
    "Indian Fairy Tales": "Folklore/Fiction, Short Stories Collection",
    "The Epic of Gilgamish: A Fragment of the Gilgamish Legend in Old-Babylonian Cuneiform": "Mythology/Poetry, Translation",
    "Primitive culture, vol. 1 (of 2) : $b Researches into the development of mythology, philosophy, religion, language, art, and custom": "Anthropology, Non-Fiction",
    "The Legends and Myths of Hawaii: The fables and folk-lore of a strange people": "Folklore/Mythology, Non-Fiction",
    "Poems by William Cullen Bryant": "Poetry Collection",
    "The Story of the American Merchant Marine": "History/Business, Non-Fiction",
    "Lands of the Slave and the Free; Or, Cuba, the United States, and Canada": "Travel/History, Non-Fiction",
    "The new freedom : $b a call for the emancipation of the generous energies of a people": "Politics, Non-Fiction",
    "Varney the Vampire; Or, the Feast of Blood": "Horror Fiction, Novel",
    "The Athenian Constitution": "History/Politics, Non-Fiction",
    "Villainage in England: Essays in English Mediaeval History": "History, Non-Fiction",
    "The Complete Opera Book: The Stories of the Operas, together with 400 of the Leading Airs and Motives in Musical Notation": "Music/Reference, Non-Fiction",
    "Simon the Jester": "Fiction, Novel",
    "Annals of Music in America: A Chronological Record of Significant Musical Events": "Music/History, Non-Fiction",
    "Chronological Retrospect of the History of Yarmouth and Neighbourhood: from A.D. 46 to 1884": "History, Non-Fiction",
    "Linnet: A Romance": "Fiction, Romance Novel",
    "Discoveries Among the Ruins of Nineveh and Babylon": "Archaeology/Travel, Non-Fiction",
    "My Experiences in Manipur and the Naga Hills": "Autobiography/Travel, Non-Fiction",
    "The Travels of Sir John Mandeville": "Travel/Fiction, Medieval",
    "The Renaissance: Studies in Art and Poetry": "Art/Literary Criticism, Non-Fiction",
    "The Traditional Games of England, Scotland, and Ireland (Vol 2 of 2): With Tunes, Singing-Rhymes, and Methods of Playing etc.": "Folklore/Games, Non-Fiction",
    "Shelters, Shacks and Shanties": "Instruction/Outdoor, Non-Fiction",
    "The City of Auckland, New Zealand, 1840-1920": "History, Non-Fiction",
    "Fredericksburg and Its Many Points of Interest": "History/Travel, Non-Fiction",
    "A Week's Tramp in Dickens-Land: Together with Personal Reminiscences of the 'Inimitable Boz' Therein Collected": "Travel/Literary, Non-Fiction",
    "On the Spanish Main; Or, Some English forays on the Isthmus of Darien": "History/Adventure, Non-Fiction",
    "Histories of two hundred and fifty-one divisions of the German army which participated in the war (1914-1918)": "History/Military, Non-Fiction",
    "To Have and to Hold": "Historical Fiction, Novel",
    "The Book of Buried Treasure: Being a True History of the Gold, Jewels, and Plate of Pirates, Galleons, etc., which are sought for to this day": "History/Adventure, Non-Fiction",
    "A Guide to Health": "Health/Self-Help, Non-Fiction",
    "Chess Strategy": "Games/Strategy, Non-Fiction",
    "The Odes of Casimire, Translated by G. Hils": "Poetry Collection, Translation",
    "Too old to fly": "Fiction, Novel",
    "The Path to Rome": "Travel/Philosophy, Non-Fiction",
    "The Damned Thing: 1898, From 'In the Midst of Life'": "Horror Fiction, Short Story",
    "Know the Truth: A Critique on the Hamiltonian Theory of Limitation: Including Some Strictures Upon the Theories of Rev. Henry L. Mansel and Mr. Herbert Spencer": "Philosophy, Non-Fiction",
    "A Martian Odyssey": "Science Fiction, Short Story",
    "Ancient Armour and Weapons in Europe: From the Iron Period of the Northern Nations to the End of the Thirteenth Century": "History/Military, Non-Fiction",
    "The Dean of Lismore's Book: A Selection of Ancient Gaelic Poetry": "Poetry Collection",
    "Mr. Honey's Banking Dictionary (German-English)": "Reference/Dictionary, Non-Fiction",
    "Worcestershire in the Nineteenth Century: A Complete Digest of Facts Occuring in the County since the Commencement of the year 1800": "History, Non-Fiction",
    "History of the Forty-Eighth Regiment M. V. M. During the Civil War": "History/Military, Non-Fiction",
    "Napoleon's Marshals": "Biography/Military, Non-Fiction",
    "Lancelot of the Laik: A Scottish Metrical Romance (About 1490-1500 A. D.)": "Poetry, Romance",
    "Stories of Old Greece and Rome": "Children's Fiction, Mythology Retellings",
    "The Mechanical Properties of Wood: Including a Discussion of the Factors Affecting the Mechanical Properties, and Methods of Timber Testing": "Science/Engineering, Non-Fiction",
    "The Reason Why: A Careful Collection of Many Hundreds of Reasons for Things Which, Though Generally Believed, Are Imperfectly Understood": "Reference/Curiosities, Non-Fiction",
    "A Middle High German Primer: Third Edition": "Education/Language, Non-Fiction",
    "The Romance of Words (4th ed.)": "Linguistics/History, Non-Fiction",
    "How to Enjoy Paris in 1842: Intended to Serve as a Companion and Monitor, Containing; Historical, Political, Commercial, Artistical, Theatrical; And Statistical Information": "Travel/Guide, Non-Fiction",
    "Nationalism": "Politics/Philosophy, Non-Fiction",
    "The Literature of the Ancient Egyptians": "Literary Criticism/History, Non-Fiction",
    "The Man with Two Left Feet, and Other Stories": "Fiction, Short Stories",
    "Astounding Stories, July, 1931": "Science Fiction, Magazine/Short Stories",
    "Sagittulae, Random Verses": "Poetry Collection",
    "Beowulf: An Introduction to the Study of the Poem with a Discussion of the Stories of Offa and Finn": "Literary Criticism/Poetry, Non-Fiction",
    "Ireland under the Stuarts and During the Interregnum, Vol. 1 (of 3), 1603-1642": "History, Non-Fiction",
    "Southern Literature From 1579-1895: A comprehensive review, with copious extracts and criticisms; for the use of schools and the general reader": "Literary Criticism/History, Non-Fiction",
    "An Etymological Dictionary of the Scottish Language: in which the words are explained in their different senses, authorized by the names of the writers by whom they are used, or the titles of the works in which they occur, and deduced from their originals": "Reference/Linguistics, Non-Fiction",
    "Rambles and Recollections of an Indian Official": "Travel/Autobiography, Non-Fiction",
    "Club Life of London, Vol. 1 (of 2): With Anecdotes of the Clubs, Coffee-Houses and Taverns of the Metropolis During the 17th, 18th, and 19th Centuries": "History/Social, Non-Fiction",
    "The Invention of the Sewing Machine": "History/Technology, Non-Fiction",
    "The Bible Story": "Religion/Children's, Non-Fiction",
    "The History of Painting in Italy, Vol. 3 (of 6): From the Period of the Revival of the Fine Arts to the End of the Eighteenth Century": "Art/History, Non-Fiction",
    "Korean Fairy Tales": "Folklore/Fiction, Short Stories Collection",
    "History of circumcision from the earliest times to the present : $b Moral and physical reasons for its performance, with a history of eunuchism, hermaphrodism, etc., and of the different operations practiced upon the prepuce": "Medicine/History, Non-Fiction",
    "Pagan Papers": "Essays/Humor, Non-Fiction",
    "New lives for old": "Fiction, Novel",
    "Anthony Trollope; His Work, Associates and Literary Originals": "Biography/Literary Criticism, Non-Fiction",
    "Devil Stories: An Anthology": "Fiction, Short Stories/Horror",
    "The Water-Babies: A Fairy Tale for a Land-Baby": "Children's Fiction, Fantasy Novel",
    "Norfolk Annals, Vol. 2: A Chronological Record of Remarkable Events in the Nineteeth Century": "History, Non-Fiction",
    "History of Brighthelmston; or, Brighton as I View it and Others Knew It: With a Chronological Table of Local Events": "History, Non-Fiction",
    "The Coming Race": "Science Fiction, Utopian Novel",
    "Memoir of Jane Austen": "Biography/Literary, Non-Fiction",
    "Laurence Sterne in Germany: A Contribution to the Study of the Literary Relations of; England and Germany in the Eighteenth Century": "Literary Criticism/History, Non-Fiction",
    "Narrative of the Circumnavigation of the Globe by the Austrian Frigate Novara, Volume II: (Commodore B. Von Wullerstorf-Urbair,) Undertaken by Order of the Imperial Government in the Years 1857, 1858, & 1859, Under the Immediate Auspices of His I. and R. Highness the Archduke Ferdinand Maximilian, Commander-In-Chief of the; Austrian Navy.": "Travel/Exploration, Non-Fiction",
    "Old Greek Stories": "Children's Fiction, Mythology Retellings",
    "A Budget of Paradoxes, Volume II": "Mathematics/History, Non-Fiction",
    "The Japanese Spirit": "Philosophy/Cultural, Non-Fiction",
    "History of the Wars, Books III and IV: The Vandalic War": "History/Military, Non-Fiction",
    "A History of Matrimonial Institutions, Vol. 3 of 3": "History/Sociology, Non-Fiction",
    "Virginia's Attitude Toward Slavery and Secession": "History, Non-Fiction",
    "The History of Modern Painting, Volume 3 (of 4): Revised edition continued by the author to the end of the XIX century": "Art/History, Non-Fiction",
    "Alexandria: A History and a Guide": "History/Travel, Non-Fiction",
    "The Life of Cesare Borgia": "Biography/History, Non-Fiction",
    "Sailing Alone Around the World": "Travel/Autobiography, Non-Fiction",
    "Emily Brontë": "Biography/Literary Criticism, Non-Fiction",
    "Humorous Ghost Stories": "Horror Fiction, Short Stories Collection",
    "Petrarch's Secret; or, the Soul's Conflict with Passion: Three Dialogues Between Himself and S. Augustine": "Philosophy/Religion, Non-Fiction",
    "The Life and Work of Susan B. Anthony (Volume 1 of 2): Including Public Addresses, Her Own Letters and Many From Her Contemporaries During Fifty Years": "Biography/Feminism, Non-Fiction",
    "The Biotic Associations of Cockroaches": "Science/Entomology, Non-Fiction",
    "The Deipnosophists; or, Banquet of the Learned of Athenæus, Vol. 1 (of 3)": "Philosophy/History, Non-Fiction",
    "Mad Shepherds, and Other Human Studies": "Essays/Social, Non-Fiction",
    "A Critical Exposition of the Popular 'Jihád': Showing that all the Wars of Mohammad Were Defensive; and; that Aggressive War, or Compulsory Conversion, is not; Allowed in The Koran - 1885": "Religion/History, Non-Fiction",
    "A History of Horncastle, from the earliest period to the present time": "History, Non-Fiction",
    "The Ornithology of Shakespeare: Critically examined, explained and illustrated": "Literary Criticism/Nature, Non-Fiction",
    "The Three Brontës": "Biography/Literary Criticism, Non-Fiction",
    "The Englishman in China During the Victorian Era, Vol. 2 (of 2): As Illustrated in the Career of Sir Rutherford Alcock, K.C.B., D.C.L., Many Years Consul and Minister in China and Japan": "History/Biography, Non-Fiction",
    "A History of Magic and Experimental Science, Volume 1 (of 2): During the First Thirteen Centuries of Our Era": "History/Science, Non-Fiction",
    "Lone Star Planet": "Science Fiction, Novel",
    "Through South Africa: His Visit to Rhodesia, the Transvaal, Cape Colony, Natal": "Travel/History, Non-Fiction",
    "The Initiates of the Flame": "Occult/Philosophy, Non-Fiction",
    "Tea-Cup Reading and Fortune-Telling by Tea Leaves, by a Highland Seer": "Occult/Guide, Non-Fiction",
    "The Ship of Fools, Volume 1": "Satire/Poetry",
    "The Veil Lifted: Modern Developments of Spirit Photography": "Occult/Photography, Non-Fiction",
    "Mr. Honey's Medium Business Dictionary (German-English)": "Reference/Dictionary, Non-Fiction",
    "Lillian's vow : $b or, The mystery of Raleigh House": "Fiction, Novel",
    "Dainty's Cruel Rivals; Or, The Fatal Birthday": "Fiction, Romance Novel",
    "The Melting-Pot": "Drama/Play",
    "Clicking red heels": "Fiction, Novel",
    "An Outline of English Speech-craft": "Linguistics, Non-Fiction",
    "The Theory of the Leisure Class": "Sociology/Economics, Non-Fiction",
    "Letters on England": "Essays/Travel, Non-Fiction",
    "The Oxford Reformers: John Colet, Erasmus, and Thomas More": "Biography/History, Non-Fiction",
    "The Modern Regime, Volume 1": "History/Politics, Non-Fiction",
    "Handbook of Birmingham: Prepared for the Members of the British Association, 1886": "History/Guide, Non-Fiction",
    "The Conspiracy of Gianluigi Fieschi, or, Genoa in the sixteenth century.": "History, Non-Fiction",
    "The rat and its relation to the public health": "Science/Medicine, Non-Fiction",
    "A Pocket Dictionary: Welsh-English": "Reference/Dictionary, Non-Fiction",
}

# === Add a new column 'genre' by mapping from title ===
df['genre'] = df['title'].map(genre_map)

# === Reorder columns so 'genre' is right after 'text' ===
col_order = ['author', 'title', 'text', 'genre', 'text_type', 'word_count']
df = df[col_order]

# === Save updated dataset (overwrite the same file) ===
df.to_csv("author_identification_dataset_final.csv", index=False, encoding="utf-8")

print("✅ Added 'genre' column to author_identification_dataset_final.csv")
print(df.head(10))

✅ Added 'genre' column to author_identification_dataset_final.csv
            author                                              title  \
0  Abraham Lincoln                                    Lincoln Letters   
1  Abraham Lincoln                  Lincoln's First Inaugural Address   
2  Abraham Lincoln  Lincoln's Gettysburg Address, given November 1...   
3  Abraham Lincoln  Lincoln's Inaugurals, Addresses and Letters (S...   
4  Abraham Lincoln                 Lincoln's Second Inaugural Address   
5  Abraham Lincoln  Speeches and Letters of Abraham Lincoln, 1832-...   
6  Abraham Lincoln                       State of the Union Addresses   
7  Abraham Lincoln                      The Emancipation Proclamation   
8  Abraham Lincoln  The Life and Public Service of General Zachary...   
9  Abraham Lincoln  The Writings of Abraham Lincoln, Volume 1_ 183...   

                                                text  \
0  LINCOLN LETTERS By Abraham Lincoln Published b...   
1  Lincoln's First

In [9]:
import pandas as pd

preview = pd.read_csv("author_identification_dataset_final.csv")

preview.head()

,author,title,text,genre,text_type,word_count
0,Abraham Lincoln,Lincoln Letters,LINCOLN LETTERS By Abraham Lincoln Published b...,"History/Politics, Non-Fiction",letters,1065
1,Abraham Lincoln,Lincoln's First Inaugural Address,"Lincoln's First Inaugural Address March 4, 186...","History/Politics, Speech/Non-Fiction",letters,3626
2,Abraham Lincoln,"Lincoln's Gettysburg Address, given November 1...","Lincoln's Gettysburg Address, given November 1...","History/Politics, Speech/Non-Fiction",letters,299
3,Abraham Lincoln,"Lincoln's Inaugurals, Addresses and Letters (S...",Longman's English Classics LINCOLN'S INAUGURAL...,"History/Politics, Non-Fiction Collection",letters,43649
4,Abraham Lincoln,Lincoln's Second Inaugural Address,"Lincoln's Second Inaugural Address March 4, 18...","History/Politics, Speech/Non-Fiction",letters,703


In [12]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# -------------------------
# 1. Read CSV (pandas)
# -------------------------
df = pd.read_csv("author_identification_dataset_final.csv")

# Use 'title' as book_id to avoid leakage
df = df[['author', 'title', 'text', 'genre']].rename(columns={'title': 'book_id'})

# -------------------------
# 2. Parameters
# -------------------------
MIN_LEN = 1000
CHUNK_SIZE = 2000
MAX_CHUNKS_PER_AUTHOR = 1000

# -------------------------
# 3. Chunking function
# -------------------------
def chunk_text(text, chunk_size=CHUNK_SIZE, min_len=MIN_LEN):
    text = str(text)
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return [c for c in chunks if len(c) >= min_len]

# -------------------------
# 4. Apply chunking across dataframe
# -------------------------
rows = []
for _, row in df.iterrows():
    chunks = chunk_text(row['text'])
    for chunk in chunks:
        rows.append({
            'author': row['author'],
            'book_id': row['book_id'],
            'genre': row['genre'],  
            'chunk': chunk
        })

df_chunks = pd.DataFrame(rows)
print("Total chunks before balance (raw dataset):", len(df_chunks))

# -------------------------
# Save the full raw 1.5M dataset
# -------------------------
os.makedirs("author_chunks_dataset", exist_ok=True)
df_chunks.to_parquet("author_chunks_dataset/full_dataset.parquet", index=False)
df_chunks.to_csv("author_chunks_dataset/full_dataset.csv", index=False)
print("Saved full raw dataset (1.5M chunks)")

# -------------------------
# 5. Cap per author to avoid imbalance
# -------------------------
df_chunks['rn'] = df_chunks.groupby('author').cumcount() + 1
df_balanced = df_chunks[df_chunks['rn'] <= MAX_CHUNKS_PER_AUTHOR].drop(columns=['rn'])
print("After capping per author (balanced dataset):", len(df_balanced))

# -------------------------
# 6. Train/Dev/Test split by book_id (no leakage)
# -------------------------
unique_books = df_balanced['book_id'].unique()
train_books, test_books = train_test_split(unique_books, test_size=0.3, random_state=42)
dev_books, test_books = train_test_split(test_books, test_size=0.5, random_state=42)

def assign_split(book_id):
    if book_id in train_books: return 'train'
    if book_id in dev_books: return 'dev'
    return 'test'

df_balanced['split'] = df_balanced['book_id'].map(assign_split)
print("Balanced dataset split counts:\n", df_balanced['split'].value_counts())

# -------------------------
# 7. Save splits as Parquet AND CSV
# -------------------------
for split in ['train', 'dev', 'test']:
    split_df = df_balanced[df_balanced['split'] == split]

    # Parquet
    out_parquet = f"author_chunks_dataset/{split}.parquet"
    split_df.to_parquet(out_parquet, index=False)

    # CSV
    out_csv = f"author_chunks_dataset/{split}.csv"
    split_df.to_csv(out_csv, index=False)

    print(f"Saved {split}: {len(split_df)} rows to {out_parquet} and {out_csv}")

Total chunks before balance (raw dataset): 1526225
Saved full raw dataset (1.5M chunks)
After capping per author (balanced dataset): 770655
Balanced dataset split counts:
 split
train    545285
dev      113592
test     111778
Name: count, dtype: int64
Saved train: 545285 rows to author_chunks_dataset/train.parquet and author_chunks_dataset/train.csv
Saved dev: 113592 rows to author_chunks_dataset/dev.parquet and author_chunks_dataset/dev.csv
Saved test: 111778 rows to author_chunks_dataset/test.parquet and author_chunks_dataset/test.csv
